In [1]:
# April, 2021
# This ipynb combines train_CC.py & ProxyParsingForCC.ipynb 
# and creates a list of relevant information 


In [1]:
from __future__ import unicode_literals, print_function
#from spacy.train_new_entity_type import TRAIN_DATA

#import plac
import random
import warnings
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding
from TRAIN_DATA import TRAIN_DATA # train examples are in this py file.
from tqdm.auto import tqdm

import os
import re
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd

import unicodedata
import jellyfish
from ast import literal_eval
import itertools

# to avoid 
set = __builtins__.set

## initial setup

In [2]:
# create a dict of cik and ticker
import urllib.request
page=urllib.request.urlopen('https://www.sec.gov/include/ticker.txt').read().decode('utf-8')
pageList=page.split('\n')
idDict={item.split('\t')[1]: item.split('\t')[0] for item in pageList}

idDict['2488'] # example

'amd'

In [4]:
startFolder=input('which Drive to start with?')
os.chdir(startFolder) # change to each folder of SP500
for folderName, subfolders, filenames in os.walk(startFolder):
    sp500List=[]
    print('The current folder is  ' + folderName)
    print('the # of subfolders is  ' + str(len(subfolders)))    
    for subfolder in subfolders:
          sp500List.append(subfolder)
    if folderName==startFolder: break # prevents os.walk from going through all the other subfolders
try:
    sp500List.remove('$RECYCLE.BIN')
    sp500List.remove('System Volume Information')
    sp500List.remove("sec-edgar-filings") # added on Apr 9, 2021
    sp500List.remove('.System') #added on Aug 5,2020
    sp500List.remove('_SP500_1995-2016-employmentK')
except:
    pass
#bigDict=defaultdict(list)

# creates a list of folder paths.
# sp500List is the list of sp500 firms
pathList=[]
for sp500Firm in sp500List:
    firmName=sp500Firm  # FIRMNAME
    absPath=os.path.join(startFolder, sp500Firm)
    #print(absPath)
    pathList.append(absPath)

len(pathList)

# get generators of folders of SP 500
#folder=(i for i in pathList)

The current folder is  f:\
the # of subfolders is  1541


1536

In [5]:
os.chdir(r'C:\Users\KWan\Documents\NLP & ML\compensation consultant')



def create_path(proxies, firmName, drive='F',):
    paths=[]
    for proxy in proxies:
        # change the drive name where appropriate
        paths.append(os.path.join('F',firmName, proxy))# firm name from listdir()
    return paths   

# integrated function

In [33]:
def find_proxies(dir):
    os.chdir(dir)  # r'F:\ADVANCED MICRO DEVICES INC')
    htmls = os.listdir()
    regex = re.compile('DEF 14')
    proxies = []
    for html in htmls:
        if regex.search(html):
            if html.endswith('html'):
                proxies.append(html)
    print(proxies)
    proxies2007 = []
    regex = re.compile('(\d\d\d\d)-\d\d-\d\d')
    if proxies == None:
        return proxies2007
    else:
        for proxy in proxies:
            try:
                match = int(regex.search(proxy).group(1))
                if match > 2006:
                    proxies2007.append(proxy)
            except AttributeError:
                print(r'no \d\d\d\d-\d\d-\d\d match')

    return proxies2007


In [7]:
def open_proxy1(proxies2007):
    """loop 2007 and after proxies and yield those with 'consultant' hits """
    from bs4 import BeautifulSoup, UnicodeDammit
    import html
    if proxies2007==[]:
        fileList, hitList = None, None
    else:
        #regex=re.compile('committee', re.I)
        fileList=[]
        hitList=[]
        TypeErrorList=[]
        for proxy in proxies2007:
            hit=[]
            try:
                with open(proxy, 'r', encoding='utf8') as f:
                    contents = f.read()
                    bs = BeautifulSoup(contents, 'lxml')
            except TypeError: # when bs4 does not work
                typeErrorList.append(proxy) # record the filename
                #hitList.append(hit) # and return a blank list
                continue
            
            for item in bs.find_all(string=re.compile('consultant', re.I)):

                text=unicodedata.normalize("NFKC", item)
                text=html.unescape(text)
                #text=UnicodeDammit(text, ["windows-1252"], smart_quotes_to="html").unicode_markup
                text=re.sub(r'\n', ' ', text)
                text=re.sub(r'\x92',"'", text) #\x92
                text=re.sub(r'\x93','"', text)
                text=re.sub(r'\x94','"', text)
                hit.append(text)

            for item in bs.find_all(string=re.compile('committee', re.I)):
                if 'compensation' not in str(item):
                    continue
                #find=item.parent.get_text()
                text=unicodedata.normalize("NFKD", item)
                text=html.unescape(text)
                #text=UnicodeDammit(text, ["windows-1252"], smart_quotes_to="html").unicode_markup
                text=re.sub(r'\n', ' ', text)
                text=re.sub(r'\x92',"'", text) #\x92
                text=re.sub(r'\x93','"', text)
                text=re.sub(r'\x94','"', text)
                hit.append(text)

            for item in bs.find_all(string=re.compile('compensation', re.I)):
                text=unicodedata.normalize("NFKD", item)
                text=html.unescape(text)
                #text=UnicodeDammit(text, ["windows-1252"], smart_quotes_to="html").unicode_markup
                text=re.sub(r'\n', ' ', text)
                text=re.sub(r'\x92',"'", text) #\x92
                text=re.sub(r'\x93','"', text)
                text=re.sub(r'\x94','"', text)
                hit.append(text)

            if hit!=[]:
                fileList.append(proxy)
                hitList.append(hit)
        #result={'file': proxy, 'text': hit}
        #resultDict.update(result)
        #hitList=list(set(hitList))
    return fileList, hitList

In [8]:
#### You can use Unicode, Dammit to convert Microsoft smart quotes to HTML or XML entities:
from bs4 import UnicodeDammit
markup = b"<p>I just \x93love\x94 Microsoft Word\x92s smart quotes</p>"

text=UnicodeDammit(markup, ["windows-1252"], smart_quotes_to="html").unicode_markup
#'<p>I just &ldquo;love&rdquo; Microsoft Word&rsquo;s smart quotes</p>'
text=re.sub('\n', ' ', text)
text=re.sub('&rsquo;',"'", text) #\x92
text=re.sub('&ldquo;','"', text)
text=re.sub('&rdquo;','"', text)
#### this is only useful when the string is in bits.
text

#UnicodeDammit(markup, ["windows-1252"], smart_quotes_to="xml").unicode_markup
##### '<p>I just &#x201C;love&#x201D; Microsoft Word&#x2019;s smart quotes</p>'
##### html and xml lead to different results


'<p>I just "love" Microsoft Word\'s smart quotes</p>'

def open_proxy2(proxies2007):
    """loop 2007 and after proxies and yield those with 'compensation committee' hits """
    if proxies2007==[]:
        fileList, hitList = None, None
    else:
        regex=re.compile('committee', re.I)
        fileList=[]
        hitList=[]
        for proxy in proxies2007:
            hit=[]
            try:
                with open(proxy, 'r') as f:
                    contents = f.read()
                    bs = BeautifulSoup(contents, 'lxml')
                    #bs = BeautifulSoup(open(proxy), 'html.parser')
            except TypeError: # when bs4 does not work
                fileList.append(proxy) # record the filename
                hitList.append(hit) # and return a blank list
                continue

            for item in bs.find_all(string=re.compile('compensation committee', re.I)):
                find=item.parent.get_text()
                text=unicodedata.normalize("NFKD", find)
                text=re.sub('\n', ' ', text)
                hit.append(text)


            if hit!=[]:
                fileList.append(proxy)
                hitList.append(hit)
        #result={'file': proxy, 'text': hit}
        #resultDict.update(result)
    return fileList, hitList

import re
S=r"f:\ADVANCED MICRO DEVICES INC\2488-ADVANCED MICRO DEVICES INC-DEF 14A-2007-03-15.html" #1. input string should be RAW
#### you can extract this information from file_name column.

s=S.split('\\') #there should be an escape character. r'\\' does not capture anythin, r'\' makes \ an escape character, causing EOL.and

regex=re.compile(r"f:\\(.*)\\(\d+).*(\d\d\d\d)-\d\d-\d\d\.html")
firm=regex.search(r'{}'.format(S)).group(1)
ID=regex.search(r'{}'.format(S)).group(2)
year=regex.search(r'{}'.format(S)).group(3)
firm, ID, year, idDict[ID]

#### get the CIK and firm name using below rather than regex
#### file = url.split("/")[-1]



#####remove firm name or abbreviation
#f:\ADVANCED MICRO DEVICES INC\2488-ADVANCED MICRO DEVICES INC-DEF 14A-2007-03-15.html
#regex=re.compile(r"f:\\(.*)\\(\d+)")
#firmName=regex.search(df['fileName'].to_string.encode('unicode_escape')).group(1)
#cik=regex.search(df['filname']).group(2)
#ticker=idDict[cik]

In [9]:
nlp=spacy.load(r'C:\Users\KWan\Documents\NLP & ML\compensation consultant\CC')
nlp.max_length = 10000000000

In [10]:
def ner(text):
    CC_set=set()
    doc=nlp(text)
    for sent in doc.sents:
        if 'consultant' in [token.text.lower() for token in sent]:
            for ent in sent.ents:
                print(ent.text)
                CC_set.add(ent.text.upper())
        elif 'committee'in [token.text.lower() for token in sent]:
            for ent in sent.ents:
                print(ent.text)
                CC_set.add(ent.text.upper())

#    for ent in doc.ents:
#        print(ent.label_, ent.text)
#        if ent.label_=="CC":
#            CC_list.append(ent.text)
    return CC_set

In [8]:
def NER(iterable):
    """NER() processes texts within a list of texts, compared with ner() that processes merged texts"""
    CC_set=set()
    for item in iterable:
        doc=nlp(item)
        for sent in doc.sents:
            if 'consultant' in [token.text.lower() for token in sent]:
                for ent in sent.ents:
                    print(ent.text)
                    CC_set.add(ent.text.upper())

            elif 'committee'in [token.text.lower() for token in sent]:
                for ent in sent.ents:
                    print(ent.text)
                    CC_set.add(ent.text.upper())

    return CC_set

In [11]:
def NER2(iterable):
    """NER() processes texts within a list of texts, compared with ner() that processes merged texts.
    NER2() combines NER() and choose_CC_sentence()"""

    CC_set=set()
    sent_set=set()
    sent_list=[]
    for item in iterable:
        doc=nlp(item)
        for sent in doc.sents:
            if 'consultant' in [token.text.lower() for token in sent]:
                for ent in sent.ents:
                    if ent.label_=="CC":
                        print(ent.text)
                        CC_set.add(ent.text.upper())
                        sent_set.add(str(sent.text).strip()) 
                        #sent_list.append(str(sent.text).strip())
            elif 'committee'in [token.text.lower() for token in sent]:
                for ent in sent.ents:
                    if ent.label_=="CC":
                        print(ent.text)
                        CC_set.add(ent.text.upper())
                        sent_set.add(str(sent.text).strip()) 
                        #sent_list.append(str(sent.text).strip()) 
    return CC_set, sent_set #, sent_list

def choose_cc_sentence(iterable):
    """when the NE is CC, copy that sentence. designed to be an alternative to select_text"""

    sent_set=set()
    for item in iterable:
        doc=nlp(item)
        for sent in doc.sents:
            if 'consultant' in [token.text.lower() for token in sent]:
                for ent in sent.ents:
                    if ent.label_=="CC":
                        sent_set.add(sent) if sent not in sent_set
            elif 'committee'in [token.text.lower() for token in sent]:
                for ent in sent.ents:
                    if ent.label_=="CC":
                        sent_set.add(sent) if sent not in sent_set

#    for ent in doc.ents:
#        print(ent.label_, ent.text)
#        if ent.label_=="CC":
#            CC_list.append(ent.text)
    return sent_set

ticdict={}
cikdict={}

ticList=[]
for index, item in enumerate(pageList):
    tic=item.split('\t')[0] #baba\t1577552
    cik=item.split('\t')[1] #baba\t1577552
    ticList.append(tic)
    ticdict.update({index:tic})
    cikdict.update({index:cik})

In [12]:
def extract_firm_info(file_name):
    import re
    regex=re.compile(r"f:\\(.*)\\(\d+).*(\d\d\d\d)-\d\d-\d\d\.html", re.IGNORECASE)
    firm=regex.search(r'{}'.format(file_name)).group(1)
    cik=regex.search(r'{}'.format(file_name)).group(2)
    year=regex.search(r'{}'.format(file_name)).group(3)
    try:
        tic=str(idDict[cik]).upper()
    except:
        tic=None
    return firm, year, cik, tic

In [13]:
def make_uppercase(df_cell):
    
    l=list(df_cell) 
    for item in l:
        #print(str(df['year'])) # df implicitly looks at the column of the same row.
        if jellyfish.jaro_similarity(item,str(df['firm']))>.7:
            l.remove(item)
            print(item, "removed for its similarity with firmName")
        elif jellyfish.jaro_similarity(item,str(df['tic']))>.8:    
            l.remove(item)
            print(item, "removed for its similarity with ticker")
        elif item in str(df['firm']):
            l.remove(item)
            print(item, "removed because it contains the firmname")
        elif item.isnumeric():
            l.remove(item)
            print(item, "removed because it is only a number")
        elif item=="RSU":
            l.remove(item)
        elif item.endswith('PLAN'):
            l.remove(item)
        elif item=='INDEPENDENT COMPENSATION CONSULTANT':
            l.remove(item)
        elif item=="INDEPENDENT COMPENSATION":
            l.remove(item)
        elif item=='TH COMPENSATION CONSULTANT':
            l.remove(item)
    return set(l)

In [14]:
def remove_duplicates(iterable):
    """1. checks string similarity
       2. creates combinations
       3. replaces strings"""

    import re
    import jellyfish
    import itertools
    l=list(iterable)
    #if len(l)>5:
        #return set(l)
    #elif len(l)==1:
        #return l
        
    combo=itertools.combinations(l, 2) # create combinations of 2
    for item in combo: 
        print(item)
        if jellyfish.jaro_similarity(item[0],item[1])>.7:
            if len(item[0])<len(item[1]):
                try:
                    l.remove(item[1])
                except:
                    pass
            else:
                try:
                    l.remove(item[0])
                except:
                    pass
    
    set2={re.sub(r'(Frederic)?\.?\s*W?\.?\s*Cook\s*&\s*Co\.?,?\s*(Inc)?\.?', "COOK", i, flags=re.I) for i in l}
    set3={re.sub(r'F\.?\s*W\.?\s*Cook', "COOK", i, flags=re.I) for i in set2}
    return set3

In [15]:
def remove_Cook(set):
    set2={re.sub(r'(Frederic)?\.?\s*W?\.?\s*Cook\s*&\s*Co\.?,?\s*(Inc)?\.?', "COOK", i, flags=re.I) for i in set}
    set3={re.sub(r'F\.?\s*W\.?\s*Cook', "COOK", i, flags=re.I) for i in set2}
    set4={re.sub(f'MERCER HUMAN RESOURCE CONSULTING', 'MERCER', i, flags=re.I) for i in set3}
    return set4

In [16]:
def hash_cc():
    df['CC_1']=df['CC2'].apply(lambda x: list(x)[0] if len(list(x))>0 else None)
    df['CC_2']=df['CC2'].apply(lambda x: list(x)[1] if len(list(x))>1 else None)
    df['CC_3']=df['CC2'].apply(lambda x: list(x)[2] if len(list(x))>2 else None)
    df['CC_4']=df['CC2'].apply(lambda x: list(x)[3] if len(list(x))>3 else None)
    return 1


def select_text(iterable):
    selectedText=set()
    for sent in iterable:
        if df['CC_1'][0]:
            if df['CC_1'][0] in str(sent).upper():
                print(df['CC_1'][0], 'match found in the sentence')
                print(sent)
                selectedText.add(sent)
                continue
        if df['CC_2'][0]:
            if df['CC_2'][0] in str(sent).upper():
                print(df['CC_2'][0], 'match found in the sentence')
                print(sent)
                selectedText.add(sent)
                continue
        if df['CC_3'][0]:
            if df['CC_3'][0] in str(sent).upper():
                print(df['CC_3'][0], 'match found in the sentence')
                print(sent)
                selectedText.add(sent)
                continue
        if df['CC_4'][0]:
            if df['CC_4'][0] in str(sent).upper():
                print(df['CC_4'][0], 'match found in the sentence')
                print(sent)
                selectedText.add(sent)
                continue

    #text='.'.join(select_text)
    return selectedText

In [17]:
def process(dir):
    os.chdir(dir)
    #os.getcwd()
    proxies=find_proxies(dir)
    data={}
    pathList=[]
    for i in proxies:
        filePath=os.path.join(dir, i)
        print(filePath)
        pathList.append(filePath) # create a list of file paths.
    if pathList==[]:
        print('No DEF 14A in the folder')
        return 0
    else:    
        filename, text=open_proxy1(pathList)
        data.update({'file_name':filename, 'text':text})
        df=pd.DataFrame()
        df=pd.DataFrame.from_dict(data)
        # create columns with firm info
        from tqdm.auto import tqdm
        for i, row in tqdm(df.iterrows(), total=len(df)): # df.iterrows() will iterate each row
            filename = str(row["file_name"])  # row['column name'] to select certain column within a row
            #print('filename is', filename)
            firm, year, cik, tic= extract_firm_info(filename)
            df.at[i, "firm"] = firm   # then, create a new column that contains the result of the operation.
            df.at[i, 'year'] = year
            df.at[i, 'cik'] = cik
            df.at[i, 'tic'] = tic

        df['result']=df['text'].apply(NER2)
        df['CC']=df['result'].apply(lambda x: x[0])
        df['CC_sentence']=df['result'].apply(lambda x: x[1])

        return df
        

In [18]:
def process2(df):
        df['CC1']=df['CC'].apply(make_uppercase)
        df['CC2']=df['CC1'].apply(remove_duplicates)
        df.drop(columns=['result'])
        column_order = ["file_name", "firm", "year", 'cik','tic','text','CC2', 'CC_sentence', 'CC', 'CC1']
        df = df.reindex(columns=column_order)

        return df
        

# pilot testing

### AMD

In [17]:
dir='f:\\ADVANCED MICRO DEVICES INC'


In [34]:
os.chdir(dir)
    #os.getcwd()
proxies=find_proxies(dir)
data={}
pathList=[]
for i in proxies:
    filePath=os.path.join(dir, i)
    print(filePath)
    pathList.append(filePath) # create a list of file paths.
if pathList==[]:
    print('No DEF 14A in the folder')
else:    
    filename, text=open_proxy1(pathList)
    data.update({'file_name':filename, 'text':text})

f:\ADVANCED MICRO DEVICES INC\2488-ADVANCED MICRO DEVICES INC-DEF 14A-2007-03-15.html
f:\ADVANCED MICRO DEVICES INC\2488-ADVANCED MICRO DEVICES INC-DEF 14A-2007-05-31.html
f:\ADVANCED MICRO DEVICES INC\2488-ADVANCED MICRO DEVICES INC-DEF 14A-2008-03-14.html
f:\ADVANCED MICRO DEVICES INC\2488-ADVANCED MICRO DEVICES INC-DEF 14A-2009-01-16.html
f:\ADVANCED MICRO DEVICES INC\2488-ADVANCED MICRO DEVICES INC-DEF 14A-2009-03-18.html
f:\ADVANCED MICRO DEVICES INC\2488-ADVANCED MICRO DEVICES INC-DEF 14A-2010-03-05.html
f:\ADVANCED MICRO DEVICES INC\2488-ADVANCED MICRO DEVICES INC-DEF 14A-2011-03-10.html
f:\ADVANCED MICRO DEVICES INC\2488-ADVANCED MICRO DEVICES INC-DEF 14A-2012-03-15.html
f:\ADVANCED MICRO DEVICES INC\2488-ADVANCED MICRO DEVICES INC-DEF 14A-2013-03-25.html
f:\ADVANCED MICRO DEVICES INC\2488-ADVANCED MICRO DEVICES INC-DEF 14A-2014-03-25.html
f:\ADVANCED MICRO DEVICES INC\2488-ADVANCED MICRO DEVICES INC-DEF 14A-2015-03-12.html
f:\ADVANCED MICRO DEVICES INC\2488-ADVANCED MICRO DEVI

In [36]:
df=pd.DataFrame()
df=pd.DataFrame.from_dict(data)
        # create columns with firm info
from tqdm.auto import tqdm
for i, row in tqdm(df.iterrows(), total=len(df)): # df.iterrows() will iterate each row
    filename = str(row["file_name"])  # row['column name'] to select certain column within a row
            #print('filename is', filename)
    firm, year, cik, tic= extract_firm_info(filename)
    df.at[i, "firm"] = firm   # then, create a new column that contains the result of the operation.
    df.at[i, 'year'] = year
    df.at[i, 'cik'] = cik
    df.at[i, 'tic'] = tic

df['result']=df['text'].apply(NER2)
df['CC']=df['result'].apply(lambda x: x[0])
df['CC_sentence']=df['result'].apply(lambda x: x[1])



100%|██████████| 12/12 [00:00<00:00, 2414.57it/s]
Mercer Human Resource Consulting
Mercer
Mercer
Mercer
Mercer Human Resource Consulting
Mercer
Mercer
Mercer
Mercer
Mercer
Mercer
Mercer
Mercer
Mercer
Mercer
Mercer
Mercer Human Resource Consulting
Mercer
Mercer
Mercer
Mercer
Mercer
Mercer
Mercer
Mercer
Mercer
Mercer
Mercer
Mercer Human Resource Consulting
Mercer
Mercer
Mercer
Mercer Human Resource Consulting
Mercer
Mercer
Mercer
Mercer
Mercer
Mercer
Mercer
Mercer
Mercer
Mercer
Mercer
Mercer Human Resource Consulting
Mercer
Mercer
Mercer
Mercer
Mercer
Mercer
Mercer
Mercer
Mercer
Mercer
Mercer
Mercer
Mercer
Mercer
Mercer
Mercer (US) Inc.
Mercer
Mercer
Mercer
Mercer
Mercer
Mercer
Mercer
Mercer
Mercer (US) Inc.
Mercer
Mercer
Mercer
Mercer
Mercer
Mercer
Mercer
Mercer
Mercer
Mercer
Mercer
Mercer
Mercer
Semler Brossy Consulting Group, LLC
Semler Brossy
Mercer
Semler Brossy
Semler Brossy
Mercer
Mercer (US) Inc.
Mercer
Mercer
Mercer
Mercer
Semler Brossy Consulting Group, LLC
Semler Brossy
Mercer

In [37]:
df['CC1']=df['CC'].apply(make_uppercase)
df['CC2']=df['CC1'].apply(remove_duplicates)
df.drop(columns=['result'])
column_order = ["file_name", "firm", "year", 'cik','tic','text','CC2', 'CC_sentence', 'CC', 'CC1']
df = df.reindex(columns=column_order)
df

('MERCER', 'MERCER HUMAN RESOURCE CONSULTING')
('MERCER', 'MERCER HUMAN RESOURCE CONSULTING')
('MERCER (US) INC.', 'MERCER')
('SEMLER BROSSY', 'MERCER (US) INC.')
('SEMLER BROSSY', 'MERCER')
('SEMLER BROSSY', 'SEMLER BROSSY CONSULTING GROUP, LLC')
('MERCER (US) INC.', 'MERCER')
('MERCER (US) INC.', 'SEMLER BROSSY CONSULTING GROUP, LLC')
('MERCER', 'SEMLER BROSSY CONSULTING GROUP, LLC')
('SEMLER BROSSY', 'INC.("MERCER')
('SEMLER BROSSY', 'MERCER (US')
('SEMLER BROSSY', 'SEMLER BROSSY CONSULTING GROUP, LLC')
('INC.("MERCER', 'MERCER (US')
('INC.("MERCER', 'SEMLER BROSSY CONSULTING GROUP, LLC')
('MERCER (US', 'SEMLER BROSSY CONSULTING GROUP, LLC')
('SEMLER BROSSY', 'SEMLER BROSSY CONSULTING GROUP, LLC')
('SEMLER BROSSY', 'SEMLER BOSSY')
('SEMLER BROSSY', 'COMPENSIA')
('SEMLER BROSSY CONSULTING GROUP, LLC', 'SEMLER BOSSY')
('SEMLER BROSSY CONSULTING GROUP, LLC', 'COMPENSIA')
('SEMLER BOSSY', 'COMPENSIA')
('COMPENSIA, INC.', 'COMPENSIA')
('COMPENSIA, INC.', 'COMPENSIA')
('COMPENSIA, INC.', 

,file_name,firm,year,cik,tic,text,CC2,CC_sentence,CC,CC1
0,f:\ADVANCED MICRO DEVICES INC\2488-ADVANCED MI...,ADVANCED MICRO DEVICES INC,2007,2488,AMD,[Dr. Silverman served as Chair of the Committe...,{MERCER},"{In assisting the Committee, Mercer provides i...","{MERCER, MERCER HUMAN RESOURCE CONSULTING}","{MERCER, MERCER HUMAN RESOURCE CONSULTING}"
1,f:\ADVANCED MICRO DEVICES INC\2488-ADVANCED MI...,ADVANCED MICRO DEVICES INC,2007,2488,AMD,[The Committee has retained Mercer Human Resou...,{MERCER},"{In assisting the Committee, Mercer provides i...","{MERCER, MERCER HUMAN RESOURCE CONSULTING}","{MERCER, MERCER HUMAN RESOURCE CONSULTING}"
2,f:\ADVANCED MICRO DEVICES INC\2488-ADVANCED MI...,ADVANCED MICRO DEVICES INC,2008,2488,AMD,[ The Compensation Committee consists of Mr...,{MERCER},"{In the course of conducting its activities, M...","{MERCER (US) INC., MERCER}","{MERCER (US) INC., MERCER}"
3,f:\ADVANCED MICRO DEVICES INC\2488-ADVANCED MI...,ADVANCED MICRO DEVICES INC,2009,2488,AMD,[contracts with independent contractors or con...,{},{},{},{}
4,f:\ADVANCED MICRO DEVICES INC\2488-ADVANCED MI...,ADVANCED MICRO DEVICES INC,2009,2488,AMD,[ The Compensation Committee consists of Ms...,"{SEMLER BROSSY, MERCER}","{In 2008, Mercer (US) Inc. (Mercer) advised th...","{SEMLER BROSSY, MERCER (US) INC., MERCER, SEML...","{SEMLER BROSSY, MERCER (US) INC., MERCER, SEML..."
5,f:\ADVANCED MICRO DEVICES INC\2488-ADVANCED MI...,ADVANCED MICRO DEVICES INC,2010,2488,AMD,"[ operations, finance and strategic consulting...","{SEMLER BROSSY, INC.(""MERCER, MERCER (US}","{Semler Brossy, in its capacity as an independ...","{SEMLER BROSSY, INC.(""MERCER, MERCER (US, SEML...","{SEMLER BROSSY, INC.(""MERCER, MERCER (US, SEML..."
6,f:\ADVANCED MICRO DEVICES INC\2488-ADVANCED MI...,ADVANCED MICRO DEVICES INC,2011,2488,AMD,"[Mr. Al Muhairi, 36, has been a director sinc...","{SEMLER BOSSY, COMPENSIA}",{In order to maintain affordability and a reas...,"{SEMLER BROSSY, COMPENSIA, SEMLER BOSSY, SEMLE...","{SEMLER BROSSY, SEMLER BROSSY CONSULTING GROUP..."
7,f:\ADVANCED MICRO DEVICES INC\2488-ADVANCED MI...,ADVANCED MICRO DEVICES INC,2012,2488,AMD,"[Mr. Muhairi, 37, has been a director since M...",{COMPENSIA},{With respect to Mr. Read's initial base salar...,"{COMPENSIA, INC., COMPENSIA}","{COMPENSIA, INC., COMPENSIA}"
8,f:\ADVANCED MICRO DEVICES INC\2488-ADVANCED MI...,ADVANCED MICRO DEVICES INC,2013,2488,AMD,[ The Compensation Committee consists of M...,{COMPENSIA},"{In 2012, Compensia did not provide any servic...","{COMPENSIA, INC., COMPENSIA}","{COMPENSIA, INC., COMPENSIA}"
9,f:\ADVANCED MICRO DEVICES INC\2488-ADVANCED MI...,ADVANCED MICRO DEVICES INC,2014,2488,AMD,[. The Compensation Committee assists the B...,{COMPENSIA},{The Compensation Committee has assessed wheth...,"{COMPENSIA, INC., COMPENSIA}","{COMPENSIA, INC., COMPENSIA}"


In [18]:
df=process()

f:\ADVANCED MICRO DEVICES INC\2488-ADVANCED MICRO DEVICES INC-DEF 14A-2007-03-15.html
f:\ADVANCED MICRO DEVICES INC\2488-ADVANCED MICRO DEVICES INC-DEF 14A-2007-05-31.html
f:\ADVANCED MICRO DEVICES INC\2488-ADVANCED MICRO DEVICES INC-DEF 14A-2008-03-14.html
f:\ADVANCED MICRO DEVICES INC\2488-ADVANCED MICRO DEVICES INC-DEF 14A-2009-01-16.html
f:\ADVANCED MICRO DEVICES INC\2488-ADVANCED MICRO DEVICES INC-DEF 14A-2009-03-18.html
f:\ADVANCED MICRO DEVICES INC\2488-ADVANCED MICRO DEVICES INC-DEF 14A-2010-03-05.html
f:\ADVANCED MICRO DEVICES INC\2488-ADVANCED MICRO DEVICES INC-DEF 14A-2011-03-10.html
f:\ADVANCED MICRO DEVICES INC\2488-ADVANCED MICRO DEVICES INC-DEF 14A-2012-03-15.html
f:\ADVANCED MICRO DEVICES INC\2488-ADVANCED MICRO DEVICES INC-DEF 14A-2013-03-25.html
f:\ADVANCED MICRO DEVICES INC\2488-ADVANCED MICRO DEVICES INC-DEF 14A-2014-03-25.html
f:\ADVANCED MICRO DEVICES INC\2488-ADVANCED MICRO DEVICES INC-DEF 14A-2015-03-12.html
f:\ADVANCED MICRO DEVICES INC\2488-ADVANCED MICRO DEVI

In [19]:
df=process2(df)

('MERCER', 'MERCER HUMAN RESOURCE CONSULTING')
('MERCER', 'MERCER HUMAN RESOURCE CONSULTING')
('MERCER (US) INC.', 'MERCER')
('SEMLER BROSSY', 'SEMLER BROSSY CONSULTING GROUP, LLC')
('SEMLER BROSSY', 'MERCER (US) INC.')
('SEMLER BROSSY', 'MERCER')
('SEMLER BROSSY CONSULTING GROUP, LLC', 'MERCER (US) INC.')
('SEMLER BROSSY CONSULTING GROUP, LLC', 'MERCER')
('MERCER (US) INC.', 'MERCER')
('MERCER (US', 'SEMLER BROSSY')
('MERCER (US', 'SEMLER BROSSY CONSULTING GROUP, LLC')
('MERCER (US', 'INC.("MERCER')
('SEMLER BROSSY', 'SEMLER BROSSY CONSULTING GROUP, LLC')
('SEMLER BROSSY', 'INC.("MERCER')
('SEMLER BROSSY CONSULTING GROUP, LLC', 'INC.("MERCER')
('SEMLER BROSSY', 'SEMLER BROSSY CONSULTING GROUP, LLC')
('SEMLER BROSSY', 'SEMLER BOSSY')
('SEMLER BROSSY', 'COMPENSIA')
('SEMLER BROSSY CONSULTING GROUP, LLC', 'SEMLER BOSSY')
('SEMLER BROSSY CONSULTING GROUP, LLC', 'COMPENSIA')
('SEMLER BOSSY', 'COMPENSIA')
('COMPENSIA, INC.', 'COMPENSIA')
('COMPENSIA, INC.', 'COMPENSIA')
('COMPENSIA, INC.', 

In [20]:
df

,file_name,firm,year,cik,tic,text,CC2,CC_sentence,CC,CC1
0,f:\ADVANCED MICRO DEVICES INC\2488-ADVANCED MI...,ADVANCED MICRO DEVICES INC,2007,2488,NaN,[Dr. Silverman served as Chair of the Committe...,{MERCER},"{In 2006, Mercer advised the Committee on a va...","{MERCER, MERCER HUMAN RESOURCE CONSULTING}","{MERCER, MERCER HUMAN RESOURCE CONSULTING}"
1,f:\ADVANCED MICRO DEVICES INC\2488-ADVANCED MI...,ADVANCED MICRO DEVICES INC,2007,2488,NaN,[The Committee has retained Mercer Human Resou...,{MERCER},{The Company's human resources department work...,"{MERCER, MERCER HUMAN RESOURCE CONSULTING}","{MERCER, MERCER HUMAN RESOURCE CONSULTING}"
2,f:\ADVANCED MICRO DEVICES INC\2488-ADVANCED MI...,ADVANCED MICRO DEVICES INC,2008,2488,NaN,[ The Compensation Committee consists of Mr...,{MERCER},{In order to maintain an objective external pe...,"{MERCER (US) INC., MERCER}","{MERCER (US) INC., MERCER}"
3,f:\ADVANCED MICRO DEVICES INC\2488-ADVANCED MI...,ADVANCED MICRO DEVICES INC,2009,2488,NaN,[contracts with independent contractors or con...,{},{},{},{}
4,f:\ADVANCED MICRO DEVICES INC\2488-ADVANCED MI...,ADVANCED MICRO DEVICES INC,2009,2488,NaN,[ The Compensation Committee consists of Ms...,"{SEMLER BROSSY, MERCER}",{To assist the Board in its annual review of d...,"{SEMLER BROSSY, SEMLER BROSSY CONSULTING GROUP...","{SEMLER BROSSY, SEMLER BROSSY CONSULTING GROUP..."
5,f:\ADVANCED MICRO DEVICES INC\2488-ADVANCED MI...,ADVANCED MICRO DEVICES INC,2010,2488,NaN,"[ operations, finance and strategic consulting...","{MERCER (US, SEMLER BROSSY, INC.(""MERCER}","{In 2009, the Compensation Committee retained ...","{MERCER (US, SEMLER BROSSY, SEMLER BROSSY CONS...","{MERCER (US, SEMLER BROSSY, SEMLER BROSSY CONS..."
6,f:\ADVANCED MICRO DEVICES INC\2488-ADVANCED MI...,ADVANCED MICRO DEVICES INC,2011,2488,NaN,"[Mr. Al Muhairi, 36, has been a director sinc...","{SEMLER BOSSY, COMPENSIA}","{In September 2010, the Compensation Committee...","{SEMLER BROSSY, SEMLER BROSSY CONSULTING GROUP...","{SEMLER BROSSY, SEMLER BROSSY CONSULTING GROUP..."
7,f:\ADVANCED MICRO DEVICES INC\2488-ADVANCED MI...,ADVANCED MICRO DEVICES INC,2012,2488,NaN,"[Mr. Muhairi, 37, has been a director since M...",{COMPENSIA},{Compensia advised the Compensation Committee ...,"{COMPENSIA, INC., COMPENSIA}","{COMPENSIA, INC., COMPENSIA}"
8,f:\ADVANCED MICRO DEVICES INC\2488-ADVANCED MI...,ADVANCED MICRO DEVICES INC,2013,2488,NaN,[ The Compensation Committee consists of M...,{COMPENSIA},{Compensia advised the Compensation Committee ...,"{COMPENSIA, INC., COMPENSIA}","{COMPENSIA, INC., COMPENSIA}"
9,f:\ADVANCED MICRO DEVICES INC\2488-ADVANCED MI...,ADVANCED MICRO DEVICES INC,2014,2488,NaN,[. The Compensation Committee assists the B...,{COMPENSIA},{Compensia advised the Compensation Committee ...,"{COMPENSIA, INC., COMPENSIA}","{COMPENSIA, INC., COMPENSIA}"


import re
regex=re.compile(r'(\d+)-')
#input cik and firmName mannually 
cik=regex.search(p[0]).group(1)
ticker=idDict[cik].upper()
ticker
firmName='Advanced Micro Devices, Inc.'.upper()

In [21]:
df['CC_sentence'].iloc[10]

{'Compensia advised the Compensation Committee on a variety of compensation-related matters in fiscal 2014, including:',
 "Compensia also consulted frequently with members of the Compensation Committee and also met with senior management to obtain and validate market data, review materials and discuss management's compensation recommendations.",
 'Compensia is retained directly by the Compensation Committee, advises the Compensation Committee on pay decisions regarding our Named Executive Officers and keeps the Compensation Committee apprised of compensation trends and best practices.',
 'During fiscal 2014, the Compensation Committee retained Compensia, a national compensation consulting firm, as its compensation consultant to provide assistance on executive and director compensation matters.',
 'In February 2015, the Compensation Committee assessed whether the services provided by Compensia raised any conflicts of interest pursuant to the rules of the SEC and the listing rules of Nas

# 2. ATT

In [22]:
dir='f:\\AT&T INC'

In [33]:
df=process()


f:\AT&T INC\732717-AT&T INC.-DEF 14A-2007-03-22.html
f:\AT&T INC\732717-AT&T INC.-DEF 14A-2008-03-11.html
f:\AT&T INC\732717-AT&T INC.-DEF 14A-2009-03-11.html
f:\AT&T INC\732717-AT&T INC.-DEF 14A-2010-03-11.html
f:\AT&T INC\732717-AT&T INC.-DEF 14A-2011-03-10.html
f:\AT&T INC\732717-AT&T INC.-DEF 14A-2012-03-12.html
f:\AT&T INC\732717-AT&T INC.-DEF 14A-2013-03-11.html
f:\AT&T INC\732717-AT&T INC.-DEF 14A-2014-03-11.html
f:\AT&T INC\732717-AT&T INC.-DEF 14A-2015-03-10.html
f:\AT&T INC\732717-AT&T INC.-DEF 14A-2016-03-11.html
100%|██████████| 10/10 [00:00<00:00, 2501.08it/s]
CCA Strategies LLC
CCA Strategies
CCA Strategies
CCA Strategies
CCA Strategies
CCA Strategies
CCA Strategies
CCA Strategies
CCA Strategies
CCA Strategies LLC
CCA Strategies
CCA Strategies
CCA Strategies
CCA Strategies
CCA Strategies
CCA Strategies
CCA Strategies
CCA Strategies
CCA Strategies
CCA Strategies LLC
CCA Strategies
CCA Strategies
CCA Strategies
CCA Strategies
CCA Strategies
CCA Strategies
CCA Strategies
CCA

In [34]:
df=process2(df)
df

('CCA STRATEGIES', 'CCA STRATEGIES LLC')
('CCA STRATEGIES', 'CCA STRATEGIES LLC')
('TOTAL REWARDS STRATEGIES', 'THE COMPENSATION CONSULTANT')
('TOTAL REWARDS STRATEGIES', 'THE COMPENSATION CONSULTANT')


,file_name,firm,year,cik,tic,text,CC2,CC_sentence,CC,CC1
0,f:\AT&T INC\732717-AT&T INC.-DEF 14A-2007-03-2...,AT&T INC,2007,732717,NaN,[ - -----------------------------------------...,{CCA STRATEGIES},{Role of Compensation Consultant The Committee...,"{CCA STRATEGIES, CCA STRATEGIES LLC}","{CCA STRATEGIES, CCA STRATEGIES LLC}"
1,f:\AT&T INC\732717-AT&T INC.-DEF 14A-2008-03-1...,AT&T INC,2008,732717,NaN,[ The Corporate Governance and Nominati...,{CCA STRATEGIES},"{From time to time, the Committee engages CCA ...","{CCA STRATEGIES, CCA STRATEGIES LLC}","{CCA STRATEGIES, CCA STRATEGIES LLC}"
2,f:\AT&T INC\732717-AT&T INC.-DEF 14A-2009-03-1...,AT&T INC,2009,732717,NaN,[The Corporate Governance and Nominating Commi...,{TOTAL REWARDS STRATEGIES},"{From time to time, the Committee engages Tota...",{TOTAL REWARDS STRATEGIES},{TOTAL REWARDS STRATEGIES}
3,f:\AT&T INC\732717-AT&T INC.-DEF 14A-2010-03-1...,AT&T INC,2010,732717,NaN,[The Corporate Governance and Nominating Commi...,{TOTAL REWARDS STRATEGIES},"{From time to time, the Committee engages Tota...",{TOTAL REWARDS STRATEGIES},{TOTAL REWARDS STRATEGIES}
4,f:\AT&T INC\732717-AT&T INC.-DEF 14A-2011-03-1...,AT&T INC,2011,732717,NaN,[The Corporate Governance and Nominating Commi...,{TOTAL REWARDS STRATEGIES},"{From time to time, the Committee engages Tota...",{TOTAL REWARDS STRATEGIES},{TOTAL REWARDS STRATEGIES}
5,f:\AT&T INC\732717-AT&T INC.-DEF 14A-2012-03-1...,AT&T INC,2012,732717,NaN,[The Corporate Governance and Nominating Commi...,{TOTAL REWARDS STRATEGIES},"{From time to time, the Committee engages Tota...",{TOTAL REWARDS STRATEGIES},{TOTAL REWARDS STRATEGIES}
6,f:\AT&T INC\732717-AT&T INC.-DEF 14A-2013-03-1...,AT&T INC,2013,732717,NaN,[The Corporate Governance and Nominating Commi...,{TOTAL REWARDS STRATEGIES},"{From time to time, the Committee engages Tota...",{TOTAL REWARDS STRATEGIES},{TOTAL REWARDS STRATEGIES}
7,f:\AT&T INC\732717-AT&T INC.-DEF 14A-2014-03-1...,AT&T INC,2014,732717,NaN,[The Corporate Governance and Nominating Commi...,{TOTAL REWARDS STRATEGIES},"{From time to time, the Committee engages Tota...",{TOTAL REWARDS STRATEGIES},{TOTAL REWARDS STRATEGIES}
8,f:\AT&T INC\732717-AT&T INC.-DEF 14A-2015-03-1...,AT&T INC,2015,732717,NaN,[The Corporate Governance and Nominating Commi...,"{TOTAL REWARDS STRATEGIES, THE COMPENSATION CO...",{The Committee has selected Total Rewards Stra...,"{TOTAL REWARDS STRATEGIES, THE COMPENSATION CO...","{TOTAL REWARDS STRATEGIES, THE COMPENSATION CO..."
9,f:\AT&T INC\732717-AT&T INC.-DEF 14A-2016-03-1...,AT&T INC,2016,732717,NaN,[The Corporate Governance and Nominating Commi...,"{TOTAL REWARDS STRATEGIES, THE COMPENSATION CO...",{The Compensation Consultant has also reviewed...,"{TOTAL REWARDS STRATEGIES, THE COMPENSATION CO...","{TOTAL REWARDS STRATEGIES, THE COMPENSATION CO..."


In [29]:
df['CC_sentence'].iloc[8]

{'The Committee has selected Total Rewards Strategies to serve as its independent consultant.',
 'The Compensation Consultant also reviewed our programs and advised the Committee that the programs did not encourage risk-taking reasonably likely to have a material adverse effect on AT&T.'}

# 3. Amazon


In [30]:
dir='f:\\AMAZON COM INC'

In [31]:
os.chdir(dir)
os.getcwd()
proxies=find_proxies(dir)
data={}
pathList=[]
for i in proxies:
    filePath=os.path.join(dir, i)
    print(filePath)
    pathList.append(filePath) # create a list of file paths.

filename, text=open_proxy1(pathList)
data.update({'file_name':filename, 'text':text})

f:\AMAZON COM INC\1018724-AMAZON COM INC-DEF 14A-2007-04-27.html
f:\AMAZON COM INC\1018724-AMAZON COM INC-DEF 14A-2008-04-18.html
f:\AMAZON COM INC\1018724-AMAZON COM INC-DEF 14A-2009-04-17.html
f:\AMAZON COM INC\1018724-AMAZON COM INC-DEF 14A-2010-04-14.html
f:\AMAZON COM INC\1018724-AMAZON COM INC-DEF 14A-2011-04-27.html
f:\AMAZON COM INC\1018724-AMAZON COM INC-DEF 14A-2012-04-13.html
f:\AMAZON COM INC\1018724-AMAZON COM INC-DEF 14A-2013-04-12.html
f:\AMAZON COM INC\1018724-AMAZON COM INC-DEF 14A-2014-04-10.html
f:\AMAZON COM INC\1018724-AMAZON COM INC-DEF 14A-2015-04-24.html
f:\AMAZON COM INC\1018724-AMAZON COM INC-DEF 14A-2016-04-05.html


In [32]:
df=pd.DataFrame.from_dict(data)
# create columns with firm info
from tqdm.auto import tqdm
for i, row in tqdm(df.iterrows(), total=len(df)): # df.iterrows() will iterate each row
    filename = str(row["file_name"])  # row['column name'] to select certain column within a row
    print('filename is', filename)
    firm, year, cik, tic= extract_firm_info(filename)
    df.at[i, "firm"] = firm   # then, create a new column that contains the result of the operation.
    df.at[i, 'year'] = year
    df.at[i, 'cik'] = cik
    df.at[i, 'tic'] = tic

column_order = ["file_name", "firm", "year", 'cik','tic','text']
df = df.reindex(columns=column_order)
df

100%|██████████| 10/10 [00:00<00:00, 2500.33it/s]filename is f:\AMAZON COM INC\1018724-AMAZON COM INC-DEF 14A-2007-04-27.html
filename is f:\AMAZON COM INC\1018724-AMAZON COM INC-DEF 14A-2008-04-18.html
filename is f:\AMAZON COM INC\1018724-AMAZON COM INC-DEF 14A-2009-04-17.html
filename is f:\AMAZON COM INC\1018724-AMAZON COM INC-DEF 14A-2010-04-14.html
filename is f:\AMAZON COM INC\1018724-AMAZON COM INC-DEF 14A-2011-04-27.html
filename is f:\AMAZON COM INC\1018724-AMAZON COM INC-DEF 14A-2012-04-13.html
filename is f:\AMAZON COM INC\1018724-AMAZON COM INC-DEF 14A-2013-04-12.html
filename is f:\AMAZON COM INC\1018724-AMAZON COM INC-DEF 14A-2014-04-10.html
filename is f:\AMAZON COM INC\1018724-AMAZON COM INC-DEF 14A-2015-04-24.html
filename is f:\AMAZON COM INC\1018724-AMAZON COM INC-DEF 14A-2016-04-05.html



,file_name,firm,year,cik,tic,text
0,f:\AMAZON COM INC\1018724-AMAZON COM INC-DEF 1...,AMAZON COM INC,2007,1018724,AMZN,"[age 48, has been a director of the Company si..."
1,f:\AMAZON COM INC\1018724-AMAZON COM INC-DEF 1...,AMAZON COM INC,2008,1018724,AMZN,"[age 49, has been a director of the Company si..."
2,f:\AMAZON COM INC\1018724-AMAZON COM INC-DEF 1...,AMAZON COM INC,2009,1018724,AMZN,[ The Leadership Development and Compensati...
3,f:\AMAZON COM INC\1018724-AMAZON COM INC-DEF 1...,AMAZON COM INC,2010,1018724,AMZN,[ The Leadership Development and Compensati...
4,f:\AMAZON COM INC\1018724-AMAZON COM INC-DEF 1...,AMAZON COM INC,2011,1018724,AMZN,[ The Leadership Development and Compensati...
5,f:\AMAZON COM INC\1018724-AMAZON COM INC-DEF 1...,AMAZON COM INC,2012,1018724,AMZN,[ The Leadership Development and Compensati...
6,f:\AMAZON COM INC\1018724-AMAZON COM INC-DEF 1...,AMAZON COM INC,2013,1018724,AMZN,[ The Leadership Development and Compensati...
7,f:\AMAZON COM INC\1018724-AMAZON COM INC-DEF 1...,AMAZON COM INC,2014,1018724,AMZN,[ The Leadership Development and Compensati...
8,f:\AMAZON COM INC\1018724-AMAZON COM INC-DEF 1...,AMAZON COM INC,2015,1018724,AMZN,[ The Leadership Development and Compensati...
9,f:\AMAZON COM INC\1018724-AMAZON COM INC-DEF 1...,AMAZON COM INC,2016,1018724,AMZN,[ The Leadership Development and Compensati...


In [33]:
df['result']=df['text'].apply(NER2)
df['CC']=df['result'].apply(lambda x: x[0])
df['CC_sentence']=df['result'].apply(lambda x: x[1])
df['CC1']=df['CC'].apply(make_uppercase)
df['CC2']=df['CC1'].apply(remove_duplicates)
df


E&Y
E&Y
E&Y
E&Y
E&Y
E&Y


,file_name,firm,year,cik,tic,text,result,CC,CC_sentence,CC1,CC2
0,f:\AMAZON COM INC\1018724-AMAZON COM INC-DEF 1...,AMAZON COM INC,2007,1018724,AMZN,"[age 48, has been a director of the Company si...","({}, {})",{},{},{},{}
1,f:\AMAZON COM INC\1018724-AMAZON COM INC-DEF 1...,AMAZON COM INC,2008,1018724,AMZN,"[age 49, has been a director of the Company si...","({}, {})",{},{},{},{}
2,f:\AMAZON COM INC\1018724-AMAZON COM INC-DEF 1...,AMAZON COM INC,2009,1018724,AMZN,[ The Leadership Development and Compensati...,"({}, {})",{},{},{},{}
3,f:\AMAZON COM INC\1018724-AMAZON COM INC-DEF 1...,AMAZON COM INC,2010,1018724,AMZN,[ The Leadership Development and Compensati...,"({}, {})",{},{},{},{}
4,f:\AMAZON COM INC\1018724-AMAZON COM INC-DEF 1...,AMAZON COM INC,2011,1018724,AMZN,[ The Leadership Development and Compensati...,"({}, {})",{},{},{},{}
5,f:\AMAZON COM INC\1018724-AMAZON COM INC-DEF 1...,AMAZON COM INC,2012,1018724,AMZN,[ The Leadership Development and Compensati...,"({}, {})",{},{},{},{}
6,f:\AMAZON COM INC\1018724-AMAZON COM INC-DEF 1...,AMAZON COM INC,2013,1018724,AMZN,[ The Leadership Development and Compensati...,"({}, {})",{},{},{},{}
7,f:\AMAZON COM INC\1018724-AMAZON COM INC-DEF 1...,AMAZON COM INC,2014,1018724,AMZN,[ The Leadership Development and Compensati...,"({E&Y}, {The Audit Committee has appointed, an...",{E&Y},"{The Audit Committee has appointed, and, as a ...",{E&Y},{E&Y}
8,f:\AMAZON COM INC\1018724-AMAZON COM INC-DEF 1...,AMAZON COM INC,2015,1018724,AMZN,[ The Leadership Development and Compensati...,"({E&Y}, {The Audit Committee has appointed, an...",{E&Y},"{The Audit Committee has appointed, and, as a ...",{E&Y},{E&Y}
9,f:\AMAZON COM INC\1018724-AMAZON COM INC-DEF 1...,AMAZON COM INC,2016,1018724,AMZN,[ The Leadership Development and Compensati...,"({E&Y}, {The Audit Committee has appointed, an...",{E&Y},"{The Audit Committee has appointed, and, as a ...",{E&Y},{E&Y}


In [34]:
df['CC_sentence'].iloc[7]

{'The Audit Committee has appointed, and, as a matter of good corporate governance, is requesting ratification by the shareholders of the appointment of, the registered public accounting firm of Ernst & Young LLP ("E&Y") to serve as independent auditors for the fiscal year ending December 31, 2014.'}

# 4. Apple

In [35]:
dir='F:\\APPLE INC'

In [36]:
os.chdir(dir)
os.getcwd()
proxies=find_proxies(dir)
data={}
pathList=[]
for i in proxies:
    filePath=os.path.join(dir, i)
    print(filePath)
    pathList.append(filePath) # create a list of file paths.

filename, text=open_proxy1(pathList)
data.update({'file_name':filename, 'text':text})

F:\APPLE INC\320193-APPLE INC-DEF 14A-2007-04-16.html
F:\APPLE INC\320193-APPLE INC-DEF 14A-2008-01-23.html
F:\APPLE INC\320193-APPLE INC-DEF 14A-2009-01-07.html
F:\APPLE INC\320193-APPLE INC-DEF 14A-2010-01-12.html
F:\APPLE INC\320193-APPLE INC-DEF 14A-2011-01-07.html
F:\APPLE INC\320193-APPLE INC-DEF 14A-2012-01-09.html
F:\APPLE INC\320193-APPLE INC-DEF 14A-2013-01-07.html
F:\APPLE INC\320193-APPLE INC-DEF 14A-2014-01-10.html
F:\APPLE INC\320193-APPLE INC-DEF 14A-2015-01-22.html
F:\APPLE INC\320193-APPLE INC-DEF 14A-2016-01-06.html


In [37]:
df=pd.DataFrame()
df=pd.DataFrame.from_dict(data)
# create columns with firm info
from tqdm.auto import tqdm
for i, row in tqdm(df.iterrows(), total=len(df)): # df.iterrows() will iterate each row
    filename = str(row["file_name"])  # row['column name'] to select certain column within a row
    print('filename is', filename)
    firm, year, cik, tic= extract_firm_info(filename)
    df.at[i, "firm"] = firm   # then, create a new column that contains the result of the operation.
    df.at[i, 'year'] = year
    df.at[i, 'cik'] = cik
    df.at[i, 'tic'] = tic

column_order = ["file_name", "firm", "year", 'cik','tic','text']
df = df.reindex(columns=column_order)

100%|██████████| 10/10 [00:00<00:00, 2500.48it/s]filename is F:\APPLE INC\320193-APPLE INC-DEF 14A-2007-04-16.html
filename is F:\APPLE INC\320193-APPLE INC-DEF 14A-2008-01-23.html
filename is F:\APPLE INC\320193-APPLE INC-DEF 14A-2009-01-07.html
filename is F:\APPLE INC\320193-APPLE INC-DEF 14A-2010-01-12.html
filename is F:\APPLE INC\320193-APPLE INC-DEF 14A-2011-01-07.html
filename is F:\APPLE INC\320193-APPLE INC-DEF 14A-2012-01-09.html
filename is F:\APPLE INC\320193-APPLE INC-DEF 14A-2013-01-07.html
filename is F:\APPLE INC\320193-APPLE INC-DEF 14A-2014-01-10.html
filename is F:\APPLE INC\320193-APPLE INC-DEF 14A-2015-01-22.html
filename is F:\APPLE INC\320193-APPLE INC-DEF 14A-2016-01-06.html



In [38]:
df['result']=df['text'].apply(NER2)
df['CC']=df['result'].apply(lambda x: x[0])
df['CC_sentence']=df['result'].apply(lambda x: x[1])
df['CC1']=df['CC'].apply(make_uppercase)
df['CC2']=df['CC1'].apply(remove_duplicates)
df

Frederic W. Cook & Co., Inc.
Frederic W. Cook & Co., Inc.
F.W. Cook
Frederic W. Cook & Co., Inc.
F.W. Cook
F.W. Cook
F.W. Cook
Frederic W. Cook & Co., Inc.
F.W. Cook
F.W. Cook
F.W. Cook
F.W. Cook
F.W. Cook
Cook
F.W. Cook
Frederic W. Cook & Co., Inc.
F.W. Cook
F.W. Cook
F.W. Cook
Cook
F.W. Cook
F.W. Cook
F.W. Cook
F.W. Cook
Frederic W. Cook & Co., Inc.
F.W. Cook
F.W. Cook
F.W. Cook
F.W. Cook
F.W. Cook
F.W. Cook
Frederic W. Cook & Co., Inc.
F.W. Cook
F.W. Cook
F.W. Cook
Cook
Cook
Cook
Cook
F.W. Cook
F.W. Cook
F.W. Cook
F.W. Cook
Frederic W. Cook & Co.
F.W. Cook
F.W. Cook
Frederic W. Cook & Co., Inc.
F.W. Cook
F.W. Cook
F.W. Cook
F.W. Cook
Frederic W. Cook & Co.
F.W. Cook
F.W. Cook
F.W. Cook
F.W. Cook
F.W. Cook
F.W. Cook
Cook
Cook
Cook
F.W. Cook
F.W. Cook
F.W. Cook
F.W. Cook
F.W. Cook
Cook
Cook
Cook
Cook
Cook
Cook
Frederic W. Cook & Co., Inc.
F.W. Cook
F.W. Cook
F.W. Cook
F.W. Cook
F.W. Cook
Cook
F.W. Cook
Cook
Cook
F.W. Cook
F.W. Cook
F.W. Cook
F.W. Cook
F.W. Cook
Cook
Cook
Cook
Cook
F.W

,file_name,firm,year,cik,tic,text,result,CC,CC_sentence,CC1,CC2
0,F:\APPLE INC\320193-APPLE INC-DEF 14A-2007-04-...,APPLE INC,2007,320193,AAPL,"[Independent consultants and advisors., The Co...","({FREDERIC W. COOK & CO., INC., F.W. COOK}, {D...","{FREDERIC W. COOK & CO., INC., F.W. COOK}","{During fiscal year 2006, the Committee engage...","{FREDERIC W. COOK & CO., INC., F.W. COOK}",{COOK}
1,F:\APPLE INC\320193-APPLE INC-DEF 14A-2008-01-...,APPLE INC,2008,320193,AAPL,"[5. The Role of Consultants , The Compensation...","({FREDERIC W. COOK & CO., INC., F.W. COOK}, {T...","{FREDERIC W. COOK & CO., INC., F.W. COOK}",{The Compensation Committee periodically seeks...,"{FREDERIC W. COOK & CO., INC., F.W. COOK}",{COOK}
2,F:\APPLE INC\320193-APPLE INC-DEF 14A-2009-01-...,APPLE INC,2009,320193,AAPL,"[""), where he serves on the Leadership Develop...","({FREDERIC W. COOK & CO., INC., F.W. COOK, COO...","{FREDERIC W. COOK & CO., INC., F.W. COOK, COOK}","{With the assistance of F.W. Cook, the Compens...","{FREDERIC W. COOK & CO., INC., F.W. COOK, COOK}",{COOK}
3,F:\APPLE INC\320193-APPLE INC-DEF 14A-2010-01-...,APPLE INC,2010,320193,AAPL,"["") since September 1999. Previously, Dr. Levi...","({FREDERIC W. COOK & CO., INC., F.W. COOK}, {F...","{FREDERIC W. COOK & CO., INC., F.W. COOK}",{F.W. Cook only provides compensation consulti...,"{FREDERIC W. COOK & CO., INC., F.W. COOK}",{COOK}
4,F:\APPLE INC\320193-APPLE INC-DEF 14A-2011-01-...,APPLE INC,2011,320193,AAPL,"["") since September 1999. Previously, Dr. Levi...","({FREDERIC W. COOK & CO., INC., F.W. COOK, COO...","{FREDERIC W. COOK & CO., INC., F.W. COOK, COOK}","{In 2010, the Compensation Committee awarded M...","{FREDERIC W. COOK & CO., INC., F.W. COOK, COOK}",{COOK}
5,F:\APPLE INC\320193-APPLE INC-DEF 14A-2012-01-...,APPLE INC,2012,320193,AAPL,"["") since September 1999. Previously, Dr. Levi...","({FREDERIC W. COOK & CO., INC., F.W. COOK, FRE...","{FREDERIC W. COOK & CO., INC., F.W. COOK, FRED...","{The Compensation Committee determined, using ...","{FREDERIC W. COOK & CO., INC., F.W. COOK, FRED...",{COOK}
6,F:\APPLE INC\320193-APPLE INC-DEF 14A-2013-01-...,APPLE INC,2013,320193,AAPL,"["") since September 1999. Previously, Dr. Levi...","({FREDERIC W. COOK & CO., INC., F.W. COOK, COO...","{FREDERIC W. COOK & CO., INC., F.W. COOK, COOK}","{With the assistance of F.W. Cook, the Compens...","{FREDERIC W. COOK & CO., INC., F.W. COOK, COOK}",{COOK}
7,F:\APPLE INC\320193-APPLE INC-DEF 14A-2014-01-...,APPLE INC,2014,320193,AAPL,"["") since September 1999, and a member of the ...","({F.W. COOK, COOK}, {With the assistance of F....","{F.W. COOK, COOK}","{With the assistance of F.W. Cook, the Compens...","{F.W. COOK, COOK}",{COOK}
8,F:\APPLE INC\320193-APPLE INC-DEF 14A-2015-01-...,APPLE INC,2015,320193,AAPL,[ Development center and as a member of Genent...,"({PAY GOVERNANCE, FREDERIC W. COOK & CO.,, COO...","{PAY GOVERNANCE, FREDERIC W. COOK & CO.,, COOK}",{Pay Governance did not provide any other serv...,"{PAY GOVERNANCE, FREDERIC W. COOK & CO.,, COOK}","{PAY GOVERNANCE, COOK}"
9,F:\APPLE INC\320193-APPLE INC-DEF 14A-2016-01-...,APPLE INC,2016,320193,AAPL,[ has served as the Chief Executive Officer of...,"({PAY GOVERNANCE}, {Pay Governance also commun...",{PAY GOVERNANCE},{Pay Governance also communicates with the Cha...,{PAY GOVERNANCE},{PAY GOVERNANCE}


In [108]:
df['result']=df['text'].apply(NER2)
df['CC']=df['result'].apply(lambda x: x[0])
df['CC_sentence']=df['result'].apply(lambda x: x[1])
df['CC1']=df['CC'].apply(make_uppercase)
df['CC2']=df['CC1'].apply(remove_duplicates)
df

Frederic W. Cook & Co., Inc.
Frederic W. Cook & Co., Inc.
F.W. Cook
Frederic W. Cook & Co., Inc.
F.W. Cook
F.W. Cook
F.W. Cook
Frederic W. Cook & Co., Inc.
F.W. Cook
F.W. Cook
F.W. Cook
F.W. Cook
F.W. Cook
Cook
F.W. Cook
Frederic W. Cook & Co., Inc.
F.W. Cook
F.W. Cook
F.W. Cook
Cook
F.W. Cook
F.W. Cook
F.W. Cook
F.W. Cook
Frederic W. Cook & Co., Inc.
F.W. Cook
F.W. Cook
F.W. Cook
F.W. Cook
F.W. Cook
F.W. Cook
Frederic W. Cook & Co., Inc.
F.W. Cook
F.W. Cook
F.W. Cook
F.W. Cook
F.W. Cook
F.W. Cook
F.W. Cook
Frederic W. Cook & Co.
F.W. Cook
F.W. Cook
Frederic W. Cook & Co., Inc.
F.W. Cook
F.W. Cook
F.W. Cook
F.W. Cook
Frederic W. Cook & Co.
F.W. Cook
F.W. Cook
F.W. Cook
F.W. Cook
F.W. Cook
F.W. Cook
F.W. Cook
F.W. Cook
F.W. Cook
F.W. Cook
F.W. Cook
Cook
Mr. Cook in
Frederic W. Cook & Co., Inc.
F.W. Cook
F.W. Cook
F.W. Cook
F.W. Cook
F.W. Cook
Cook
F.W. Cook
Cook
Cook
Cook
of F. Hoffman
F.W. Cook
F.W. Cook
F.W. Cook
F.W. Cook
F.W. Cook
F.W. Cook
F.W. Cook
F.W. Cook
F.W. Cook
F.W. Cook
F.

,file_name,firm,year,cik,tic,text,result,CC,CC_sentence,CC1,CC2
0,F:\APPLE INC\320193-APPLE INC-DEF 14A-2007-04-...,APPLE INC,2007,320193,AAPL,"[Independent consultants and advisors., The Co...","({FREDERIC W. COOK & CO., INC., F.W. COOK}, {D...","{FREDERIC W. COOK & CO., INC., F.W. COOK}","{During fiscal year 2006, the Committee engage...","{FREDERIC W. COOK & CO., INC., F.W. COOK}",{COOK}
1,F:\APPLE INC\320193-APPLE INC-DEF 14A-2008-01-...,APPLE INC,2008,320193,AAPL,"[5. The Role of Consultants , The Compensation...","({F.W. COOK, FREDERIC W. COOK & CO., INC.}, {T...","{F.W. COOK, FREDERIC W. COOK & CO., INC.}",{The Compensation Committee periodically seeks...,"{F.W. COOK, FREDERIC W. COOK & CO., INC.}",{COOK}
2,F:\APPLE INC\320193-APPLE INC-DEF 14A-2009-01-...,APPLE INC,2009,320193,AAPL,"[""), where he serves on the Leadership Develop...","({COOK, F.W. COOK, FREDERIC W. COOK & CO., INC...","{COOK, F.W. COOK, FREDERIC W. COOK & CO., INC.}","{With the assistance of F.W. Cook, the Compens...","{COOK, F.W. COOK, FREDERIC W. COOK & CO., INC.}",{COOK}
3,F:\APPLE INC\320193-APPLE INC-DEF 14A-2010-01-...,APPLE INC,2010,320193,AAPL,"["") since September 1999. Previously, Dr. Levi...","({F.W. COOK, FREDERIC W. COOK & CO., INC.}, {W...","{F.W. COOK, FREDERIC W. COOK & CO., INC.}","{With the assistance of F.W. Cook, the Compens...","{F.W. COOK, FREDERIC W. COOK & CO., INC.}",{COOK}
4,F:\APPLE INC\320193-APPLE INC-DEF 14A-2011-01-...,APPLE INC,2011,320193,AAPL,"["") since September 1999. Previously, Dr. Levi...","({F.W. COOK, FREDERIC W. COOK & CO., INC.}, {W...","{F.W. COOK, FREDERIC W. COOK & CO., INC.}","{With the assistance of F.W. Cook, the Compens...","{F.W. COOK, FREDERIC W. COOK & CO., INC.}",{COOK}
5,F:\APPLE INC\320193-APPLE INC-DEF 14A-2012-01-...,APPLE INC,2012,320193,AAPL,"["") since September 1999. Previously, Dr. Levi...","({FREDERIC W. COOK & CO., F.W. COOK, FREDERIC ...","{FREDERIC W. COOK & CO., F.W. COOK, FREDERIC W...","{With the assistance of F.W. Cook, the Compens...","{FREDERIC W. COOK & CO., F.W. COOK, FREDERIC W...",{COOK}
6,F:\APPLE INC\320193-APPLE INC-DEF 14A-2013-01-...,APPLE INC,2013,320193,AAPL,"["") since September 1999. Previously, Dr. Levi...","({COOK, FREDERIC W. COOK & CO., INC., F.W. COO...","{COOK, FREDERIC W. COOK & CO., INC., F.W. COOK...","{In connection with this program, the Compensa...","{COOK, FREDERIC W. COOK & CO., INC., F.W. COOK...",{COOK}
7,F:\APPLE INC\320193-APPLE INC-DEF 14A-2014-01-...,APPLE INC,2014,320193,AAPL,"["") since September 1999, and a member of the ...","({OF F. HOFFMAN, F.W. COOK}, {The Compensation...","{OF F. HOFFMAN, F.W. COOK}",{The Compensation Committee selected and direc...,"{OF F. HOFFMAN, F.W. COOK}","{COOK, OF F. HOFFMAN}"
8,F:\APPLE INC\320193-APPLE INC-DEF 14A-2015-01-...,APPLE INC,2015,320193,AAPL,[ Development center and as a member of Genent...,"({MR. COOK, PAY GOVERNANCE, FREDERIC W. COOK &...","{MR. COOK, PAY GOVERNANCE, FREDERIC W. COOK & ...",{The Compensation Committee sought input from ...,"{MR. COOK, PAY GOVERNANCE, FREDERIC W. COOK & ...","{MR. COOK, COOK, PAY GOVERNANCE}"
9,F:\APPLE INC\320193-APPLE INC-DEF 14A-2016-01-...,APPLE INC,2016,320193,AAPL,[ has served as the Chief Executive Officer of...,"({OF F. HOFFMAN, PAY GOVERNANCE}, {During 2015...","{OF F. HOFFMAN, PAY GOVERNANCE}","{During 2015, Pay Governance did not provide a...","{OF F. HOFFMAN, PAY GOVERNANCE}","{OF F. HOFFMAN, PAY GOVERNANCE}"


In [109]:
df['CC_sentence'].iloc[7]

{'Dr. Levinson has also served as a director of F. Hoffman-La Roche Ltd since March 2010, where he serves on the Remuneration Committee, and in September 2013, was named the Chief Executive Officer of Calico LLC, a company focused on health, aging and well-being.',
 "F.W. Cook did not provide any other services to the Company and worked with the Company's management only on matters for which the Compensation Committee is responsible.",
 "Finally, F.W. Cook provided general observations in connection with the Compensation Committee's consideration of the proposed Apple Inc. 2014 Employee Stock Plan, but it did not determine or recommend any specific share limits for the plan.",
 'In considering the time frame to measure relative TSR, the Compensation Committee determined, after consultation with F.W. Cook and shareholders, to use a three-year period and to ramp up to this three-year period starting with the tranche vesting in 2014.',
 'The Compensation Committee assessed the independenc

# 5. F:\ADVANCE AUTO PARTS INC

In [39]:
dir='F:\\ADVANCE AUTO PARTS INC'

In [40]:
os.chdir(dir)
os.getcwd()
proxies=os.chdir(dir)
os.getcwd()
proxies=find_proxies(dir)
data={}
pathList=[]
for i in proxies:
    filePath=os.path.join(dir, i)
    print(filePath)
    pathList.append(filePath) # create a list of file paths.

filename, text=open_proxy1(pathList)
data.update({'file_name':filename, 'text':text})

df=pd.DataFrame.from_dict(data)
# create columns with firm info
from tqdm.auto import tqdm
for i, row in tqdm(df.iterrows(), total=len(df)): # df.iterrows() will iterate each row
    filename = str(row["file_name"])  # row['column name'] to select certain column within a row
    print('filename is', filename)
    firm, year, cik, tic= extract_firm_info(filename)
    df.at[i, "firm"] = firm   # then, create a new column that contains the result of the operation.
    df.at[i, 'year'] = year
    df.at[i, 'cik'] = cik
    df.at[i, 'tic'] = tic

column_order = ["file_name", "firm", "year", 'cik','tic','text']
df = df.reindex(columns=column_order)

df['result']=df['text'].apply(NER2)
df['CC']=df['result'].apply(lambda x: x[0])
df['CC_sentence']=df['result'].apply(lambda x: x[1])
df['CC1']=df['CC'].apply(make_uppercase)
df['CC2']=df['CC1'].apply(remove_duplicates)
df


F:\ADVANCE AUTO PARTS INC\1158449-ADVANCE AUTO PARTS INC-DEF 14A-2007-04-11.html
F:\ADVANCE AUTO PARTS INC\1158449-ADVANCE AUTO PARTS INC-DEF 14A-2008-04-09.html
F:\ADVANCE AUTO PARTS INC\1158449-ADVANCE AUTO PARTS INC-DEF 14A-2009-04-14.html
F:\ADVANCE AUTO PARTS INC\1158449-ADVANCE AUTO PARTS INC-DEF 14A-2010-04-09.html
F:\ADVANCE AUTO PARTS INC\1158449-ADVANCE AUTO PARTS INC-DEF 14A-2011-04-07.html
F:\ADVANCE AUTO PARTS INC\1158449-ADVANCE AUTO PARTS INC-DEF 14A-2012-04-16.html
F:\ADVANCE AUTO PARTS INC\1158449-ADVANCE AUTO PARTS INC-DEF 14A-2013-04-18.html
F:\ADVANCE AUTO PARTS INC\1158449-ADVANCE AUTO PARTS INC-DEF 14A-2014-04-09.html
F:\ADVANCE AUTO PARTS INC\1158449-ADVANCE AUTO PARTS INC-DEF 14A-2015-04-06.html
F:\ADVANCE AUTO PARTS INC\1158449-ADVANCE AUTO PARTS INC-DEF 14A-2016-04-14.html
100%|██████████| 10/10 [00:00<00:00, 2501.22it/s]
filename is F:\ADVANCE AUTO PARTS INC\1158449-ADVANCE AUTO PARTS INC-DEF 14A-2007-04-11.html
filename is F:\ADVANCE AUTO PARTS INC\1158449-A

,file_name,firm,year,cik,tic,text,result,CC,CC_sentence,CC1,CC2
0,F:\ADVANCE AUTO PARTS INC\1158449-ADVANCE AUTO...,ADVANCE AUTO PARTS INC,2007,1158449,AAP,[The Compensation Committee may select a...,"({HAYGROUP}, {The Committee has engaged ...",{HAYGROUP},"{The Committee has engaged HayGroup, an ...",{HAYGROUP},{HAYGROUP}
1,F:\ADVANCE AUTO PARTS INC\1158449-ADVANCE AUTO...,ADVANCE AUTO PARTS INC,2008,1158449,AAP,"[The Committee decides the type, mix and vesti...","({FREDERIC W. COOK & CO., INC., HAY GROUP}, {A...","{FREDERIC W. COOK & CO., INC., HAY GROUP}",{After adjusting the data using standard stati...,"{FREDERIC W. COOK & CO., INC., HAY GROUP}","{HAY GROUP, COOK}"
2,F:\ADVANCE AUTO PARTS INC\1158449-ADVANCE AUTO...,ADVANCE AUTO PARTS INC,2009,1158449,AAP,"[Mr. Michael Marolt, Senior Vice President, Cu...","({FREDERIC W. COOK & CO., INC., HAY GROUP, COO...","{FREDERIC W. COOK & CO., INC., HAY GROUP, COOK}",{The Committee has determined that in order to...,"{FREDERIC W. COOK & CO., INC., HAY GROUP, COOK}","{HAY GROUP, COOK}"
3,F:\ADVANCE AUTO PARTS INC\1158449-ADVANCE AUTO...,ADVANCE AUTO PARTS INC,2010,1158449,AAP,[ Cook also provides information and advice o...,"({FREDERIC W. COOK & CO., INC., HAY GROUP, COO...","{FREDERIC W. COOK & CO., INC., HAY GROUP, COOK}",{After adjusting the data using standard stati...,"{FREDERIC W. COOK & CO., INC., HAY GROUP, COOK}","{HAY GROUP, COOK}"
4,F:\ADVANCE AUTO PARTS INC\1158449-ADVANCE AUTO...,ADVANCE AUTO PARTS INC,2011,1158449,AAP,[performance-based vesting of our SARs and res...,"({FREDERIC W. COOK & CO., INC., HAY GROUP, COO...","{FREDERIC W. COOK & CO., INC., HAY GROUP, COOK}",{In determining appropriate compensation oppor...,"{FREDERIC W. COOK & CO., INC., HAY GROUP, COOK}","{HAY GROUP, COOK}"
5,F:\ADVANCE AUTO PARTS INC\1158449-ADVANCE AUTO...,ADVANCE AUTO PARTS INC,2012,1158449,AAP,"[Committee, with the guidance and assistance o...","({FREDERIC W. COOK & CO., INC., HAY GROUP, COO...","{FREDERIC W. COOK & CO., INC., HAY GROUP, COOK}",{Cook provides information regarding market co...,"{FREDERIC W. COOK & CO., INC., HAY GROUP, COOK}","{HAY GROUP, COOK}"
6,F:\ADVANCE AUTO PARTS INC\1158449-ADVANCE AUTO...,ADVANCE AUTO PARTS INC,2013,1158449,AAP,[the compensation of our executives is primari...,"({FREDERIC W. COOK & CO., INC., HAY GROUP, COO...","{FREDERIC W. COOK & CO., INC., HAY GROUP, COOK}","{, Cook did not provide any services to the Co...","{FREDERIC W. COOK & CO., INC., HAY GROUP, COOK}","{HAY GROUP, COOK}"
7,F:\ADVANCE AUTO PARTS INC\1158449-ADVANCE AUTO...,ADVANCE AUTO PARTS INC,2014,1158449,AAP,[We have reviewed all of our compensation prog...,"({FREDERIC W. COOK & CO., INC., HAY GROUP, COO...","{FREDERIC W. COOK & CO., INC., HAY GROUP, COOK}",{Following the completion of our acquisition i...,"{FREDERIC W. COOK & CO., INC., HAY GROUP, COOK}","{HAY GROUP, COOK}"
8,F:\ADVANCE AUTO PARTS INC\1158449-ADVANCE AUTO...,ADVANCE AUTO PARTS INC,2015,1158449,AAP,[We have reviewed all of our compensation prog...,"({FREDERIC W. COOK & CO., INC., HAY GROUP, COO...","{FREDERIC W. COOK & CO., INC., HAY GROUP, COOK}",{The Committee reviews the information provide...,"{FREDERIC W. COOK & CO., INC., HAY GROUP, COOK}","{HAY GROUP, COOK}"
9,F:\ADVANCE AUTO PARTS INC\1158449-ADVANCE AUTO...,ADVANCE AUTO PARTS INC,2016,1158449,AAP,[We have reviewed all of our compensation prog...,"({FREDERIC W. COOK & CO., INC., COOK}, {Cook r...","{FREDERIC W. COOK & CO., INC., COOK}","{Cook reports directly to the Committee, and a...","{FREDERIC W. COOK & CO., INC., COOK}",{COOK}


In [114]:
df['CC_sentence'].iloc[3]

{'A principal of Cook attends meetings of the Committee, as requested, and communicates with the Chair of the Compensation Committee, as necessary or advisable, between meetings.',
 'After adjusting the data using standard statistical methods based on revenue to make the information more comparable for a company our size, Hay Group provided the retail compensation data to the Committee and Cook in a summary form.',
 'Cook reports directly to the Committee, and all services provided by Cook are provided on behalf of the Committee.',
 'For 2009, the Committee established base salary, annual incentive opportunities, and long-term incentive target grants for our named executive officers primarily with reference to the peer group data provided by Cook.',
 'For Mr.       Jackson, the definition of “Good Reason” includes failure of the       Nominating Committee of the Board',
 'Frederic W. Cook & Co., Inc. (“Cook”), an independent consulting firm, to provide advice and assistance to the Comm

# 6. Ebay

In [41]:
dir=r'F:\EBAY INC'


In [42]:
os.chdir(dir)
os.getcwd()
proxies=find_proxies(dir)
data={}
pathList=[]
for i in proxies:os.chdir(dir)
os.getcwd()
proxies=os.chdir(dir)
os.getcwd()
proxies=find_proxies(dir)
data={}
pathList=[]
for i in proxies:
    filePath=os.path.join(dir, i)
    print(filePath)
    pathList.append(filePath) # create a list of file paths.

filename, text=open_proxy1(pathList)
data.update({'file_name':filename, 'text':text})

df=pd.DataFrame.from_dict(data)
# create columns with firm info
from tqdm.auto import tqdm
for i, row in tqdm(df.iterrows(), total=len(df)): # df.iterrows() will iterate each row
    filename = str(row["file_name"])  # row['column name'] to select certain column within a row
    print('filename is', filename)
    firm, year, cik, tic= extract_firm_info(filename)
    df.at[i, "firm"] = firm   # then, create a new column that contains the result of the operation.
    df.at[i, 'year'] = year
    df.at[i, 'cik'] = cik
    df.at[i, 'tic'] = tic

column_order = ["file_name", "firm", "year", 'cik','tic','text']
df = df.reindex(columns=column_order)

df['result']=df['text'].apply(NER2)
df['CC']=df['result'].apply(lambda x: x[0])
df['CC_sentence']=df['result'].apply(lambda x: x[1])
df['CC1']=df['CC'].apply(make_uppercase)
df['CC2']=df['CC1'].apply(remove_duplicates)



F:\EBAY INC\1065088-EBAY INC-DEF 14A-2007-04-30.html
F:\EBAY INC\1065088-EBAY INC-DEF 14A-2008-04-28.html
F:\EBAY INC\1065088-EBAY INC-DEF 14A-2009-03-19.html
F:\EBAY INC\1065088-EBAY INC-DEF 14A-2010-03-19.html
F:\EBAY INC\1065088-EBAY INC-DEF 14A-2011-03-21.html
F:\EBAY INC\1065088-EBAY INC-DEF 14A-2012-03-19.html
F:\EBAY INC\1065088-EBAY INC-DEF 14A-2013-03-18.html
F:\EBAY INC\1065088-EBAY INC-DEF 14A-2015-03-23.html
F:\EBAY INC\1065088-EBAY INC-DEF 14A-2016-03-23.html
100%|██████████| 9/9 [00:00<00:00, 1800.04it/s]
filename is F:\EBAY INC\1065088-EBAY INC-DEF 14A-2007-04-30.html
filename is F:\EBAY INC\1065088-EBAY INC-DEF 14A-2008-04-28.html
filename is F:\EBAY INC\1065088-EBAY INC-DEF 14A-2009-03-19.html
filename is F:\EBAY INC\1065088-EBAY INC-DEF 14A-2010-03-19.html
filename is F:\EBAY INC\1065088-EBAY INC-DEF 14A-2011-03-21.html
filename is F:\EBAY INC\1065088-EBAY INC-DEF 14A-2012-03-19.html
filename is F:\EBAY INC\1065088-EBAY INC-DEF 14A-2013-03-18.html
filename is F:\EBAY 

In [43]:
df

,file_name,firm,year,cik,tic,text,result,CC,CC_sentence,CC1,CC2
0,F:\EBAY INC\1065088-EBAY INC-DEF 14A-2007-04-3...,EBAY INC,2007,1065088,EBAYL,[Role of Executive Officers and Consultant...,"({MERCER HUMAN RESOURCES CONSULTING, MERCE...","{MERCER HUMAN RESOURCES CONSULTING, MERCER...","{As discussed above, in 2006 the committee ret...","{MERCER HUMAN RESOURCES CONSULTING, MERCER...","{MERCER, TOWERS PERRIN}"
1,F:\EBAY INC\1065088-EBAY INC-DEF 14A-2008-04-2...,EBAY INC,2008,1065088,EBAYL,[Role of Executive Officers and Consultants in...,"({TOWERS PERRIN, TOWERS PERRIN}, {As part ...","{TOWERS PERRIN, TOWERS PERRIN}","{As part of its engagement, the Compensati...","{TOWERS PERRIN, TOWERS PERRIN}",{TOWERS PERRIN}
2,F:\EBAY INC\1065088-EBAY INC-DEF 14A-2009-03-1...,EBAY INC,2009,1065088,EBAYL,[Role of Executive Officers and Consultants in...,"({TOWERS PERRIN, TOWERS PERRIN}, {Towers P...","{TOWERS PERRIN, TOWERS PERRIN}",{Towers Perrin also meets with the committee ...,"{TOWERS PERRIN, TOWERS PERRIN}",{TOWERS PERRIN}
3,F:\EBAY INC\1065088-EBAY INC-DEF 14A-2010-03-1...,EBAY INC,2010,1065088,EBAYL,[Role of Executive Officers and Consultants in...,"({PAY GOVERNANCE LLC, TOWERS WATSON, TOWERS WA...","{PAY GOVERNANCE LLC, TOWERS WATSON, TOWERS WAT...","{In 2009, with the committee's approval, manag...","{PAY GOVERNANCE LLC, TOWERS WATSON, TOWERS WAT...","{PAY GOVERNANCE LLC, TOWERS PERRIN, WATSON WYATT}"
4,F:\EBAY INC\1065088-EBAY INC-DEF 14A-2011-03-2...,EBAY INC,2011,1065088,EBAYL,[Role of Executive Officers and Consultants in...,"({PAY GOVERNANCE, TOWERS WATSON & CO., TOWERS ...","{PAY GOVERNANCE, TOWERS WATSON & CO., TOWERS W...",{Pay Governance also meets with the committee ...,"{PAY GOVERNANCE, TOWERS WATSON & CO., TOWERS W...","{PAY GOVERNANCE, TOWERS WATSON}"
5,F:\EBAY INC\1065088-EBAY INC-DEF 14A-2012-03-1...,EBAY INC,2012,1065088,EBAYL,[Role of Executive Officers and Consultants in...,"({PAY GOVERNANCE, PAY GOVERNANCE LLC}, {Pay Go...","{PAY GOVERNANCE, PAY GOVERNANCE LLC}",{Pay Governance also meets with the committee ...,"{PAY GOVERNANCE, PAY GOVERNANCE LLC}",{PAY GOVERNANCE}
6,F:\EBAY INC\1065088-EBAY INC-DEF 14A-2013-03-1...,EBAY INC,2013,1065088,EBAYL,[Role of Executive Officers and Consultants in...,"({PAY GOVERNANCE, PAY GOVERNANCE LLC, TOWERS W...","{PAY GOVERNANCE, PAY GOVERNANCE LLC, TOWERS WA...",{Pay Governance also meets with the committee ...,"{PAY GOVERNANCE, PAY GOVERNANCE LLC, TOWERS WA...","{PAY GOVERNANCE, TOWERS WATSON}"
7,F:\EBAY INC\1065088-EBAY INC-DEF 14A-2015-03-2...,EBAY INC,2015,1065088,EBAYL,[The Compensation Committee retains an indepen...,"({PAY GOVERNANCE, PAY GOVERNANCE LLC}, {Pay Go...","{PAY GOVERNANCE, PAY GOVERNANCE LLC}",{Pay Governance's fees for consulting advice t...,"{PAY GOVERNANCE, PAY GOVERNANCE LLC}",{PAY GOVERNANCE}
8,F:\EBAY INC\1065088-EBAY INC-DEF 14A-2016-03-2...,EBAY INC,2016,1065088,EBAYL,[The Compensation Committee retains an i...,"({PAY GOVERNANCE, TOWERS WATSON}, {The Compens...","{PAY GOVERNANCE, TOWERS WATSON}",{The Compensation Committee also reviewed Towe...,"{PAY GOVERNANCE, TOWERS WATSON}","{PAY GOVERNANCE, TOWERS WATSON}"


# 7. F:\FOSSIL INC

In [44]:
dir=r'F:\FOSSIL INC'

In [45]:
os.chdir(dir)
os.getcwd()
proxies=find_proxies(dir)
data={}
pathList=[]
for i in proxies:os.chdir(dir)
os.getcwd()
proxies=os.chdir(dir)
os.getcwd()
proxies=find_proxies(dir)
data={}
pathList=[]
for i in proxies:
    filePath=os.path.join(dir, i)
    print(filePath)
    pathList.append(filePath) # create a list of file paths.

filename, text=open_proxy1(pathList)
data.update({'file_name':filename, 'text':text})

df=pd.DataFrame.from_dict(data)
# create columns with firm info
from tqdm.auto import tqdm
for i, row in tqdm(df.iterrows(), total=len(df)): # df.iterrows() will iterate each row
    filename = str(row["file_name"])  # row['column name'] to select certain column within a row
    print('filename is', filename)
    firm, year, cik, tic= extract_firm_info(filename)
    df.at[i, "firm"] = firm   # then, create a new column that contains the result of the operation.
    df.at[i, 'year'] = year
    df.at[i, 'cik'] = cik
    df.at[i, 'tic'] = tic

column_order = ["file_name", "firm", "year", 'cik','tic','text']
df = df.reindex(columns=column_order)

df['result']=df['text'].apply(NER2)
df['CC']=df['result'].apply(lambda x: x[0])
df['CC_sentence']=df['result'].apply(lambda x: x[1])
df['CC1']=df['CC'].apply(make_uppercase)
df['CC2']=df['CC1'].apply(remove_duplicates)

F:\FOSSIL INC\883569-FOSSIL INC-DEF 14A-2007-10-10.html
F:\FOSSIL INC\883569-FOSSIL INC-DEF 14A-2008-04-10.html
F:\FOSSIL INC\883569-FOSSIL INC-DEF 14A-2009-04-10.html
F:\FOSSIL INC\883569-FOSSIL INC-DEF 14A-2010-04-15.html
F:\FOSSIL INC\883569-FOSSIL INC-DEF 14A-2011-04-15.html
F:\FOSSIL INC\883569-FOSSIL INC-DEF 14A-2012-04-23.html
F:\FOSSIL INC\883569-FOSSIL INC-DEF 14A-2013-04-10.html
100%|██████████| 7/7 [00:00<00:00, 1763.16it/s]
filename is F:\FOSSIL INC\883569-FOSSIL INC-DEF 14A-2007-10-10.html
filename is F:\FOSSIL INC\883569-FOSSIL INC-DEF 14A-2008-04-10.html
filename is F:\FOSSIL INC\883569-FOSSIL INC-DEF 14A-2009-04-10.html
filename is F:\FOSSIL INC\883569-FOSSIL INC-DEF 14A-2010-04-15.html
filename is F:\FOSSIL INC\883569-FOSSIL INC-DEF 14A-2011-04-15.html
filename is F:\FOSSIL INC\883569-FOSSIL INC-DEF 14A-2012-04-23.html
filename is F:\FOSSIL INC\883569-FOSSIL INC-DEF 14A-2013-04-10.html
E&Y
Ernst and Young
Frederic W. Cooke & Co, Inc.
FWC
FWC
Frederic W. Cooke & Co, Inc

In [46]:
df

,file_name,firm,year,cik,tic,text,result,CC,CC_sentence,CC1,CC2
0,F:\FOSSIL INC\883569-FOSSIL INC-DEF 14A-2007-1...,FOSSIL INC,2007,883569,FOSL,"[Compensation Consultant, Historically, there ...","({E&Y}, {In fiscal 2006, the Compensation Comm...",{E&Y},"{In fiscal 2006, the Compensation Committee co...",{E&Y},{E&Y}
1,F:\FOSSIL INC\883569-FOSSIL INC-DEF 14A-2008-0...,FOSSIL INC,2008,883569,FOSL,"[ Historically, there has not been a pr...","({ERNST AND YOUNG}, {In fiscal 2007, the Compe...",{ERNST AND YOUNG},"{In fiscal 2007, the Compensation Committee co...",{ERNST AND YOUNG},{ERNST AND YOUNG}
2,F:\FOSSIL INC\883569-FOSSIL INC-DEF 14A-2009-0...,FOSSIL INC,2009,883569,FOSL,"[ Historically, there has not been a pr...","({}, {})",{},{},{},{}
3,F:\FOSSIL INC\883569-FOSSIL INC-DEF 14A-2010-0...,FOSSIL INC,2010,883569,FOSL,"[ Historically, there has not been a pr...","({FREDERIC W. COOKE & CO, INC., FWC}, {In late...","{FREDERIC W. COOKE & CO, INC., FWC}","{In late 2009, the Compensation Committee enga...","{FREDERIC W. COOKE & CO, INC., FWC}","{FREDERIC W. COOKE & CO, INC., FWC}"
4,F:\FOSSIL INC\883569-FOSSIL INC-DEF 14A-2011-0...,FOSSIL INC,2011,883569,FOSL,[ Governance Committee. Mr. Zimmerman currentl...,"({FREDERIC W. COOKE & CO, INC., FWC}, {In late...","{FREDERIC W. COOKE & CO, INC., FWC}","{In late 2009, the Compensation Committee enga...","{FREDERIC W. COOKE & CO, INC., FWC}","{FREDERIC W. COOKE & CO, INC., FWC}"
5,F:\FOSSIL INC\883569-FOSSIL INC-DEF 14A-2012-0...,FOSSIL INC,2012,883569,FOSL,[Based upon the recommendation of Frederic W. ...,"({FREDERIC W. COOKE & CO., INC., FWC}, {Based ...","{FREDERIC W. COOKE & CO., INC., FWC}",{Based upon the recommendation of Frederic W. ...,"{FREDERIC W. COOKE & CO., INC., FWC}","{FREDERIC W. COOKE & CO., INC., FWC}"
6,F:\FOSSIL INC\883569-FOSSIL INC-DEF 14A-2013-0...,FOSSIL INC,2013,883569,FOSL,[ Thomas M. Kennedy has served as Execu...,"({FREDRICK W. COOKE & CO., INC., FWC}, {Based ...","{FREDRICK W. COOKE & CO., INC., FWC}",{Based upon the 2011 recommendation of Fredric...,"{FREDRICK W. COOKE & CO., INC., FWC}","{FREDRICK W. COOKE & CO., INC., FWC}"


# 8. F:\WELLS FARGO & CO_MN

In [47]:
dir=r'F:\WELLS FARGO & CO_MN'

In [48]:
os.chdir(dir)
os.getcwd()
proxies=find_proxies(dir)
data={}
pathList=[]
for i in proxies:os.chdir(dir)
os.getcwd()
proxies=os.chdir(dir)
os.getcwd()
proxies=find_proxies(dir)
data={}
pathList=[]
for i in proxies:
    filePath=os.path.join(dir, i)
    print(filePath)
    pathList.append(filePath) # create a list of file paths.

filename, text=open_proxy1(pathList)
data.update({'file_name':filename, 'text':text})

df=pd.DataFrame.from_dict(data)
# create columns with firm info
from tqdm.auto import tqdm
for i, row in tqdm(df.iterrows(), total=len(df)): # df.iterrows() will iterate each row
    filename = str(row["file_name"])  # row['column name'] to select certain column within a row
    print('filename is', filename)
    firm, year, cik, tic= extract_firm_info(filename)
    df.at[i, "firm"] = firm   # then, create a new column that contains the result of the operation.
    df.at[i, 'year'] = year
    df.at[i, 'cik'] = cik
    df.at[i, 'tic'] = tic

column_order = ["file_name", "firm", "year", 'cik','tic','text']
df = df.reindex(columns=column_order)

df['result']=df['text'].apply(NER2)
df['CC']=df['result'].apply(lambda x: x[0])
df['CC_sentence']=df['result'].apply(lambda x: x[1])
df['CC1']=df['CC'].apply(make_uppercase)
df['CC2']=df['CC1'].apply(remove_duplicates)

F:\WELLS FARGO & CO_MN\72971-WELLS FARGO & CO_MN-DEF 14A-2007-03-16.html
F:\WELLS FARGO & CO_MN\72971-WELLS FARGO & CO_MN-DEF 14A-2008-03-17.html
F:\WELLS FARGO & CO_MN\72971-WELLS FARGO & CO_MN-DEF 14A-2009-03-18.html
F:\WELLS FARGO & CO_MN\72971-WELLS FARGO & CO_MN-DEF 14A-2010-03-17.html
F:\WELLS FARGO & CO_MN\72971-WELLS FARGO & CO_MN-DEF 14A-2011-03-21.html
F:\WELLS FARGO & CO_MN\72971-WELLS FARGO & CO_MN-DEF 14A-2012-03-15.html
100%|██████████| 6/6 [00:00<00:00, 1510.19it/s]
filename is F:\WELLS FARGO & CO_MN\72971-WELLS FARGO & CO_MN-DEF 14A-2007-03-16.html
filename is F:\WELLS FARGO & CO_MN\72971-WELLS FARGO & CO_MN-DEF 14A-2008-03-17.html
filename is F:\WELLS FARGO & CO_MN\72971-WELLS FARGO & CO_MN-DEF 14A-2009-03-18.html
filename is F:\WELLS FARGO & CO_MN\72971-WELLS FARGO & CO_MN-DEF 14A-2010-03-17.html
filename is F:\WELLS FARGO & CO_MN\72971-WELLS FARGO & CO_MN-DEF 14A-2011-03-21.html
filename is F:\WELLS FARGO & CO_MN\72971-WELLS FARGO & CO_MN-DEF 14A-2012-03-15.html
F.W.

In [49]:
df

,file_name,firm,year,cik,tic,text,result,CC,CC_sentence,CC1,CC2
0,F:\WELLS FARGO & CO_MN\72971-WELLS FARGO & CO_...,WELLS FARGO & CO_MN,2007,72971,WFC-PA,[ With respect to director compensation mat...,"({F.W. COOK}, {Inc. (F.W. Cook), a compensatio...",{F.W. COOK},"{Inc. (F.W. Cook), a compensation consulting f...",{F.W. COOK},{COOK}
1,F:\WELLS FARGO & CO_MN\72971-WELLS FARGO & CO_...,WELLS FARGO & CO_MN,2008,72971,WFC-PA,"[HRC and GNC Use of Compensation Consultant , ...","({F.W. COOK}, {the GNC has retained F.W. Cook ...",{F.W. COOK},{the GNC has retained F.W. Cook to act as the ...,{F.W. COOK},{COOK}
2,F:\WELLS FARGO & CO_MN\72971-WELLS FARGO & CO_...,WELLS FARGO & CO_MN,2009,72971,WFC-PA,[ HRC and GNC Use of Compensation Cons...,"({COOK & CO.}, {Co. The total amount of fees t...",{COOK & CO.},{Co. The total amount of fees the Company paid...,{COOK & CO.},{COOK}
3,F:\WELLS FARGO & CO_MN\72971-WELLS FARGO & CO_...,WELLS FARGO & CO_MN,2010,72971,WFC-PA,[ HRC and GNC Use of Compensation Consult...,"({COOK & CO.}, {To set approximate competitive...",{COOK & CO.},{To set approximate competitive benchmarks for...,{COOK & CO.},{COOK}
4,F:\WELLS FARGO & CO_MN\72971-WELLS FARGO & CO_...,WELLS FARGO & CO_MN,2011,72971,WFC-PA,[ HRC and GNC Use of Compensation Consu...,"({COOK & CO.}, {To set approximate competitive...",{COOK & CO.},{To set approximate competitive benchmarks for...,{COOK & CO.},{COOK}
5,F:\WELLS FARGO & CO_MN\72971-WELLS FARGO & CO_...,WELLS FARGO & CO_MN,2012,72971,WFC-PA,[ HRC and GNC Use of Compensation Consul...,"({COOK & CO.}, {Co. The total amount of fees t...",{COOK & CO.},{Co. The total amount of fees the Company paid...,{COOK & CO.},{COOK}


# 9.F:\TIME WARNER INC

In [50]:
dir=r'F:\TIME WARNER INC'

In [51]:
f=r'F:\TIME WARNER INC\1105705-TIME WARNER INC-DEF 14A-2007-04-09.html'
firm, year, cik, tic= extract_firm_info(f)

In [52]:
os.chdir(dir)
os.getcwd()
proxies=find_proxies(dir)
data={}
pathList=[]
for i in proxies:
    filePath=os.path.join(dir, i)
    print(filePath)
    pathList.append(filePath) # create a list of file paths.

filename, text=open_proxy1(pathList)
data.update({'file_name':filename, 'text':text})

df=pd.DataFrame.from_dict(data)
# create columns with firm info
from tqdm.auto import tqdm
for i, row in tqdm(df.iterrows(), total=len(df)): # df.iterrows() will iterate each row
    filename = str(row["file_name"])  # row['column name'] to select certain column within a row
    print('filename is', filename)
    firm, year, cik, tic= extract_firm_info(filename)
    df.at[i, "firm"] = firm   # then, create a new column that contains the result of the operation.
    df.at[i, 'year'] = year
    df.at[i, 'cik'] = cik
    df.at[i, 'tic'] = tic

column_order = ["file_name", "firm", "year", 'cik','tic','text']
df = df.reindex(columns=column_order)

df['result']=df['text'].apply(NER2)
df['CC']=df['result'].apply(lambda x: x[0])
df['CC_sentence']=df['result'].apply(lambda x: x[1])
df['CC1']=df['CC'].apply(make_uppercase)
df['CC2']=df['CC1'].apply(remove_duplicates)



F:\TIME WARNER INC\1105705-TIME WARNER INC-DEF 14A-2007-04-09.html
F:\TIME WARNER INC\1105705-TIME WARNER INC.-DEF 14A-2008-03-31.html
F:\TIME WARNER INC\1105705-TIME WARNER INC.-DEF 14A-2008-12-01.html
F:\TIME WARNER INC\1105705-TIME WARNER INC.-DEF 14A-2009-04-08.html
F:\TIME WARNER INC\1105705-TIME WARNER INC.-DEF 14A-2010-04-06.html
F:\TIME WARNER INC\1105705-TIME WARNER INC.-DEF 14A-2011-04-08.html
F:\TIME WARNER INC\1105705-TIME WARNER INC.-DEF 14A-2012-04-02.html
F:\TIME WARNER INC\1105705-TIME WARNER INC.-DEF 14A-2013-04-08.html
F:\TIME WARNER INC\1105705-TIME WARNER INC.-DEF 14A-2014-04-21.html
F:\TIME WARNER INC\1105705-TIME WARNER INC.-DEF 14A-2015-04-24.html
F:\TIME WARNER INC\1105705-TIME WARNER INC.-DEF 14A-2016-04-29.html
100%|██████████| 11/11 [00:00<00:00, 2750.53it/s]
filename is F:\TIME WARNER INC\1105705-TIME WARNER INC-DEF 14A-2007-04-09.html
filename is F:\TIME WARNER INC\1105705-TIME WARNER INC.-DEF 14A-2008-03-31.html
filename is F:\TIME WARNER INC\1105705-TIME 

In [53]:
df

,file_name,firm,year,cik,tic,text,result,CC,CC_sentence,CC1,CC2
0,F:\TIME WARNER INC\1105705-TIME WARNER INC-DEF...,TIME WARNER INC,2007,1105705,NaN,"[Since May 19, 2006, Mr. Caufield has served a...","({JOHN ENGLAND,, TOWERS PERRIN}, {The aggregat...","{JOHN ENGLAND,, TOWERS PERRIN}",{The aggregate fees billed by Towers Perrin to...,"{JOHN ENGLAND,, TOWERS PERRIN}","{JOHN ENGLAND,, TOWERS PERRIN}"
1,F:\TIME WARNER INC\1105705-TIME WARNER INC.-DE...,TIME WARNER INC,2008,1105705,NaN,"[Since May 19, 2006, Mr. Caufield has served a...","({JOHN ENGLAND,, TOWERS PERRIN, E&Y}, {The Com...","{JOHN ENGLAND,, TOWERS PERRIN, E&Y}","{The Committee has retained John England, Mana...","{JOHN ENGLAND,, TOWERS PERRIN, E&Y}","{JOHN ENGLAND,, TOWERS PERRIN, E&Y}"
2,F:\TIME WARNER INC\1105705-TIME WARNER INC.-DE...,TIME WARNER INC,2008,1105705,NaN,[ Time Warner will bear all expenses of th...,"({}, {})",{},{},{},{}
3,F:\TIME WARNER INC\1105705-TIME WARNER INC.-DE...,TIME WARNER INC,2009,1105705,NaN,"[Since May 19, 2006, Mr. Caufield has served a...","({JOHN ENGLAND,, TOWERS PERRIN, E&Y}, {The Com...","{JOHN ENGLAND,, TOWERS PERRIN, E&Y}","{The Committee has retained John England, Mana...","{JOHN ENGLAND,, TOWERS PERRIN, E&Y}","{JOHN ENGLAND,, TOWERS PERRIN, E&Y}"
4,F:\TIME WARNER INC\1105705-TIME WARNER INC.-DE...,TIME WARNER INC,2010,1105705,NaN,"[Independent Compensation Consultant, The Comm...","({PAY GOVERNANCE LLC, TOWERS PERRIN, JOHN ENGL...","{PAY GOVERNANCE LLC, TOWERS PERRIN, JOHN ENGLA...",{The fact that Mr. England was then employed b...,"{STEVEN HALL & PARTNERS, PAY GOVERNANCE LLC, J...","{STEVEN HALL & PARTNERS, PAY GOVERNANCE LLC, J..."
5,F:\TIME WARNER INC\1105705-TIME WARNER INC.-DE...,TIME WARNER INC,2011,1105705,NaN,"[Independent Compensation Consultant, . The Co...","({PAY GOVERNANCE LLC, PAY GOVERNANCE LLC, ...","{PAY GOVERNANCE LLC, PAY GOVERNANCE LLC, T...","{In carrying out this responsibility, the ...","{PAY GOVERNANCE LLC, PAY GOVERNANCE LLC, T...","{HALL & PARTNERS, PAY GOVERNANCE LLC, TOWERS, ..."
6,F:\TIME WARNER INC\1105705-TIME WARNER INC.-DE...,TIME WARNER INC,2012,1105705,NaN,"[Independent Compensation Consultant , Ms...","({PAY GOVERNANCE LLC, FARIENT ADVISORS LLC}, {...","{PAY GOVERNANCE LLC, FARIENT ADVISORS LLC}","{In 2011, the Nominating Committee engaged Far...","{PAY GOVERNANCE LLC, FARIENT ADVISORS LLC}","{PAY GOVERNANCE LLC, FARIENT ADVISORS LLC}"
7,F:\TIME WARNER INC\1105705-TIME WARNER INC.-DE...,TIME WARNER INC,2013,1105705,NaN,"[Independent Compensation Consultant, Ms. Ein...","({PAY GOVERNANCE LLC}, {Based on its satisfact...",{PAY GOVERNANCE LLC},{Based on its satisfaction with the consultant...,{PAY GOVERNANCE LLC},{PAY GOVERNANCE LLC}
8,F:\TIME WARNER INC\1105705-TIME WARNER INC.-DE...,TIME WARNER INC,2014,1105705,NaN,"[Independent Compensation Consultant, Ms. Ein...","({FARIENT, FARIENT ADVISORS LLC, PAY GOVERNANC...","{FARIENT, FARIENT ADVISORS LLC, PAY GOVERNANCE...",{The Compensation Committee retained Pay Gover...,"{FARIENT, FARIENT ADVISORS LLC, PAY GOVERNANCE...","{FARIENT, PAY GOVERNANCE LLC}"
9,F:\TIME WARNER INC\1105705-TIME WARNER INC.-DE...,TIME WARNER INC,2015,1105705,NaN,"[Independent Compensation Consultant, Ms. Einh...","({PAY GOVERNANCE LLC}, {Pay Governance LLC pro...",{PAY GOVERNANCE LLC},{Pay Governance LLC provides advice to the Com...,{PAY GOVERNANCE LLC},{PAY GOVERNANCE LLC}


In [55]:
df['CC_sentence'].iloc[1]

{'Accordingly, the Audit and Finance Committee has appointed Ernst & Young LLP ("E&Y") to perform audit and other permissible non-audit services for the Company and its subsidiaries.',
 'Finally, at least annually, the Committee reviews the types of projects performed by Towers Perrin and the fees charged for the services rendered.',
 "In accordance with the Nominating and Governance Committee's charter, the Committee reviews information on compensation paid to non-employee directors at other public companies that is provided by John England, Managing Principal of Towers Perrin.",
 'The Committee has retained John England, Managing Principal of Towers Perrin, as its independent executive compensation consultant since 2002.',
 'Towers Perrin and Mr. England have also taken their own steps to separate the consultant from the other services provided by Towers Perrin.'}

# 10. F:\V F CORP

In [56]:
dir=r'F:\V F CORP'

In [57]:
os.chdir(dir)
os.getcwd()
proxies=find_proxies(dir)
data={}
pathList=[]
for i in proxies:
    filePath=os.path.join(dir, i)
    print(filePath)
    pathList.append(filePath) # create a list of file paths.

filename, text=open_proxy1(pathList)
data.update({'file_name':filename, 'text':text})

df=pd.DataFrame.from_dict(data)
# create columns with firm info
from tqdm.auto import tqdm
for i, row in tqdm(df.iterrows(), total=len(df)): # df.iterrows() will iterate each row
    filename = str(row["file_name"])  # row['column name'] to select certain column within a row
    print('filename is', filename)
    firm, year, cik, tic= extract_firm_info(filename)
    df.at[i, "firm"] = firm   # then, create a new column that contains the result of the operation.
    df.at[i, 'year'] = year
    df.at[i, 'cik'] = cik
    df.at[i, 'tic'] = tic

df['result']=df['text'].apply(NER2)
df['CC']=df['result'].apply(lambda x: x[0])
df['CC_sentence']=df['result'].apply(lambda x: x[1])
df['CC1']=df['CC'].apply(make_uppercase)
df['CC2']=df['CC1'].apply(remove_duplicates)

df.drop(columns=['result'])
column_order = ["file_name", "firm", "year", 'cik','tic','text','CC2', 'CC_sentence', 'CC', 'CC1']
df = df.reindex(columns=column_order)


F:\V F CORP\103379-V F CORP-DEF 14A-2007-03-22.html
F:\V F CORP\103379-V F CORP-DEF 14A-2008-03-17.html
F:\V F CORP\103379-V F CORP-DEF 14A-2009-03-20.html
F:\V F CORP\103379-V F CORP-DEF 14A-2010-03-19.html
F:\V F CORP\103379-V F CORP-DEF 14A-2011-03-23.html
F:\V F CORP\103379-V F CORP-DEF 14A-2012-03-21.html
F:\V F CORP\103379-V F CORP-DEF 14A-2013-03-20.html
F:\V F CORP\103379-V F CORP-DEF 14A-2014-03-21.html
F:\V F CORP\103379-V F CORP-DEF 14A-2015-03-19.html
F:\V F CORP\103379-V F CORP-DEF 14A-2016-03-24.html
100%|██████████| 10/10 [00:00<00:00, 2000.34it/s]
filename is F:\V F CORP\103379-V F CORP-DEF 14A-2007-03-22.html
filename is F:\V F CORP\103379-V F CORP-DEF 14A-2008-03-17.html
filename is F:\V F CORP\103379-V F CORP-DEF 14A-2009-03-20.html
filename is F:\V F CORP\103379-V F CORP-DEF 14A-2010-03-19.html
filename is F:\V F CORP\103379-V F CORP-DEF 14A-2011-03-23.html
filename is F:\V F CORP\103379-V F CORP-DEF 14A-2012-03-21.html
filename is F:\V F CORP\103379-V F CORP-DEF 14

In [58]:
df

,file_name,firm,year,cik,tic,text,CC2,CC_sentence,CC,CC1
0,F:\V F CORP\103379-V F CORP-DEF 14A-2007-03-22...,V F CORP,2007,103379,VFC,[ Mr. Fellows is President and Chief Exec...,{TOWERS PERRIN},{The Committee has retained Towers Perrin as i...,{TOWERS PERRIN},{TOWERS PERRIN}
1,F:\V F CORP\103379-V F CORP-DEF 14A-2008-03-17...,V F CORP,2008,103379,VFC,"[ Consultant , Ms. Feigin has been a ...",{TOWERS PERRIN},{The Committee has retained Towers Perrin ...,{TOWERS PERRIN},{TOWERS PERRIN}
2,F:\V F CORP\103379-V F CORP-DEF 14A-2009-03-20...,V F CORP,2009,103379,VFC,"[ Consultant , Ms. Feigin has been a ...","{FREDERIC, TOWERS PERRIN, COOK}","{Towers Perrin, while no longer the outsid...","{FREDERIC, COOK, TOWERS PERRIN, FREDERIC C...","{FREDERIC, COOK, TOWERS PERRIN, FREDERIC C..."
3,F:\V F CORP\103379-V F CORP-DEF 14A-2010-03-19...,V F CORP,2010,103379,VFC,[ Mr. Fellows has been President and Chief...,"{FREDERIC W., COOK}","{In setting the targets, the Committee, in...","{FREDERIC W. COOK & CO., INC., COOK & CO., INC...","{FREDERIC W. COOK & CO., INC., COOK & CO., INC..."
4,F:\V F CORP\103379-V F CORP-DEF 14A-2011-03-23...,V F CORP,2011,103379,VFC,[ Corporation. She is a member of the Exec...,"{FREDERIC W., TOWERS, COOK}",{Neither the Committee nor management receives...,"{FREDERIC W., TOWERS, COOK & CO., INC., FREDER...","{FREDERIC W., TOWERS, COOK & CO., INC., FREDER..."
5,F:\V F CORP\103379-V F CORP-DEF 14A-2012-03-21...,V F CORP,2012,103379,VFC,[Ms. Fairbairn has served as President and Chi...,"{TOWERS, FREDERIC COOK}",{The Committee has retained Frederic W. Cook &...,"{FREDERIC W. COOK & CO., INC., TOWERS WATSON, ...","{FREDERIC W. COOK & CO., INC., TOWERS WATSON, ..."
6,F:\V F CORP\103379-V F CORP-DEF 14A-2013-03-20...,V F CORP,2013,103379,VFC,[Mr. Fellows was President and Chief Executive...,"{TOWERS, FREDERIC COOK}",{The Committee has retained Frederic W. Cook &...,"{FREDERIC W. COOK & CO., INC., TOWERS WATSON, ...","{FREDERIC W. COOK & CO., INC., TOWERS WATSON, ..."
7,F:\V F CORP\103379-V F CORP-DEF 14A-2014-03-21...,V F CORP,2014,103379,VFC,[Ms. Fairbairn has served as President and Chi...,"{TOWERS, FREDERIC COOK}",{The Committee has retained Frederic W. Cook &...,"{FREDERIC W. COOK & CO., INC., TOWERS WATSON, ...","{FREDERIC W. COOK & CO., INC., TOWERS WATSON, ..."
8,F:\V F CORP\103379-V F CORP-DEF 14A-2015-03-19...,V F CORP,2015,103379,VFC,[The Committee has the authority to retain or ...,"{TOWERS, FREDERIC COOK}",{The Committee has retained Frederic W. Cook &...,"{FREDERIC W. COOK & CO., INC., TOWERS WATSON, ...","{FREDERIC W. COOK & CO., INC., TOWERS WATSON, ..."
9,F:\V F CORP\103379-V F CORP-DEF 14A-2016-03-24...,V F CORP,2016,103379,VFC,[The Committee has the authority to retain or ...,"{TOWERS, FREDERIC COOK}",{The Committee has retained Frederic W. Cook &...,"{FREDERIC W. COOK & CO., INC., TOWERS WATSON, ...","{FREDERIC W. COOK & CO., INC., TOWERS WATSON, ..."


# 11. F:\VISA INC

In [59]:
dir=r'F:\VISA INC'

In [60]:
os.chdir(dir)
os.getcwd()
proxies=find_proxies(dir)
data={}
pathList=[]
for i in proxies:
    filePath=os.path.join(dir, i)
    print(filePath)
    pathList.append(filePath) # create a list of file paths.

filename, text=open_proxy1(pathList)
data.update({'file_name':filename, 'text':text})

df=pd.DataFrame.from_dict(data)
# create columns with firm info
from tqdm.auto import tqdm
for i, row in tqdm(df.iterrows(), total=len(df)): # df.iterrows() will iterate each row
    filename = str(row["file_name"])  # row['column name'] to select certain column within a row
    print('filename is', filename)
    firm, year, cik, tic= extract_firm_info(filename)
    df.at[i, "firm"] = firm   # then, create a new column that contains the result of the operation.
    df.at[i, 'year'] = year
    df.at[i, 'cik'] = cik
    df.at[i, 'tic'] = tic

df['result']=df['text'].apply(NER2)
df['CC']=df['result'].apply(lambda x: x[0])
df['CC_sentence']=df['result'].apply(lambda x: x[1])
df['CC1']=df['CC'].apply(make_uppercase)
df['CC2']=df['CC1'].apply(remove_duplicates)

df.drop(columns=['result'])
column_order = ["file_name", "firm", "year", 'cik','tic','text','CC2', 'CC_sentence', 'CC', 'CC1']
df = df.reindex(columns=column_order)

F:\VISA INC\1403161-Visa Inc.-DEF 14A-2008-09-04.html
F:\VISA INC\1403161-VISA INC.-DEF 14A-2008-11-14.html
F:\VISA INC\1403161-VISA INC.-DEF 14A-2009-03-02.html
F:\VISA INC\1403161-VISA INC.-DEF 14A-2009-12-01.html
F:\VISA INC\1403161-VISA INC.-DEF 14A-2010-12-13.html
F:\VISA INC\1403161-VISA INC.-DEF 14A-2011-12-14.html
F:\VISA INC\1403161-VISA INC.-DEF 14A-2012-12-17.html
F:\VISA INC\1403161-VISA INC.-DEF 14A-2013-12-13.html
F:\VISA INC\1403161-VISA INC.-DEF 14A-2014-12-12.html
F:\VISA INC\1403161-VISA INC.-DEF 14A-2015-12-11.html
100%|██████████| 9/9 [00:00<00:00, 1810.93it/s]
filename is F:\VISA INC\1403161-Visa Inc.-DEF 14A-2008-09-04.html
filename is F:\VISA INC\1403161-VISA INC.-DEF 14A-2009-03-02.html
filename is F:\VISA INC\1403161-VISA INC.-DEF 14A-2009-12-01.html
filename is F:\VISA INC\1403161-VISA INC.-DEF 14A-2010-12-13.html
filename is F:\VISA INC\1403161-VISA INC.-DEF 14A-2011-12-14.html
filename is F:\VISA INC\1403161-VISA INC.-DEF 14A-2012-12-17.html
filename is F:\V

In [61]:
df

,file_name,firm,year,cik,tic,text,CC2,CC_sentence,CC,CC1
0,F:\VISA INC\1403161-Visa Inc.-DEF 14A-2008-09-...,VISA INC,2008,1403161,V,[Reference is made to restrictions on the issu...,{},{},{},{}
1,F:\VISA INC\1403161-VISA INC.-DEF 14A-2009-03-...,VISA INC,2009,1403161,V,[ has served as a director of Visa Inc. since ...,{},{},{},{}
2,F:\VISA INC\1403161-VISA INC.-DEF 14A-2009-12-...,VISA INC,2009,1403161,V,[has served as a director since October 2007. ...,"{FREDRICKCOOK, TOWERS PERRIN}","{For fiscal year 2009, the Compensation Commit...","{FREDRICK W. COOK & CO., TOWERS PERRIN}","{FREDRICK W. COOK & CO., TOWERS PERRIN}"
3,F:\VISA INC\1403161-VISA INC.-DEF 14A-2010-12-...,VISA INC,2010,1403161,V,[the Compensation Committee's engagement of an...,"{POLLITT, COOK}",{As a result of the review by the Committee an...,"{COOK & CO, POLLITT, COOK & CO., FREDERIC W. C...","{COOK & CO, POLLITT, COOK & CO., FREDERIC W. C..."
4,F:\VISA INC\1403161-VISA INC.-DEF 14A-2011-12-...,VISA INC,2011,1403161,V,[the Compensation Committee's engagement of an...,{COOK},{The Compensation Committee has engaged Freder...,"{COOK & CO, COOK & CO., FREDERIC W. COOK & CO.}","{COOK & CO, COOK & CO., FREDERIC W. COOK & CO.}"
5,F:\VISA INC\1403161-VISA INC.-DEF 14A-2012-12-...,VISA INC,2012,1403161,V,[The Compensation Committee engages its own in...,{COOK},"{In making this determination, the Compensatio...","{COOK & CO, COOK & CO., FREDERIC W. COOK & CO.}","{COOK & CO, COOK & CO., FREDERIC W. COOK & CO.}"
6,F:\VISA INC\1403161-VISA INC.-DEF 14A-2013-12-...,VISA INC,2013,1403161,V,[Consultant and Former CEO of the Corporate De...,{COOK},"{In approving the grants, the Compensation Com...","{COOK & CO, COOK & CO., FREDERIC W. COOK & CO.}","{COOK & CO, COOK & CO., FREDERIC W. COOK & CO.}"
7,F:\VISA INC\1403161-VISA INC.-DEF 14A-2014-12-...,VISA INC,2014,1403161,V,[Consultant and Former Chief Executive Officer...,{COOK},{The Compensation Committee has engaged Freder...,"{COOK & CO., FREDERIC W. COOK & CO.}","{COOK & CO., FREDERIC W. COOK & CO.}"
8,F:\VISA INC\1403161-VISA INC.-DEF 14A-2015-12-...,VISA INC,2015,1403161,V,[Consultant and Former Chief Executive Officer...,"{FREDERIC W. COOK &, COOK &}",{The Compensation Committee has engaged Cook &...,"{FREDERIC W. COOK &, COOK & CO., COOK &}","{FREDERIC W. COOK &, COOK & CO., COOK &}"


In [62]:
df['CC_sentence'].iloc[3]

{'As a result of the review by the Committee and Cook & Co., several new companies that met the established criteria were added, including Bank of New York Mellon, BB&T, Franklin Resources, Qualcomm, Texas Instruments and Yahoo!.',
 'During its annual review of the base salaries of our named executive officers for fiscal year 2010, the Compensation Committee, with the assistance of Cook & Co., considered:',
 'For all of the other NEOs, the Compensation Committee adopted the Executive Severance Plan and sent notices to each of Messrs. Partridge, Pollitt, Floum and Sheedy that their current employment agreements would not be renewed beyond the scheduled expiration dates in March 2011 or May 2011 in the case of Mr. Sheedy.',
 'However, after reviewing current market practices with Cook & Co., the Compensation Committee elected to increase the proportion of the annual equity award delivered as performance shares from 25% to 50% and to decrease the proportion delivered as stock options from

# 12. F:\TYCO INTERNATIONAL LTD

In [63]:
dir=r'F:\TYCO INTERNATIONAL LTD'

In [64]:
os.chdir(dir)
os.getcwd()
proxies=find_proxies(dir)
data={}
pathList=[]
for i in proxies:
    filePath=os.path.join(dir, i)
    print(filePath)
    pathList.append(filePath) # create a list of file paths.

filename, text=open_proxy1(pathList)
data.update({'file_name':filename, 'text':text})

df=pd.DataFrame.from_dict(data)
# create columns with firm info
from tqdm.auto import tqdm
for i, row in tqdm(df.iterrows(), total=len(df)): # df.iterrows() will iterate each row
    filename = str(row["file_name"])  # row['column name'] to select certain column within a row
    print('filename is', filename)
    firm, year, cik, tic= extract_firm_info(filename)
    df.at[i, "firm"] = firm   # then, create a new column that contains the result of the operation.
    df.at[i, 'year'] = year
    df.at[i, 'cik'] = cik
    df.at[i, 'tic'] = tic

df['result']=df['text'].apply(NER2)
df['CC']=df['result'].apply(lambda x: x[0])
df['CC_sentence']=df['result'].apply(lambda x: x[1])
df['CC1']=df['CC'].apply(make_uppercase)
df['CC2']=df['CC1'].apply(remove_duplicates)

df.drop(columns=['result'])
column_order = ["file_name", "firm", "year", 'cik','tic','text','CC2', 'CC_sentence', 'CC', 'CC1']
df = df.reindex(columns=column_order)

F:\TYCO INTERNATIONAL LTD\833444-TYCO INTERNATIONAL LTD-DEF 14A-2011-01-14.html
F:\TYCO INTERNATIONAL LTD\833444-TYCO INTERNATIONAL LTD-DEF 14A-2012-01-13.html
F:\TYCO INTERNATIONAL LTD\833444-TYCO INTERNATIONAL LTD-DEF 14A-2013-01-16.html
F:\TYCO INTERNATIONAL LTD\833444-TYCO INTERNATIONAL LTD-DEF 14A-2014-01-10.html
100%|██████████| 4/4 [00:00<00:00, 1000.19it/s]
filename is F:\TYCO INTERNATIONAL LTD\833444-TYCO INTERNATIONAL LTD-DEF 14A-2011-01-14.html
filename is F:\TYCO INTERNATIONAL LTD\833444-TYCO INTERNATIONAL LTD-DEF 14A-2012-01-13.html
filename is F:\TYCO INTERNATIONAL LTD\833444-TYCO INTERNATIONAL LTD-DEF 14A-2013-01-16.html
filename is F:\TYCO INTERNATIONAL LTD\833444-TYCO INTERNATIONAL LTD-DEF 14A-2014-01-10.html
Exequity
Exequity LLP
Exequity LLP
Exequity
Exequity LLP
Exequity
Exequity LLP
Exequity LLP
Exequity
Exequity LLP
Farient Advisors LLC
Farient
Farient
Farient Advisors LLC
Farient
Farient
Farient Advisors LLC
Farient
Farient
Farient Advisors LLC
Farient
Farient
Fa

In [65]:
df

,file_name,firm,year,cik,tic,text,CC2,CC_sentence,CC,CC1
0,F:\TYCO INTERNATIONAL LTD\833444-TYCO INTERNAT...,TYCO INTERNATIONAL LTD,2011,833444,JCI,"[Mr. Daniels, age 56, joined our Board in Mar...",{EXEQUITY},"{Since fiscal 2007, the Committee has retained...","{EXEQUITY, EXEQUITY LLP}","{EXEQUITY, EXEQUITY LLP}"
1,F:\TYCO INTERNATIONAL LTD\833444-TYCO INTERNAT...,TYCO INTERNATIONAL LTD,2012,833444,JCI,"[Role of Independent Compensation Consultant, ...",{EXEQUITY},"{Since fiscal 2007, the Committee has retained...","{EXEQUITY, EXEQUITY LLP}","{EXEQUITY, EXEQUITY LLP}"
2,F:\TYCO INTERNATIONAL LTD\833444-TYCO INTERNAT...,TYCO INTERNATIONAL LTD,2013,833444,JCI,[The share capital of the Company shall be inc...,{FARIENT},"{Going forward, the Committee expects to conti...","{FARIENT, FARIENT ADVISORS LLC}","{FARIENT, FARIENT ADVISORS LLC}"
3,F:\TYCO INTERNATIONAL LTD\833444-TYCO INTERNAT...,TYCO INTERNATIONAL LTD,2014,833444,JCI,"[As noted above, for our CEO, over 85% of annu...",{FARIENT},"{Going forward, the Compensation Committee exp...","{FARIENT, FARIENT ADVISORS LLC}","{FARIENT, FARIENT ADVISORS LLC}"


# 13. F:\UNITED HEALTHCARE CORP

In [66]:
dir=r'F:\UNITED HEALTHCARE CORP'


In [67]:
process()

No DEF 14A in the folder


0

In [194]:
# F:\UNIVERSAL HEALTH SERVICES INC

In [68]:
dir=r"F:\UNIVERSAL HEALTH SERVICES INC"

In [69]:
df=process()

F:\UNIVERSAL HEALTH SERVICES INC\352915-UNIVERSAL HEALTH SERVICES INC-DEF 14A-2007-04-20.html
F:\UNIVERSAL HEALTH SERVICES INC\352915-UNIVERSAL HEALTH SERVICES INC-DEF 14A-2008-04-16.html
F:\UNIVERSAL HEALTH SERVICES INC\352915-UNIVERSAL HEALTH SERVICES INC-DEF 14A-2009-04-08.html
F:\UNIVERSAL HEALTH SERVICES INC\352915-UNIVERSAL HEALTH SERVICES INC-DEF 14A-2010-04-09.html
F:\UNIVERSAL HEALTH SERVICES INC\352915-UNIVERSAL HEALTH SERVICES INC-DEF 14A-2011-04-01.html
F:\UNIVERSAL HEALTH SERVICES INC\352915-UNIVERSAL HEALTH SERVICES INC-DEF 14A-2012-04-05.html
F:\UNIVERSAL HEALTH SERVICES INC\352915-UNIVERSAL HEALTH SERVICES INC-DEF 14A-2013-04-05.html
F:\UNIVERSAL HEALTH SERVICES INC\352915-UNIVERSAL HEALTH SERVICES INC-DEF 14A-2014-04-09.html
F:\UNIVERSAL HEALTH SERVICES INC\352915-UNIVERSAL HEALTH SERVICES INC-DEF 14A-2015-04-09.html
F:\UNIVERSAL HEALTH SERVICES INC\352915-UNIVERSAL HEALTH SERVICES INC-DEF 14A-2016-04-07.html
100%|██████████| 10/10 [00:00<00:00, 2011.95it/s]
filename i

In [70]:
df

,file_name,firm,year,cik,tic,text,CC2,CC_sentence,CC,CC1
0,F:\UNIVERSAL HEALTH SERVICES INC\352915-UNIVER...,UNIVERSAL HEALTH SERVICES INC,2007,352915,UHID,[Consultant to the law firm of Eckert Seamans ...,{},{},{},{}
1,F:\UNIVERSAL HEALTH SERVICES INC\352915-UNIVER...,UNIVERSAL HEALTH SERVICES INC,2008,352915,UHID,[Consultant to the law firm of Eckert Seamans ...,{TOWERS PERRIN},"{above, Towers Perrin was engaged by the Compe...",{TOWERS PERRIN},{TOWERS PERRIN}
2,F:\UNIVERSAL HEALTH SERVICES INC\352915-UNIVER...,UNIVERSAL HEALTH SERVICES INC,2009,352915,UHID,[Awards under the Stock Incentive Plan may be ...,{TOWERS PERRIN},"{In early 2009, in connection with the annual ...",{TOWERS PERRIN},{TOWERS PERRIN}
3,F:\UNIVERSAL HEALTH SERVICES INC\352915-UNIVER...,UNIVERSAL HEALTH SERVICES INC,2010,352915,UHID,[The Restricted Stock Plan provides for the pu...,"{TOWERS WATSON & CO., TOWERS PERRIN}","{In late 2009, the Compensation Committee enga...","{TOWERS WATSON & CO., TOWERS PERRIN}","{TOWERS WATSON & CO., TOWERS PERRIN}"
4,F:\UNIVERSAL HEALTH SERVICES INC\352915-UNIVER...,UNIVERSAL HEALTH SERVICES INC,2011,352915,UHID,[. Awards may be granted to our and our aff...,{PAY GOVERNANCE LLC},"{In addition, the Compensation Committee revie...",{PAY GOVERNANCE LLC},{PAY GOVERNANCE LLC}
5,F:\UNIVERSAL HEALTH SERVICES INC\352915-UNIVER...,UNIVERSAL HEALTH SERVICES INC,2012,352915,UHID,[Awards under the Stock Incentive Plan may be ...,"{TOWERS WATSON & CO., PAY GOVERNANCE LLC}",{In determining this stock-based incentive awa...,"{TOWERS WATSON & CO., PAY GOVERNANCE, LLC, PAY...","{PAY GOVERNANCE, LLC, TOWERS WATSON & CO., PAY..."
6,F:\UNIVERSAL HEALTH SERVICES INC\352915-UNIVER...,UNIVERSAL HEALTH SERVICES INC,2013,352915,UHID,[Awards under the Stock Incentive Plan may be ...,"{TOWERS WATSON & CO., PAY GOVERNANCE LLC}","{In addition, the Compensation Committee also ...","{TOWERS WATSON & CO., PAY GOVERNANCE, LLC, PAY...","{PAY GOVERNANCE, LLC, TOWERS WATSON & CO., PAY..."
7,F:\UNIVERSAL HEALTH SERVICES INC\352915-UNIVER...,UNIVERSAL HEALTH SERVICES INC,2014,352915,UHID,[Awards under the Stock Incentive Plan may be ...,{PAY GOVERNANCE LLC},"{In addition, the Compensation Committee also ...","{PAY GOVERNANCE, LLC, PAY GOVERNANCE LLC}","{PAY GOVERNANCE, LLC, PAY GOVERNANCE LLC}"
8,F:\UNIVERSAL HEALTH SERVICES INC\352915-UNIVER...,UNIVERSAL HEALTH SERVICES INC,2015,352915,UHID,[. Awards may be granted to our and our sub...,{PAY GOVERNANCE LLC},"{In addition, the Compensation Committee also ...","{PAY GOVERNANCE, LLC, PAY GOVERNANCE LLC}","{PAY GOVERNANCE, LLC, PAY GOVERNANCE LLC}"
9,F:\UNIVERSAL HEALTH SERVICES INC\352915-UNIVER...,UNIVERSAL HEALTH SERVICES INC,2016,352915,UHID,[The Compensation Committee seeks guidance fro...,{PAY GOVERNANCE LLC},"{In addition, for Mr. Alan Miller and the Comp...",{PAY GOVERNANCE LLC},{PAY GOVERNANCE LLC}


# 14. F:\SUNTRUST BANKS INC

In [71]:
dir=r'F:\SUNTRUST BANKS INC'

In [72]:
df=process()
df

F:\SUNTRUST BANKS INC\750556-SUNTRUST BANKS INC-DEF 14A-2007-03-02.html
F:\SUNTRUST BANKS INC\750556-SUNTRUST BANKS INC-DEF 14A-2008-02-29.html
F:\SUNTRUST BANKS INC\750556-SUNTRUST BANKS INC-DEF 14A-2009-03-06.html
F:\SUNTRUST BANKS INC\750556-SUNTRUST BANKS INC-DEF 14A-2010-03-05.html
F:\SUNTRUST BANKS INC\750556-SUNTRUST BANKS INC-DEF 14A-2011-03-08.html
F:\SUNTRUST BANKS INC\750556-SUNTRUST BANKS INC-DEF 14A-2012-03-05.html
F:\SUNTRUST BANKS INC\750556-SUNTRUST BANKS INC-DEF 14A-2013-03-11.html
F:\SUNTRUST BANKS INC\750556-SUNTRUST BANKS INC-DEF 14A-2014-03-10.html
F:\SUNTRUST BANKS INC\750556-SUNTRUST BANKS INC-DEF 14A-2015-03-16.html
F:\SUNTRUST BANKS INC\750556-SUNTRUST BANKS INC-DEF 14A-2016-03-14.html
100%|██████████| 10/10 [00:00<00:00, 2499.44it/s]
filename is F:\SUNTRUST BANKS INC\750556-SUNTRUST BANKS INC-DEF 14A-2007-03-02.html
filename is F:\SUNTRUST BANKS INC\750556-SUNTRUST BANKS INC-DEF 14A-2008-02-29.html
filename is F:\SUNTRUST BANKS INC\750556-SUNTRUST BANKS INC-DE

,file_name,firm,year,cik,tic,text,CC2,CC_sentence,CC,CC1
0,F:\SUNTRUST BANKS INC\750556-SUNTRUST BANKS IN...,SUNTRUST BANKS INC,2007,750556,NaN,"[ HBC at the end of 2001, Mr. Isdell serve...",{TOWERS PERRIN},{To assist in its efforts to meet the objectiv...,{TOWERS PERRIN},{TOWERS PERRIN}
1,F:\SUNTRUST BANKS INC\750556-SUNTRUST BANKS IN...,SUNTRUST BANKS INC,2008,750556,NaN,"[On February 20, 2008, the Company entered int...",{TOWERS PERRIN},{To assist in its efforts to meet the objectiv...,{TOWERS PERRIN},{TOWERS PERRIN}
2,F:\SUNTRUST BANKS INC\750556-SUNTRUST BANKS IN...,SUNTRUST BANKS INC,2009,750556,NaN,[To assist in its efforts to meet the objectiv...,{TOWERS PERRIN},{To assist in its efforts to meet the objectiv...,{TOWERS PERRIN},{TOWERS PERRIN}
3,F:\SUNTRUST BANKS INC\750556-SUNTRUST BANKS IN...,SUNTRUST BANKS INC,2010,750556,NaN,[To assist in its efforts to meet the objectiv...,{TOWERS PERRIN},{To assist in its efforts to meet the objectiv...,{TOWERS PERRIN},{TOWERS PERRIN}
4,F:\SUNTRUST BANKS INC\750556-SUNTRUST BANKS IN...,SUNTRUST BANKS INC,2011,750556,NaN,"[Compensation Consultant. , To assist in its e...","{TOWERS WATSON, PAY GOVERNANCE LLC}",{To assist in its efforts to meet the objectiv...,"{TOWERS WATSON, PAY GOVERNANCE LLC}","{TOWERS WATSON, PAY GOVERNANCE LLC}"
5,F:\SUNTRUST BANKS INC\750556-SUNTRUST BANKS IN...,SUNTRUST BANKS INC,2012,750556,NaN,[Obtaining definitive peer information regardi...,{PAY GOVERNANCE LLC},"{In 2011, Pay Governance LLC performed service...",{PAY GOVERNANCE LLC},{PAY GOVERNANCE LLC}
6,F:\SUNTRUST BANKS INC\750556-SUNTRUST BANKS IN...,SUNTRUST BANKS INC,2013,750556,NaN,"[transformation, technology and process re-eng...",{PAY GOVERNANCE},"{Additionally, the Committee determined that P...","{PAY GOVERNANCE, PAY GOVERNANCE LLC}","{PAY GOVERNANCE, PAY GOVERNANCE LLC}"
7,F:\SUNTRUST BANKS INC\750556-SUNTRUST BANKS IN...,SUNTRUST BANKS INC,2014,750556,NaN,[financial services industry clients a...,{PAY GOVERNANCE LLC},"{Additionally, the Committee determined that P...",{PAY GOVERNANCE LLC},{PAY GOVERNANCE LLC}
8,F:\SUNTRUST BANKS INC\750556-SUNTRUST BANKS IN...,SUNTRUST BANKS INC,2015,750556,NaN,[Previously he was a Senior Consultant with El...,"{PAY GOVERNANCE LLC, COOK}","{In 2014, Pay Governance LLC and Frederic W. C...","{FREDERIC W. COOK & CO., INC., PAY GOVERNANCE ...","{FREDERIC W. COOK & CO., INC., PAY GOVERNANCE ..."
9,F:\SUNTRUST BANKS INC\750556-SUNTRUST BANKS IN...,SUNTRUST BANKS INC,2016,750556,NaN,[The Governance and Nominating Committee deter...,{COOK},"{In 2015, Frederic W. Cook & Co., Inc. perform...","{FREDERIC W. COOK & CO., INC.}","{FREDERIC W. COOK & CO., INC.}"


# 15. F:\TARGET CORP

In [73]:
dir=r'F:\TARGET CORP'
df=process()

F:\TARGET CORP\27419-TARGET CORP-DEF 14A-2007-04-09.html
F:\TARGET CORP\27419-TARGET CORP-DEF 14A-2008-04-07.html
F:\TARGET CORP\27419-TARGET CORP-DEF 14A-2010-04-29.html
F:\TARGET CORP\27419-TARGET CORP-DEF 14A-2011-04-28.html
F:\TARGET CORP\27419-TARGET CORP-DEF 14A-2012-04-30.html
F:\TARGET CORP\27419-TARGET CORP-DEF 14A-2013-04-29.html
F:\TARGET CORP\27419-TARGET CORP-DEF 14A-2014-05-19.html
F:\TARGET CORP\27419-TARGET CORP-DEF 14A-2015-04-27.html
F:\TARGET CORP\27419-TARGET CORP-DEF 14A-2016-04-25.html
100%|██████████| 9/9 [00:00<00:00, 2269.65it/s]
filename is F:\TARGET CORP\27419-TARGET CORP-DEF 14A-2007-04-09.html
filename is F:\TARGET CORP\27419-TARGET CORP-DEF 14A-2008-04-07.html
filename is F:\TARGET CORP\27419-TARGET CORP-DEF 14A-2010-04-29.html
filename is F:\TARGET CORP\27419-TARGET CORP-DEF 14A-2011-04-28.html
filename is F:\TARGET CORP\27419-TARGET CORP-DEF 14A-2012-04-30.html
filename is F:\TARGET CORP\27419-TARGET CORP-DEF 14A-2013-04-29.html
filename is F:\TARGET COR

In [74]:
df

,file_name,firm,year,cik,tic,text,CC2,CC_sentence,CC,CC1
0,F:\TARGET CORP\27419-TARGET CORP-DEF 14A-2007-...,TARGET CORP,2007,27419,TGT,"[management team, with input from the Committe...","{HEWITT, SEMLER BROSSY}",{Our Compensation Committee engages an indepen...,"{HEWITT ASSOCIATES, LLC, HEWITT, SEMLER BROSSY...","{HEWITT ASSOCIATES, LLC, HEWITT, SEMLER BROSSY..."
1,F:\TARGET CORP\27419-TARGET CORP-DEF 14A-2008-...,TARGET CORP,2008,27419,TGT,[ The Compensation Committee has respon...,"{HEWITT, SEMLER BROSSY CONSULTING GROUP, LLC}","{In addition to the advice of SBCG, the Compen...","{HEWITT ASSOCIATES, LLC, HEWITT, SEMLER BROSSY...","{HEWITT ASSOCIATES, LLC, HEWITT, SEMLER BROSSY..."
2,F:\TARGET CORP\27419-TARGET CORP-DEF 14A-2010-...,TARGET CORP,2010,27419,TGT,[ The Compensation Committee may not de...,"{HEWITT, SEMLER BROSSY CONSULTING GROUP, LLC}","{These enhancements, along with our existing c...","{HEWITT, SEMLER BROSSY CONSULTING GROUP, LLC}","{HEWITT, SEMLER BROSSY CONSULTING GROUP, LLC}"
3,F:\TARGET CORP\27419-TARGET CORP-DEF 14A-2011-...,TARGET CORP,2011,27419,TGT,[ As part of our regular review of our ...,"{SEMLER BROSSY CONSULTING GROUP, LLC}","{In broad terms, our compensation programs for...","{SEMLER BROSSY CONSULTING GROUP, LLC}","{SEMLER BROSSY CONSULTING GROUP, LLC}"
4,F:\TARGET CORP\27419-TARGET CORP-DEF 14A-2012-...,TARGET CORP,2012,27419,TGT,[ As part of our regular review of our ...,{SEMLER BROSSY CONSULTING GROUP},{Conducting an annual compensation risk assess...,{SEMLER BROSSY CONSULTING GROUP},{SEMLER BROSSY CONSULTING GROUP}
5,F:\TARGET CORP\27419-TARGET CORP-DEF 14A-2013-...,TARGET CORP,2013,27419,TGT,[The Compensation Committee has responsibility...,{SEMLER BROSSY CONSULTING GROUP},"{An independent compensation consultant, Semle...",{SEMLER BROSSY CONSULTING GROUP},{SEMLER BROSSY CONSULTING GROUP}
6,F:\TARGET CORP\27419-TARGET CORP-DEF 14A-2014-...,TARGET CORP,2014,27419,TGT,[Discusses with its compensation consultant th...,{SEMLER BROSSY CONSULTING GROUP},{The Compensation Committee receives assistanc...,{SEMLER BROSSY CONSULTING GROUP},{SEMLER BROSSY CONSULTING GROUP}
7,F:\TARGET CORP\27419-TARGET CORP-DEF 14A-2015-...,TARGET CORP,2015,27419,TGT,[Given the evolving environment around risk ov...,{SEMLER BROSSY CONSULTING GROUP},{The Compensation Committee receives assistanc...,{SEMLER BROSSY CONSULTING GROUP},{SEMLER BROSSY CONSULTING GROUP}
8,F:\TARGET CORP\27419-TARGET CORP-DEF 14A-2016-...,TARGET CORP,2016,27419,TGT,[Given the evolving environment around risk ov...,{SEMLER BROSSY CONSULTING GROUP},{The Human Resources and Compensation Committe...,{SEMLER BROSSY CONSULTING GROUP},{SEMLER BROSSY CONSULTING GROUP}


In [75]:
df['CC_sentence'].iloc[0]

{'Based on the data provided by Hewitt, management formulates and presents the Compensation Committee with recommendations for the compensation of executive officers (other than the CEO) and other compensation plan features pursuant to guidance from the Committee.',
 'In addition to the advice of Semler Brossy, the Committee uses comprehensive data prepared by a separate compensation consulting firm, Hewitt Associates, LLC, covering a representative group of retail and general industry companies throughout the United States.',
 'Our Compensation Committee engages an independent consultant, Semler Brossy Consulting Group, LLC, to assist in the review and planning of executive compensation.',
 'Semler Brossy reports directly to the Compensation Committee and works collaboratively as directed by the Committee with management and the Chairman of the Compensation Committee.',
 'The composition of the survey group is reviewed annually and any changes made are reviewed with Semler Brossy and 

# 16. F:\SUNOCO INC

In [76]:
dir=r'F:\SUNOCO INC'
df=process()

F:\SUNOCO INC\95304-SUNOCO INC-DEF 14A-2007-03-09.html
F:\SUNOCO INC\95304-SUNOCO INC-DEF 14A-2008-03-17.html
F:\SUNOCO INC\95304-SUNOCO INC-DEF 14A-2009-03-17.html
F:\SUNOCO INC\95304-SUNOCO INC-DEF 14A-2010-03-17.html
F:\SUNOCO INC\95304-SUNOCO INC-DEF 14A-2011-03-18.html
F:\SUNOCO INC\95304-SUNOCO INC-DEF 14A-2012-03-16.html
100%|██████████| 6/6 [00:00<00:00, 1499.66it/s]
filename is F:\SUNOCO INC\95304-SUNOCO INC-DEF 14A-2007-03-09.html
filename is F:\SUNOCO INC\95304-SUNOCO INC-DEF 14A-2008-03-17.html
filename is F:\SUNOCO INC\95304-SUNOCO INC-DEF 14A-2009-03-17.html
filename is F:\SUNOCO INC\95304-SUNOCO INC-DEF 14A-2010-03-17.html
filename is F:\SUNOCO INC\95304-SUNOCO INC-DEF 14A-2011-03-18.html
filename is F:\SUNOCO INC\95304-SUNOCO INC-DEF 14A-2012-03-16.html
Towers Perrin
Towers Perrin
Pearl Meyer & Partners
Towers Perrin
Towers Perrin
Pearl Meyer & Partners
Towers Perrin
Towers Perrin
Pearl Meyer & Partners
Towers Perrin
Towers Perrin
Towers Perrin
Semler Brossy Consulting 

In [77]:
df

,file_name,firm,year,cik,tic,text,CC2,CC_sentence,CC,CC1
0,F:\SUNOCO INC\95304-SUNOCO INC-DEF 14A-2007-03...,SUNOCO INC,2007,95304,NaN,[All of Sunoco's directors are elected each ye...,"{PEARL MEYER & PARTNERS, TOWERS PERRIN}","{The compensation consultant, Pearl Meyer & Pa...","{PEARL MEYER & PARTNERS, TOWERS PERRIN}","{PEARL MEYER & PARTNERS, TOWERS PERRIN}"
1,F:\SUNOCO INC\95304-SUNOCO INC-DEF 14A-2008-03...,SUNOCO INC,2008,95304,NaN,[All of Sunoco's directors are elected each ye...,"{TOWERS PERRIN, SEMLER BROSSY}",{Semler Brossy has responsibility for working ...,"{SEMLER BROSSY CONSULTING GROUP, TOWERS PERRIN...","{SEMLER BROSSY CONSULTING GROUP, TOWERS PERRIN..."
2,F:\SUNOCO INC\95304-SUNOCO INC-DEF 14A-2009-03...,SUNOCO INC,2009,95304,NaN,[All of Sunoco's directors are elected each ye...,"{TOWERS PERRIN, SEMLER BROSSY}",{Semler Brossy has responsibility for working ...,"{TOWERS PERRIN, SEMLER BROSSY CONSULTING GROUP...","{TOWERS PERRIN, SEMLER BROSSY CONSULTING GROUP..."
3,F:\SUNOCO INC\95304-SUNOCO INC-DEF 14A-2010-03...,SUNOCO INC,2010,95304,NaN,[The Board annually recommends the slate of di...,"{TOWERS WATSON, SEMLER BROSSY, COMPENSATION AD...",{compensation is determined based on targets d...,"{COMPENSATION ADVISORY PARTNERS, TOWERS WATSON...","{TOWERS WATSON, SEMLER BROSSY CONSULTING GROUP..."
4,F:\SUNOCO INC\95304-SUNOCO INC-DEF 14A-2011-03...,SUNOCO INC,2011,95304,NaN,[The Board annually recommends the slate of di...,"{TOWERS WATSON, SEMLER BROSSY, COMPENSATION AD...","{Towers Watson, a compensation consultant, pro...","{COMPENSATION ADVISORY PARTNERS, TOWERS WATSON...","{TOWERS WATSON, SEMLER BROSSY CONSULTING GROUP..."
5,F:\SUNOCO INC\95304-SUNOCO INC-DEF 14A-2012-03...,SUNOCO INC,2012,95304,NaN,[The Board annually recommends the slate of di...,"{TOWERS WATSON, SEMLER BROSSY, COMPENSATION AD...","{However, the Committee, with the assistance a...","{COMPENSATION ADVISORY PARTNERS, TOWERS WATSON...","{TOWERS WATSON, SEMLER BROSSY CONSULTING GROUP..."


In [78]:
df['CC_sentence'].iloc[0]

{'The Committee reviews the effectiveness and competitiveness of the executive compensation structure in the context of the foregoing philosophy and objectives, with the assistance of a compensation consultant, Towers Perrin, who is directly engaged by the Committee.',
 "The compensation consultant, Pearl Meyer & Partners, also benchmarks Sunoco's director compensation compared to the Sunoco's peer group, the oil industry generally and general industry data.",
 'Towers Perrin provides the Committee with relevant Market Data (defined below) and alternatives for the Committee to consider when making compensation decisions.'}

In [227]:
df['CC_sentence'].iloc[5]

{'A representative from Semler Brossy attends all regularly scheduled Committee meetings.',
 'However, the Committee, with the assistance and input of Semler Brossy, ultimately makes all final decisions with regard to setting goals or performance criteria.',
 'In consultation with Semler Brossy and management, the Committee has annually reviewed the estimated compensation that would be received by the CEO and the other NEOs under various scenarios, including normal retirement and voluntary termination, involuntary termination, and termination due to a change in control of Sunoco.',
 "Semler Brossy has responsibility for working with the Committee, and does not perform any work for management except at the request of the Committee and/or the Committee Chair as the Committee's representative, nor does Semler Brossy receive any compensation from management.",
 "Semler Brossy provides a separate, independent review of Compensation Advisory Partners' and Towers Watson's information for the 

# 17. F:\STAPLES INC

In [79]:
dir=r'F:\STAPLES INC'

In [80]:
df=process()

F:\STAPLES INC\791519-STAPLES INC-DEF 14A-2007-05-03.html
F:\STAPLES INC\791519-STAPLES INC-DEF 14A-2008-04-28.html
F:\STAPLES INC\791519-STAPLES INC-DEF 14A-2009-04-27.html
F:\STAPLES INC\791519-STAPLES INC-DEF 14A-2010-04-26.html
F:\STAPLES INC\791519-STAPLES INC-DEF 14A-2011-04-25.html
F:\STAPLES INC\791519-STAPLES INC-DEF 14A-2012-04-23.html
F:\STAPLES INC\791519-STAPLES INC-DEF 14A-2013-04-12.html
F:\STAPLES INC\791519-STAPLES INC-DEF 14A-2014-04-11.html
F:\STAPLES INC\791519-STAPLES INC-DEF 14A-2015-04-13.html
F:\STAPLES INC\791519-STAPLES INC-DEF 14A-2016-04-26.html
100%|██████████| 10/10 [00:00<00:00, 2500.48it/s]
filename is F:\STAPLES INC\791519-STAPLES INC-DEF 14A-2007-05-03.html
filename is F:\STAPLES INC\791519-STAPLES INC-DEF 14A-2008-04-28.html
filename is F:\STAPLES INC\791519-STAPLES INC-DEF 14A-2009-04-27.html
filename is F:\STAPLES INC\791519-STAPLES INC-DEF 14A-2010-04-26.html
filename is F:\STAPLES INC\791519-STAPLES INC-DEF 14A-2011-04-25.html
filename is F:\STAPL

In [81]:
df

,file_name,firm,year,cik,tic,text,CC2,CC_sentence,CC,CC1
0,F:\STAPLES INC\791519-STAPLES INC-DEF 14A-2007...,STAPLES INC,2007,791519,NaN,"[Independent Compensation Consultant, All memb...",{HEWITT ASSOCIATES},{Hewitt Associates performed services under a ...,"{HEWITT ASSOCIATES, HEWITT ASSOCIATES LLC}","{HEWITT ASSOCIATES, HEWITT ASSOCIATES LLC}"
1,F:\STAPLES INC\791519-STAPLES INC-DEF 14A-2008...,STAPLES INC,2008,791519,NaN,"[attract, motivate and retain key employees an...","{EXEQUITY, HEWITT ASSOCIATES LLC}",{The principal consultant from Exequity attend...,"{EXEQUITY, HEWITT ASSOCIATES LLC, EXEQUITY LLP}","{EXEQUITY, HEWITT ASSOCIATES LLC, EXEQUITY LLP}"
2,F:\STAPLES INC\791519-STAPLES INC-DEF 14A-2009...,STAPLES INC,2009,791519,NaN,[Independent Compensation Committee Consultant...,"{HAY GROUP, EXEQUITY, HEWITT, WATSON WYATT}",{The principal consultant from Exequity attend...,"{HAY GROUP, WATSON WYATT, EXEQUITY LLP, HEWITT...","{HAY GROUP, WATSON WYATT, EXEQUITY LLP, HEWITT..."
3,F:\STAPLES INC\791519-STAPLES INC-DEF 14A-2010...,STAPLES INC,2010,791519,NaN,"[attract, motivate and retain key employees an...","{HEWITT ASSOCIATES, HAYGROUP, MERCER, PEARL ME...",{The principal consultant from Exequity attend...,"{HEWITT ASSOCIATES, HAYGROUP, MERCER, PEARL ME...","{HEWITT ASSOCIATES, HAYGROUP, MERCER, PEARL ME..."
4,F:\STAPLES INC\791519-STAPLES INC-DEF 14A-2011...,STAPLES INC,2011,791519,NaN,[Independent Compensation Committee Consultant...,{EXEQUITY},"{In March 2010, the Committee set compensation...","{EXEQUITY, EXEQUITY LLP}","{EXEQUITY, EXEQUITY LLP}"
5,F:\STAPLES INC\791519-STAPLES INC-DEF 14A-2012...,STAPLES INC,2012,791519,NaN,"[Independent Compensation Consultant, Independ...",{EXEQUITY},"{In March 2011, the Committee set compensation...","{EXEQUITY, EXEQUITY LLP}","{EXEQUITY, EXEQUITY LLP}"
6,F:\STAPLES INC\791519-STAPLES INC-DEF 14A-2013...,STAPLES INC,2013,791519,NaN,[. As required by our Corporate Governance Gu...,{EXEQUITY},"{In March 2013, the Committee performed a conf...","{EXEQUITY, EXEQUITY LLP}","{EXEQUITY, EXEQUITY LLP}"
7,F:\STAPLES INC\791519-STAPLES INC-DEF 14A-2014...,STAPLES INC,2014,791519,NaN,[. As required by our Corporate Governance Gu...,{EXEQUITY},"{In our 2013 fiscal year, the Committee contin...","{EXEQUITY, EXEQUITY LLP}","{EXEQUITY, EXEQUITY LLP}"
8,F:\STAPLES INC\791519-STAPLES INC-DEF 14A-2015...,STAPLES INC,2015,791519,NaN,[Assists in the engagement of investment...,{EXEQUITY},{The principal consultant from Exequity attend...,"{EXEQUITY, EXEQUITY LLP}","{EXEQUITY, EXEQUITY LLP}"
9,F:\STAPLES INC\791519-STAPLES INC-DEF 14A-2016...,STAPLES INC,2016,791519,NaN,[Assists in the engagement of investment...,"{EXEQUITY, WILLIS TOWERS WATSON}","{During 2015, with the Committee's approval, E...","{EXEQUITY, EXEQUITY LLP, WILLIS TOWERS WATSON}","{EXEQUITY, EXEQUITY LLP, WILLIS TOWERS WATSON}"


In [82]:
df['CC_sentence'].iloc[1]

{'Accordingly, in our 2007 fiscal year, the Compensation Committee retained, pursuant to a written agreement, Exequity LLP as an independent advisor reporting to the Compensation Committee to advise on and assist with executive compensation matters.',
 'Consistent with the terms of the written agreement, Exequity has, with the knowledge and consent of the Compensation Committee, provided advice and expertise to management on matters to be presented by management to the Compensation Committee, and Exequity has not performed services for Staples that were unrelated to Compensation Committee-related executive compensation matters.',
 'From 2002 through 2006, the Compensation Committee had engaged Hewitt Associates LLC.',
 "In 2007, this analysis included data from peer company proxy filings, data from Equilar, Inc., an independent provider of executive and board compensation analyses, and published outside director pay studies, including a study conducted by Exequity, the Compensation Com

In [239]:
df['CC_sentence'].iloc[9]

{'Consistent with the terms of the written agreement and the Committee charter, Exequity has, with the knowledge and consent of the Committee, provided input to management on matters to be presented by management to the Committee.',
 "During 2015, with the Committee's approval, Exequity assisted management by performing Section 280G calculations and providing compensation data related to executive and non-executive positions.",
 'Exequity has not performed services for Staples that were unrelated to Committee matters.',
 "In 2015, management engaged its own compensation consultant, Willis Towers Watson, separately from the Committee's compensation consultant, to help develop the CEO's recommendations to the Committee.",
 'In March 2015, the Committee set compensation for the NEOs based on its December 2014 review of 2011-2013 compensation, its assessment of our 2014 performance, stockholder feedback and results of 2014 Say-on-Pay advisory vote, and general consideration of the totality

# 18. F:\SMUCKER J M CO

In [83]:
dir=r'F:\SMUCKER J M CO'

In [84]:
df=process()

F:\SMUCKER J M CO\91419-SMUCKER J M CO-DEF 14A-2007-07-09.html
F:\SMUCKER J M CO\91419-SMUCKER J M CO-DEF 14A-2008-07-14.html
F:\SMUCKER J M CO\91419-SMUCKER J M CO-DEF 14A-2009-07-09.html
F:\SMUCKER J M CO\91419-SMUCKER J M CO-DEF 14A-2010-07-08.html
F:\SMUCKER J M CO\91419-SMUCKER J M CO-DEF 14A-2011-07-07.html
F:\SMUCKER J M CO\91419-SMUCKER J M CO-DEF 14A-2012-07-03.html
F:\SMUCKER J M CO\91419-SMUCKER J M CO-DEF 14A-2013-07-02.html
F:\SMUCKER J M CO\91419-SMUCKER J M CO-DEF 14A-2014-07-02.html
100%|██████████| 8/8 [00:00<00:00, 1600.19it/s]
filename is F:\SMUCKER J M CO\91419-SMUCKER J M CO-DEF 14A-2007-07-09.html
filename is F:\SMUCKER J M CO\91419-SMUCKER J M CO-DEF 14A-2008-07-14.html
filename is F:\SMUCKER J M CO\91419-SMUCKER J M CO-DEF 14A-2009-07-09.html
filename is F:\SMUCKER J M CO\91419-SMUCKER J M CO-DEF 14A-2010-07-08.html
filename is F:\SMUCKER J M CO\91419-SMUCKER J M CO-DEF 14A-2011-07-07.html
filename is F:\SMUCKER J M CO\91419-SMUCKER J M CO-DEF 14A-2012-07-03.htm

In [85]:
df

,file_name,firm,year,cik,tic,text,CC2,CC_sentence,CC,CC1
0,F:\SMUCKER J M CO\91419-SMUCKER J M CO-DEF 14A...,SMUCKER J M CO,2007,91419,SJM,[ Directors who are employees of Smucker r...,{TOWERS PERRIN},{Informed of competitive industry salaries and...,{TOWERS PERRIN},{TOWERS PERRIN}
1,F:\SMUCKER J M CO\91419-SMUCKER J M CO-DEF 14A...,SMUCKER J M CO,2008,91419,SJM,[ Directors who are employees of the Compa...,"{MERCER, TOWERS PERRIN, WATSON WYATT}","{Pursuant to its governance model, the Exe...","{TOWERS PERRIN, MERCER, TOWERS PERRIN, WAT...","{TOWERS PERRIN, MERCER, TOWERS PERRIN, WAT..."
2,F:\SMUCKER J M CO\91419-SMUCKER J M CO-DEF 14A...,SMUCKER J M CO,2009,91419,SJM,[coffee transaction and a review of director c...,"{MERCER, TOWERS PERRIN, WATSON WYATT}",{Pursuant to its governance corporate mode...,"{TOWERS PERRIN, MERCER, TOWERS PERRIN, WAT...","{TOWERS PERRIN, MERCER, TOWERS PERRIN, WAT..."
3,F:\SMUCKER J M CO\91419-SMUCKER J M CO-DEF 14A...,SMUCKER J M CO,2010,91419,SJM,[ Directors who are employees of the Compa...,"{TOWERS WATSON, MERCER, WATSON}",{Watson reports directly to the Compensation C...,"{TOWERS WATSON, MERCER, TOWERS WATSON & CO., W...","{TOWERS WATSON, TOWERS WATSON & CO., MERCER, W..."
4,F:\SMUCKER J M CO\91419-SMUCKER J M CO-DEF 14A...,SMUCKER J M CO,2011,91419,SJM,[ Directors who are employees of the Compa...,"{WATSON WYATT, COOK, FREDERIC W., MERCER, ...","{During fiscal year 2011, Towers Watson attend...","{WATSON WYATT, FREDERIC W., TOWERS WATSON,...","{WATSON WYATT, FREDERIC W., TOWERS WATSON,..."
5,F:\SMUCKER J M CO\91419-SMUCKER J M CO-DEF 14A...,SMUCKER J M CO,2012,91419,SJM,[Directors who are employees of the Company re...,"{HEWITT, FWC, TOWERS, COOK}",{This increase in compensation paid to non-emp...,"{FREDERIC W. COOK & CO., INC., HEWITT, FWC, TO...","{FREDERIC W. COOK & CO., INC., HEWITT, FWC, TO..."
6,F:\SMUCKER J M CO\91419-SMUCKER J M CO-DEF 14A...,SMUCKER J M CO,2013,91419,SJM,[We use a combination of cash and stock-based ...,{FREDERIC COOK &},{The Compensation Committee has retained Frede...,"{FREDERIC COOK & CO., FREDERIC W. COOK & CO.,,...","{FREDERIC COOK & CO., FREDERIC W. COOK & CO.,,..."
7,F:\SMUCKER J M CO\91419-SMUCKER J M CO-DEF 14A...,SMUCKER J M CO,2014,91419,SJM,[We use a combination of cash and stock-based ...,{FREDERIC COOK &},{The Compensation Committee has retained Frede...,"{FREDERIC COOK & CO., FREDERIC W. COOK & CO.,,...","{FREDERIC COOK & CO., FREDERIC W. COOK & CO.,,..."


In [87]:
df['CC_sentence'].iloc[4]

{"As Towers Watson did, FWC also reports     directly to the Compensation Committee and participates in     executive sessions with the Compensation Committee, without     members of the Company's management present.",
 "At that time, the Compensation Committee     retained FWC to serve as the Compensation Committee's     outside consultant.",
 'During fiscal year 2011, Towers Watson attended     the June 2010 meeting and FWC attended all Compensation     Committee meetings beginning with the October 2010 meeting.',
 "Further, the Compensation Committee is kept apprised     of other work performed by Towers Watson on behalf of the     Company and also has considered the compatibility of     non-compensation services with Towers Watson's     independence.",
 'Inc. ("FWC"), an independent     outside compensation consultant.',
 'Management makes no recommendation     regarding long-term incentive awards for the Co-CEOs, but the     Compensation Committee, after considering input from FWC

# 19. F:\Seagate Technology

In [88]:
dir=r'F:\Seagate Technology'
df=process()

F:\Seagate Technology\1137789-SEAGATE TECHNOLOGY-DEF 14A-2007-09-21.html
F:\Seagate Technology\1137789-SEAGATE TECHNOLOGY-DEF 14A-2008-09-19.html
F:\Seagate Technology\1137789-Seagate Technology-DEF 14A-2009-09-18.html
F:\Seagate Technology\1137789-Seagate Technology-DEF 14A-2010-03-05.html
100%|██████████| 4/4 [00:00<00:00, 1000.07it/s]
filename is F:\Seagate Technology\1137789-SEAGATE TECHNOLOGY-DEF 14A-2007-09-21.html
filename is F:\Seagate Technology\1137789-SEAGATE TECHNOLOGY-DEF 14A-2008-09-19.html
filename is F:\Seagate Technology\1137789-Seagate Technology-DEF 14A-2009-09-18.html
filename is F:\Seagate Technology\1137789-Seagate Technology-DEF 14A-2010-03-05.html
Frederick W. Cook & Co.
F.W. Cook
F.W. Cook
F.W. Cook
James
Frederick W. Cook & Co.
F.W. Cook
F.W. Cook
F.W. Cook
F.W. Cook
Hewitt Associates, Inc.
Frederick W. Cook
F.W. Cook
F.W. Cook
Frederick W. Cook
F.W. Cook
F.W. Cook
F.W. Cook
F.W. Cook
Frederick W. Cook
F.W. Cook
F.W. Cook
F.W. Cook
F.W. Cook
Hewitt Associates,

In [89]:
df

,file_name,firm,year,cik,tic,text,CC2,CC_sentence,CC,CC1
0,F:\Seagate Technology\1137789-SEAGATE TECHNOLO...,Seagate Technology,2007,1137789,STX,[Mr. Reyes became a member of our Board in...,{},{},{},{}
1,F:\Seagate Technology\1137789-SEAGATE TECHNOLO...,Seagate Technology,2008,1137789,STX,[ became a member of our Board in April 2004. ...,"{JAMES, FREDERICKCOOK, HEWITT ASSOCIATES, INC....",{F.W. Cook does not provide services to the Co...,"{F.W. COOK, JAMES, HEWITT ASSOCIATES, INC., FR...","{F.W. COOK, JAMES, FREDERICK W. COOK & CO., HE..."
2,F:\Seagate Technology\1137789-Seagate Technolo...,Seagate Technology,2009,1137789,STX,[Mr. Reyes has been a private investor and man...,"{FREDERICK W. COOK, HEWITT ASSOCIATES, INC., C...",{F.W. Cook did not provide any services to Sea...,"{FREDERICK W. COOK, F.W. COOK, HEWITT ASSOCIAT...","{FREDERICK W. COOK, F.W. COOK, HEWITT ASSOCIAT..."
3,F:\Seagate Technology\1137789-Seagate Technolo...,Seagate Technology,2010,1137789,STX,[To establish and support or aid in the establ...,{},{},{},{}


In [90]:
df['CC_sentence'].iloc[1]

{'During fiscal year 2008 the Compensation Committee utilized the services of Frederick W. Cook & Co. ("F.W. Cook") and approved the following ongoing role for the independent consultant:',
 'Executive management also employed Hewitt Associates, Inc. throughout the year to advise them and assist in developing proposals to the Compensation Committee including but not limited to Performance Share Awards, Stock Ownership guidelines, and severance/change in control policies.',
 'F.W. Cook does not provide services to the Company management except as directed by the Compensation Committee.',
 'F.W. Cook provided the Compensation Committee with the following reports and supporting consultation during fiscal year 2008:',
 'Seagate paid no fees or other compensation to F.W. Cook for any services other than those in support of the Compensation Committee.',
 'The members of our Compensation Committee during fiscal year 2008 were Messrs. Thompson, Reyes, Marquardt and James A. Davidson.'}

In [91]:
df['CC2'].iloc[2]

{'COOK', 'FREDERICK W. COOK', 'HEWITT ASSOCIATES, INC.'}

# 19. F:\SEAGATE TECHNOLOGY INC

In [257]:
dir=r'F:\SEAGATE TECHNOLOGY INC'
df=process()

No DEF 14A in the folder


# 20. F:\PRUDENTIAL FINANCIAL INC

In [93]:
dir=r'F:\PRUDENTIAL FINANCIAL INC'
df=process()

F:\PRUDENTIAL FINANCIAL INC\1137774-PRUDENTIAL FINANCIAL INC-DEF 14A-2007-03-22.html
F:\PRUDENTIAL FINANCIAL INC\1137774-PRUDENTIAL FINANCIAL INC-DEF 14A-2008-03-21.html
F:\PRUDENTIAL FINANCIAL INC\1137774-PRUDENTIAL FINANCIAL INC-DEF 14A-2009-03-20.html
F:\PRUDENTIAL FINANCIAL INC\1137774-PRUDENTIAL FINANCIAL INC-DEF 14A-2010-03-22.html
F:\PRUDENTIAL FINANCIAL INC\1137774-PRUDENTIAL FINANCIAL INC-DEF 14A-2011-03-22.html
F:\PRUDENTIAL FINANCIAL INC\1137774-PRUDENTIAL FINANCIAL INC-DEF 14A-2012-03-27.html
F:\PRUDENTIAL FINANCIAL INC\1137774-PRUDENTIAL FINANCIAL INC-DEF 14A-2013-03-26.html
F:\PRUDENTIAL FINANCIAL INC\1137774-PRUDENTIAL FINANCIAL INC-DEF 14A-2014-03-25.html
F:\PRUDENTIAL FINANCIAL INC\1137774-PRUDENTIAL FINANCIAL INC-DEF 14A-2015-03-24.html
F:\PRUDENTIAL FINANCIAL INC\1137774-PRUDENTIAL FINANCIAL INC-DEF 14A-2016-03-22.html
100%|██████████| 10/10 [00:00<00:00, 2501.08it/s]
filename is F:\PRUDENTIAL FINANCIAL INC\1137774-PRUDENTIAL FINANCIAL INC-DEF 14A-2007-03-22.html
fil

In [94]:
df

,file_name,firm,year,cik,tic,text,CC2,CC_sentence,CC,CC1
0,F:\PRUDENTIAL FINANCIAL INC\1137774-PRUDENTIAL...,PRUDENTIAL FINANCIAL INC,2007,1137774,PFH,[The Corporate Governance and Business Ethics ...,"{FREDERIC W. COOK, & CO., ROE}","{Each year, the Compensation Committee adjusts...","{FREDERIC W. COOK, & CO., ROE}","{FREDERIC W. COOK, & CO., ROE}"
1,F:\PRUDENTIAL FINANCIAL INC\1137774-PRUDENTIAL...,PRUDENTIAL FINANCIAL INC,2008,1137774,PFH,[The Corporate Governance and Business Ethics ...,"{FREDERIC W. COOK &, ROE}","{Each year, the Compensation Committee adjusts...","{FREDERIC W. COOK & CO., INC., FREDERIC W. COO...","{FREDERIC W. COOK & CO., INC., FREDERIC W. COO..."
2,F:\PRUDENTIAL FINANCIAL INC\1137774-PRUDENTIAL...,PRUDENTIAL FINANCIAL INC,2009,1137774,PFH,[The Corporate Governance and Business Ethics ...,{FREDERIC W. COOK &},{The Committee sought the advice of Frederic W...,"{FREDERIC W. COOK & CO., INC., FREDERIC W. COO...","{FREDERIC W. COOK & CO., INC., FREDERIC W. COO..."
3,F:\PRUDENTIAL FINANCIAL INC\1137774-PRUDENTIAL...,PRUDENTIAL FINANCIAL INC,2010,1137774,PFH,[Be authorized to retain outside advisors and ...,{COOK},{The Committee has retained Frederic W. Cook &...,"{FREDERIC W. COOK & CO., INC.}","{FREDERIC W. COOK & CO., INC.}"
4,F:\PRUDENTIAL FINANCIAL INC\1137774-PRUDENTIAL...,PRUDENTIAL FINANCIAL INC,2011,1137774,PFH,[Mr. Casellas has been a consultant since 2010...,"{JAMES F. REDA & ASSOCIATES, LLC, COOK}",{The Compensation Committee's independent comp...,"{FREDERIC W. COOK & CO., INC., JAMES F. REDA &...","{FREDERIC W. COOK & CO., INC., JAMES F. REDA &..."
5,F:\PRUDENTIAL FINANCIAL INC\1137774-PRUDENTIAL...,PRUDENTIAL FINANCIAL INC,2012,1137774,PFH,[Because Prudential values your views concerni...,{COOK},{The Committee's independent compensation cons...,"{FREDERIC W. COOK & CO., INC.}","{FREDERIC W. COOK & CO., INC.}"
6,F:\PRUDENTIAL FINANCIAL INC\1137774-PRUDENTIAL...,PRUDENTIAL FINANCIAL INC,2013,1137774,PFH,"[Role of the Compensation Consultant, In order...",{COOK},{The Committee's independent compensation cons...,"{FREDERIC W. COOK & CO., INC.}","{FREDERIC W. COOK & CO., INC.}"
7,F:\PRUDENTIAL FINANCIAL INC\1137774-PRUDENTIAL...,PRUDENTIAL FINANCIAL INC,2014,1137774,PFH,"[Role of the Compensation Consultant, The Corp...","{JAMES F. REDA & ASSOCIATES, LLC, COOK}",{The Committee has retained Frederic W. Cook &...,"{FREDERIC W. COOK & CO., INC., JAMES F. REDA &...","{FREDERIC W. COOK & CO., INC., JAMES F. REDA &..."
8,F:\PRUDENTIAL FINANCIAL INC\1137774-PRUDENTIAL...,PRUDENTIAL FINANCIAL INC,2015,1137774,PFH,"[Role of the Compensation Consultant, In my ro...",{COOK},{The Committee has retained Frederic W. Cook &...,"{FREDERIC W. COOK & CO., INC.}","{FREDERIC W. COOK & CO., INC.}"
9,F:\PRUDENTIAL FINANCIAL INC\1137774-PRUDENTIAL...,PRUDENTIAL FINANCIAL INC,2016,1137774,PFH,"[Role of the Compensation Consultant, authoriz...",{COOK},{The Committee has retained Frederic W. Cook &...,"{FREDERIC W. COOK & CO., INC.}","{FREDERIC W. COOK & CO., INC.}"


In [96]:
df['CC_sentence'].iloc[4]

{'In 2010, the Committee engaged an independent compensation consultant, James F. Reda & Associates, LLC, to review the existing Director compensation program.',
 'The Committee has retained Frederic W. Cook & Co., Inc. as its executive compensation consultant.',
 "The Compensation Committee's independent compensation consultant, Frederic W. Cook & Co., Inc., is retained directly by the Committee and performs no other consulting or other services for us."}

In [97]:
# 21. F:\AGL RESOURCES INC

In [98]:
dir=r'F:\AGL RESOURCES INC'

In [99]:
df=process()

F:\AGL RESOURCES INC\1004155-AGL RESOURCES INC-DEF 14A-2007-03-19.html
F:\AGL RESOURCES INC\1004155-AGL RESOURCES INC-DEF 14A-2008-03-19.html
F:\AGL RESOURCES INC\1004155-AGL RESOURCES INC-DEF 14A-2009-03-16.html
F:\AGL RESOURCES INC\1004155-AGL RESOURCES INC-DEF 14A-2010-03-15.html
F:\AGL RESOURCES INC\1004155-AGL RESOURCES INC-DEF 14A-2011-03-14.html
F:\AGL RESOURCES INC\1004155-AGL RESOURCES INC-DEF 14A-2012-03-16.html
F:\AGL RESOURCES INC\1004155-AGL RESOURCES INC-DEF 14A-2013-03-15.html
F:\AGL RESOURCES INC\1004155-AGL RESOURCES INC-DEF 14A-2014-03-18.html
F:\AGL RESOURCES INC\1004155-AGL RESOURCES INC-DEF 14A-2015-03-17.html
F:\AGL RESOURCES INC\1004155-AGL RESOURCES INC-DEF 14A-2015-10-13.html
100%|██████████| 10/10 [00:00<00:00, 2499.14it/s]
filename is F:\AGL RESOURCES INC\1004155-AGL RESOURCES INC-DEF 14A-2007-03-19.html
filename is F:\AGL RESOURCES INC\1004155-AGL RESOURCES INC-DEF 14A-2008-03-19.html
filename is F:\AGL RESOURCES INC\1004155-AGL RESOURCES INC-DEF 14A-2009-03

In [100]:
df

,file_name,firm,year,cik,tic,text,CC2,CC_sentence,CC,CC1
0,F:\AGL RESOURCES INC\1004155-AGL RESOURCES INC...,AGL RESOURCES INC,2007,1004155,NaN,[The Compensation and Management Development C...,{TOWERS PERRIN},"{For 2006, the Committee engaged the services ...",{TOWERS PERRIN},{TOWERS PERRIN}
1,F:\AGL RESOURCES INC\1004155-AGL RESOURCES INC...,AGL RESOURCES INC,2008,1004155,NaN,[The Compensation and Management Development C...,"{JAMES, TOWERS PERRIN}","{For 2007, the Compensation Committee engaged ...","{JAMES, TOWERS PERRIN}","{JAMES, TOWERS PERRIN}"
2,F:\AGL RESOURCES INC\1004155-AGL RESOURCES INC...,AGL RESOURCES INC,2009,1004155,NaN,[The Compensation and Management Development C...,{TOWERS PERRIN},"{Instead, the Compensation Committee determine...",{TOWERS PERRIN},{TOWERS PERRIN}
3,F:\AGL RESOURCES INC\1004155-AGL RESOURCES INC...,AGL RESOURCES INC,2010,1004155,NaN,[The Compensation and Management Development C...,"{TOWERS PERRIN, COOK}",{In mid-2009 the Compensation Committee change...,"{F.W. COOK, F. W. COOK, TOWERS, PERRIN, FORSTE...","{F.W. COOK, F. W. COOK, TOWERS, PERRIN, FORSTE..."
4,F:\AGL RESOURCES INC\1004155-AGL RESOURCES INC...,AGL RESOURCES INC,2011,1004155,NaN,[The Compensation and Management Development C...,{COOK},{F.W. Cook's role is to provide expertise and ...,"{FREDERIC W. COOK & CO., INC., F.W. COOK, F.W....","{FREDERIC W. COOK & CO., INC., F.W. COOK, F.W...."
5,F:\AGL RESOURCES INC\1004155-AGL RESOURCES INC...,AGL RESOURCES INC,2012,1004155,NaN,[The Compensation and Management Development C...,{COOK},{F.W. Cook (Independent consultant to the Comp...,"{FREDERIC W. COOK & CO., INC., F.W. COOK, F.W....","{FREDERIC W. COOK & CO., INC., F.W. COOK, F.W...."
6,F:\AGL RESOURCES INC\1004155-AGL RESOURCES INC...,AGL RESOURCES INC,2013,1004155,NaN,[The Compensation and Management Development C...,{COOK},"{In December 2012, the Compensation Committee ...","{FREDERIC W. COOK & CO., INC., F.W. COOK, F.W....","{FREDERIC W. COOK & CO., INC., F.W. COOK, F.W...."
7,F:\AGL RESOURCES INC\1004155-AGL RESOURCES INC...,AGL RESOURCES INC,2014,1004155,NaN,[The Compensation and Management Development C...,"{JAMES, COOK}",{The following directors served on the Compens...,"{FREDERIC W. COOK & CO.,, F.W. COOK, JAMES, FR...","{FREDERIC W. COOK & CO.,, F.W. COOK, JAMES, FR..."
8,F:\AGL RESOURCES INC\1004155-AGL RESOURCES INC...,AGL RESOURCES INC,2015,1004155,NaN,[The Compensation Committee's charter provides...,"{JAMES, COOK}","{In determining base salary, the Compensation ...","{F.W. COOK, F.W. COOK (INDEPENDENT, FREDERIC W...","{F.W. COOK, JAMES, F.W. COOK (INDEPENDENT, FRE..."
9,F:\AGL RESOURCES INC\1004155-AGL RESOURCES INC...,AGL RESOURCES INC,2015,1004155,NaN,"[""), and (v) compensatory fringe benefit or st...",{},{},{},{}


In [101]:
df['CC_sentence'].iloc[1]

{'Accordingly, Towers Perrin was retained directly by the Compensation and Management Development Committee to assist it in 2007.',
 "Although benchmark levels of severance provided by Towers Perrin indicated that a three-times multiple was still the most prevalent level, at least for a chief executive officer and chief financial officer, management proposed to the Compensation Committee, and the Compensation Committee adopted, the proposal that no executive's agreement provide for a severance multiple greater than two.",
 'For 2007, the Compensation Committee engaged the services of Towers Perrin as its independent compensation consultant to benchmark base salary, target annual incentive opportunities and annual long-term incentive market rates from two market perspectives:',
 'Instead, the Compensation Committee determines individual compensation amounts in a deliberative process with input from Towers Perrin and our chief executive officer.',
 'The Compensation Committee developed t

# 22. F:\VERIZON COMMUNICATIONS INC

In [36]:
dir=r'F:\VERIZON COMMUNICATIONS INC'

In [37]:
df=process()

F:\VERIZON COMMUNICATIONS INC\732712-VERIZON COMMUNICATIONS INC-DEF 14A-2007-03-19.html
F:\VERIZON COMMUNICATIONS INC\732712-VERIZON COMMUNICATIONS INC-DEF 14A-2008-03-17.html
F:\VERIZON COMMUNICATIONS INC\732712-VERIZON COMMUNICATIONS INC-DEF 14A-2009-03-23.html
F:\VERIZON COMMUNICATIONS INC\732712-VERIZON COMMUNICATIONS INC-DEF 14A-2010-03-22.html
F:\VERIZON COMMUNICATIONS INC\732712-VERIZON COMMUNICATIONS INC-DEF 14A-2011-03-21.html
F:\VERIZON COMMUNICATIONS INC\732712-VERIZON COMMUNICATIONS INC-DEF 14A-2012-03-19.html
F:\VERIZON COMMUNICATIONS INC\732712-VERIZON COMMUNICATIONS INC-DEF 14A-2013-03-18.html
F:\VERIZON COMMUNICATIONS INC\732712-VERIZON COMMUNICATIONS INC-DEF 14A-2014-03-17.html
F:\VERIZON COMMUNICATIONS INC\732712-VERIZON COMMUNICATIONS INC-DEF 14A-2015-03-23.html
F:\VERIZON COMMUNICATIONS INC\732712-VERIZON COMMUNICATIONS INC-DEF 14A-2016-03-21.html
100%|██████████| 10/10 [00:00<00:00, 2500.63it/s]
[
Pearl Meyer and Partners
Pearl Meyer
Pearl Meyer & Partners
Hewitt A

In [40]:
df=process2(df)

('PEARL MEYER AND PARTNERS', '[')
('PEARL MEYER AND PARTNERS', 'PEARL MEYER')
('PEARL MEYER AND PARTNERS', 'PEARL MEYER & PARTNERS')
('PEARL MEYER AND PARTNERS', 'HEWITT ASSOCIATES LLP')
('[', 'PEARL MEYER')
('[', 'PEARL MEYER & PARTNERS')
('[', 'HEWITT ASSOCIATES LLP')
('PEARL MEYER', 'PEARL MEYER & PARTNERS')
('PEARL MEYER', 'HEWITT ASSOCIATES LLP')
('PEARL MEYER & PARTNERS', 'HEWITT ASSOCIATES LLP')
('PEARL MEYER & PARTNERS', 'STEVEN HALL')
('PEARL MEYER & PARTNERS', 'STEVEN HALL')
('PEARL MEYER', 'ERNST & YOUNG')


In [41]:
df

,file_name,firm,year,cik,tic,text,CC2,CC_sentence,CC,CC1
0,F:\VERIZON COMMUNICATIONS INC\732712-VERIZON C...,VERIZON COMMUNICATIONS INC,2007,732712,NaN,[Compensation Consultant Disclosure (Item 5 on...,"{[, PEARL MEYER, HEWITT ASSOCIATES LLP}","{The authors add that, ""Even if the CEO has no...","{PEARL MEYER AND PARTNERS, [, PEARL MEYER, PEA...","{PEARL MEYER AND PARTNERS, [, PEARL MEYER, PEA..."
1,F:\VERIZON COMMUNICATIONS INC\732712-VERIZON C...,VERIZON COMMUNICATIONS INC,2008,732712,NaN,[Role of the Compensation Consultant and Input...,{PEARL MEYER & PARTNERS},{The Committee has retained Pearl Meyer & Part...,{PEARL MEYER & PARTNERS},{PEARL MEYER & PARTNERS}
2,F:\VERIZON COMMUNICATIONS INC\732712-VERIZON C...,VERIZON COMMUNICATIONS INC,2009,732712,NaN,[Adopt a policy prohibiting the Committee's in...,"{PEARL MEYER & PARTNERS, STEVEN HALL}","{In compliance with the terms of this policy, ...","{PEARL MEYER & PARTNERS, STEVEN HALL}","{PEARL MEYER & PARTNERS, STEVEN HALL}"
3,F:\VERIZON COMMUNICATIONS INC\732712-VERIZON C...,VERIZON COMMUNICATIONS INC,2010,732712,NaN,[Adopt a policy prohibiting the Committee's in...,"{PEARL MEYER & PARTNERS, STEVEN HALL}",{The Committee has retained Pearl Meyer & Part...,"{PEARL MEYER & PARTNERS, STEVEN HALL}","{PEARL MEYER & PARTNERS, STEVEN HALL}"
4,F:\VERIZON COMMUNICATIONS INC\732712-VERIZON C...,VERIZON COMMUNICATIONS INC,2011,732712,NaN,[In order to ensure that the succession planni...,{PEARL MEYER & PARTNERS},{The Committee has retained Pearl Meyer & Part...,{PEARL MEYER & PARTNERS},{PEARL MEYER & PARTNERS}
5,F:\VERIZON COMMUNICATIONS INC\732712-VERIZON C...,VERIZON COMMUNICATIONS INC,2012,732712,NaN,[Role and Function of the Independent Compensa...,{PEARL MEYER & PARTNERS},{The Committee has retained Pearl Meyer & Part...,{PEARL MEYER & PARTNERS},{PEARL MEYER & PARTNERS}
6,F:\VERIZON COMMUNICATIONS INC\732712-VERIZON C...,VERIZON COMMUNICATIONS INC,2013,732712,NaN,[Role and Function of the Independent Compensa...,{PEARL MEYER & PARTNERS},"{At the request of the Committee, management a...",{PEARL MEYER & PARTNERS},{PEARL MEYER & PARTNERS}
7,F:\VERIZON COMMUNICATIONS INC\732712-VERIZON C...,VERIZON COMMUNICATIONS INC,2014,732712,NaN,"[Be solely responsible for retaining, terminat...",{PEARL MEYER & PARTNERS},"{In compliance with the terms of this policy, ...",{PEARL MEYER & PARTNERS},{PEARL MEYER & PARTNERS}
8,F:\VERIZON COMMUNICATIONS INC\732712-VERIZON C...,VERIZON COMMUNICATIONS INC,2015,732712,NaN,[Independent compensation consultant since 200...,{PEARL MEYER & PARTNERS},{The Committee has retained Pearl Meyer & Part...,{PEARL MEYER & PARTNERS},{PEARL MEYER & PARTNERS}
9,F:\VERIZON COMMUNICATIONS INC\732712-VERIZON C...,VERIZON COMMUNICATIONS INC,2016,732712,NaN,[ Independent compensation consultant sinc...,"{PEARL MEYER, ERNST & YOUNG}",{Neither Pearl Meyer nor its affiliates have p...,"{PEARL MEYER, ERNST & YOUNG}","{PEARL MEYER, ERNST & YOUNG}"


In [42]:
df['CC_sentence'].iloc[9]

{"In considering Ernst & Young's appointment for the 2016 fiscal year, the Committee reviewed the firm's qualifications and competencies, including the following factors:",
 'Neither Pearl Meyer nor its affiliates have performed any work for the Company or any Company affiliate since the Committee retained the Consultant in 2006.',
 'The Committee has concluded that no conflict of interest exists that would prevent Pearl Meyer from serving as an independent consultant to the Committee.',
 "The Committee has retained Pearl Meyer as its compensation consultant (Consultant) based on the firm's independence and expertise in representing the compensation committees of large corporations."}

In [107]:
# F:\TripAdvisor, Inc

In [43]:
dir=r'F:\TripAdvisor, Inc'
df=process()

F:\TripAdvisor, Inc\1526520-TripAdvisor, Inc.-DEF 14A-2012-04-30.html
F:\TripAdvisor, Inc\1526520-TripAdvisor, Inc.-DEF 14A-2013-05-13.html
F:\TripAdvisor, Inc\1526520-TripAdvisor, Inc.-DEF 14A-2014-04-30.html
F:\TripAdvisor, Inc\1526520-TripAdvisor, Inc.-DEF 14A-2015-04-28.html
F:\TripAdvisor, Inc\1526520-TripAdvisor, Inc.-DEF 14A-2016-04-29.html
100%|██████████| 5/5 [00:00<00:00, 1249.87it/s]
Compensia
Compensia
Compensia
Compensia
Compensia
Compensia
Compensia


In [44]:
df=process2(df)

In [45]:
df

,file_name,firm,year,cik,tic,text,CC2,CC_sentence,CC,CC1
0,"F:\TripAdvisor, Inc\1526520-TripAdvisor, Inc.-...","TripAdvisor, Inc",2012,1526520,NaN,[Under the NASDAQ Stock Market Listing Rules (...,{},{},{},{}
1,"F:\TripAdvisor, Inc\1526520-TripAdvisor, Inc.-...","TripAdvisor, Inc",2013,1526520,NaN,[The Board of Directors believes that stock op...,{},{},{},{}
2,"F:\TripAdvisor, Inc\1526520-TripAdvisor, Inc.-...","TripAdvisor, Inc",2014,1526520,NaN,"[Role of Compensation Consultant , Pursuant to...",{COMPENSIA},"{In addition, in compliance with the Compensat...",{COMPENSIA},{COMPENSIA}
3,"F:\TripAdvisor, Inc\1526520-TripAdvisor, Inc.-...","TripAdvisor, Inc",2015,1526520,NaN,"[Role of Compensation Consultant, Pursuant to ...",{},{},{},{}
4,"F:\TripAdvisor, Inc\1526520-TripAdvisor, Inc.-...","TripAdvisor, Inc",2016,1526520,NaN,[The Board of Directors believes that stock op...,{COMPENSIA},{Based on competitive market data provided by ...,{COMPENSIA},{COMPENSIA}


# F:\TYSON FOODS INC

In [114]:
dir=r'F:\TYSON FOODS INC'
df=process()

F:\TYSON FOODS INC\100493-TYSON FOODS INC-DEF 14A-2007-12-26.html
F:\TYSON FOODS INC\100493-TYSON FOODS INC-DEF 14A-2008-12-30.html
F:\TYSON FOODS INC\100493-TYSON FOODS INC-DEF 14A-2009-01-09.html
F:\TYSON FOODS INC\100493-TYSON FOODS INC-DEF 14A-2009-12-22.html
F:\TYSON FOODS INC\100493-TYSON FOODS INC-DEF 14A-2010-12-17.html
F:\TYSON FOODS INC\100493-TYSON FOODS INC-DEF 14A-2011-12-22.html
F:\TYSON FOODS INC\100493-TYSON FOODS INC-DEF 14A-2012-12-21.html
F:\TYSON FOODS INC\100493-TYSON FOODS INC-DEF 14A-2013-12-20.html
F:\TYSON FOODS INC\100493-TYSON FOODS INC-DEF 14A-2014-12-19.html
F:\TYSON FOODS INC\100493-TYSON FOODS INC-DEF 14A-2015-12-18.html
100%|██████████| 9/9 [00:00<00:00, 1810.14it/s]
filename is F:\TYSON FOODS INC\100493-TYSON FOODS INC-DEF 14A-2007-12-26.html
filename is F:\TYSON FOODS INC\100493-TYSON FOODS INC-DEF 14A-2008-12-30.html
filename is F:\TYSON FOODS INC\100493-TYSON FOODS INC-DEF 14A-2009-12-22.html
filename is F:\TYSON FOODS INC\100493-TYSON FOODS INC-DEF 

In [115]:
df

,file_name,firm,year,cik,tic,text,CC2,CC_sentence,CC,CC1
0,F:\TYSON FOODS INC\100493-TYSON FOODS INC-DEF ...,TYSON FOODS INC,2007,100493,TSN,"[Don Tyson, 77, served as Senior Chairman of t...","{HAY GROUP, HEWITT ASSOCIATES, INC.}","{Specifically, Hewitt Associates, Inc. provide...","{HAY GROUP, HEWITT ASSOCIATES, INC.}","{HAY GROUP, HEWITT ASSOCIATES, INC.}"
1,F:\TYSON FOODS INC\100493-TYSON FOODS INC-DEF ...,TYSON FOODS INC,2008,100493,TSN,"[Don Tyson, 78, served as Senior Chairman of t...","{HAY GROUP, HEWITT}",{Hewitt was instructed to provide the Compensa...,"{HAY GROUP, HEWITT ASSOCIATES, HEWITT}","{HEWITT ASSOCIATES, HAY GROUP, HEWITT}"
2,F:\TYSON FOODS INC\100493-TYSON FOODS INC-DEF ...,TYSON FOODS INC,2009,100493,TSN,"[Don Tyson, 79, served as Senior Chairman of t...","{HAY GROUP, HEWITT}","{In fiscal year 2009, the Compensation Committ...","{HAY GROUP, HEWITT ASSOCIATES, HEWITT}","{HEWITT ASSOCIATES, HAY GROUP, HEWITT}"
3,F:\TYSON FOODS INC\100493-TYSON FOODS INC-DEF ...,TYSON FOODS INC,2010,100493,TSN,"[Don Tyson, 80, served as Senior Chairman of t...","{HAY GROUP, HEWITT}",{Hay Group furnishes the data and analyses to ...,"{HAY GROUP, HEWITT ASSOCIATES, HEWITT}","{HEWITT ASSOCIATES, HAY GROUP, HEWITT}"
4,F:\TYSON FOODS INC\100493-TYSON FOODS INC-DEF ...,TYSON FOODS INC,2011,100493,TSN,"[Barbara A. Tyson, 62, served as Vice Presiden...","{HAY GROUP, TOWERS}","{In fiscal year 2011, the Compensation Committ...","{HAY GROUP, TOWERS WATSON, TOWERS}","{HAY GROUP, TOWERS WATSON, TOWERS}"
5,F:\TYSON FOODS INC\100493-TYSON FOODS INC-DEF ...,TYSON FOODS INC,2012,100493,TSN,"[Barbara A. Tyson, 63, served as Vice Presiden...",{HAY GROUP},{Hay Group furnishes the data and analyses to ...,{HAY GROUP},{HAY GROUP}
6,F:\TYSON FOODS INC\100493-TYSON FOODS INC-DEF ...,TYSON FOODS INC,2013,100493,TSN,"[Barbara A. Tyson, 64, served as Vice Presiden...",{HAY GROUP},"{The Compensation Committee, which worked with...",{HAY GROUP},{HAY GROUP}
7,F:\TYSON FOODS INC\100493-TYSON FOODS INC-DEF ...,TYSON FOODS INC,2014,100493,TSN,"[Barbara A. Tyson, 65, served as Vice Presiden...",{HAY GROUP},{Hay Group furnishes the data and analyses to ...,{HAY GROUP},{HAY GROUP}
8,F:\TYSON FOODS INC\100493-TYSON FOODS INC-DEF ...,TYSON FOODS INC,2015,100493,TSN,"[, served as Vice President of the Company unt...",{HAY GROUP},{Hay Group furnishes the data and analyses to ...,{HAY GROUP},{HAY GROUP}


In [49]:
dir=r'F:\CALPINE CORP'
df=process()

F:\CALPINE CORP\916457-CALPINE CORP-DEF 14A-2009-03-20.html
F:\CALPINE CORP\916457-CALPINE CORP-DEF 14A-2010-04-05.html
F:\CALPINE CORP\916457-CALPINE CORP-DEF 14A-2011-03-30.html
F:\CALPINE CORP\916457-CALPINE CORP-DEF 14A-2012-04-02.html
F:\CALPINE CORP\916457-CALPINE CORP-DEF 14A-2013-03-25.html
F:\CALPINE CORP\916457-CALPINE CORP-DEF 14A-2014-04-01.html
F:\CALPINE CORP\916457-CALPINE CORP-DEF 14A-2015-03-31.html
F:\CALPINE CORP\916457-CALPINE CORP-DEF 14A-2016-04-01.html
100%|██████████| 8/8 [00:00<00:00, 1987.00it/s]
Towers Perrin
Towers Perrin
Towers Perrin
Towers Perrin
Towers Perrin
Towers Perrin
Towers Perrin
Towers Perrin
Towers Perrin
Towers Perrin
Towers Perrin
Towers Perrin
Towers Perrin
Towers Perrin
Towers Perrin
Towers Perrin
Towers Perrin
Towers Perrin
Towers Watson
Towers Watson
Towers Perrin
Towers Watson
PSEG
Towers Watson
Towers Watson
Towers Perrin
Towers Watson
Pay Governance
Towers Watson
Pay Governance
Towers Watson
Pay Governance
PSEG
Pay Governance
Towers Wat

In [50]:
df=process2(df)

('TOWERS PERRIN', 'PSEG')
('TOWERS PERRIN', 'TOWERS WATSON')
('PSEG', 'TOWERS WATSON')
('PAY GOVERNANCE', 'TOWERS WATSON')
('PAY GOVERNANCE', 'PSEG')
('TOWERS WATSON', 'PSEG')
('MERIDIAN', 'MERIDIAN COMPENSATION PARTNERS, LLC')
('MERIDIAN', 'MERIDIAN COMPENSATION PARTNERS, LLC')
('MERIDIAN', 'MERIDIAN COMPENSATION PARTNERS, LLC')
('MERIDIAN', 'MERIDIAN COMPENSATION PARTNERS, LLC')


In [51]:
df

,file_name,firm,year,cik,tic,text,CC2,CC_sentence,CC,CC1
0,F:\CALPINE CORP\916457-CALPINE CORP-DEF 14A-20...,CALPINE CORP,2009,916457,NaN,"[Consultant, Consultant, iCRETE LLC, became a...",{TOWERS PERRIN},{Towers Perrin provided a summary of its asses...,{TOWERS PERRIN},{TOWERS PERRIN}
1,F:\CALPINE CORP\916457-CALPINE CORP-DEF 14A-20...,CALPINE CORP,2010,916457,NaN,"[Compensation Consultant. , The Chief Executiv...","{PSEG, TOWERS WATSON}",{As part of the process of evaluating our comp...,"{TOWERS PERRIN, PSEG, TOWERS WATSON}","{TOWERS PERRIN, PSEG, TOWERS WATSON}"
2,F:\CALPINE CORP\916457-CALPINE CORP-DEF 14A-20...,CALPINE CORP,2011,916457,NaN,"[Compensation Consultant, Compensation Consult...","{PAY GOVERNANCE, TOWERS WATSON, PSEG}",{As part of the process of evaluating our comp...,"{PAY GOVERNANCE, TOWERS WATSON, PSEG}","{PAY GOVERNANCE, TOWERS WATSON, PSEG}"
3,F:\CALPINE CORP\916457-CALPINE CORP-DEF 14A-20...,CALPINE CORP,2012,916457,NaN,"[Compensation Consultant, Compensation Consult...",{PAY GOVERNANCE},{As part of the process of evaluating our comp...,{PAY GOVERNANCE},{PAY GOVERNANCE}
4,F:\CALPINE CORP\916457-CALPINE CORP-DEF 14A-20...,CALPINE CORP,2013,916457,NaN,"[Compensation Consultant, The Equity Incentive...",{MERIDIAN},{While our advisor regularly consults with man...,"{MERIDIAN, MERIDIAN COMPENSATION PARTNERS, LLC...","{MERIDIAN, MERIDIAN COMPENSATION PARTNERS, LLC}"
5,F:\CALPINE CORP\916457-CALPINE CORP-DEF 14A-20...,CALPINE CORP,2014,916457,NaN,"[Compensation Consultant, Private Investor and...",{MERIDIAN},"{Based on Meridian’s input, effective with the...","{MERIDIAN, MERIDIAN COMPENSATION PARTNERS, LLC...","{MERIDIAN, MERIDIAN COMPENSATION PARTNERS, LLC}"
6,F:\CALPINE CORP\916457-CALPINE CORP-DEF 14A-20...,CALPINE CORP,2015,916457,NaN,"[Compensation Consultant, Private Investor and...",{MERIDIAN},{independence of Meridian pursuant to applicab...,"{MERIDIAN, MERIDIAN COMPENSATION PARTNERS, LLC...","{MERIDIAN, MERIDIAN COMPENSATION PARTNERS, LLC}"
7,F:\CALPINE CORP\916457-CALPINE CORP-DEF 14A-20...,CALPINE CORP,2016,916457,NaN,"[Compensation Consultant, Private Investor and...",{MERIDIAN},"{Since 2012, the Compensation Committee has re...","{MERIDIAN, MERIDIAN COMPENSATION PARTNERS, LLC...","{MERIDIAN, MERIDIAN COMPENSATION PARTNERS, LLC}"


In [54]:
df['CC_sentence'].iloc[4]

{'As a result, the Compensation Committee retained Meridian Compensation Partners, LLC (“Meridian”), a national compensation consulting firm, as its independent compensation advisor.',
 'Based on Meridian’s input, effective with the first performance unit awards in February 2013, the Committee approved modifications to the long-term incentive compensation program as further described elsewhere in this proxy statement.',
 'In addition, the Compensation Discussion and Analysis addresses the role of management and of the Compensation Committee’s independent compensation advisor, Meridian Compensation Partners, LLC, in determining and recommending executive compensation.',
 'Independent Compensation Consultant.',
 'Meridian also prepared analysis that compared the current level of compensation for our named executive officers to compensation paid to comparable positions at companies in an industry comparator group approved by the Compensation Committee.',
 'The Compensation Committee has a

In [61]:
pathList[338]

'f:\\COMMERCE BANCORP INC _NJ_'

In [67]:
dir=pathList[357]
df=process()

f:\CONSOL Energy Inc\1070412-CONSOL ENERGY INC-DEF 14A-2007-03-30.html
f:\CONSOL Energy Inc\1070412-CONSOL ENERGY INC-DEF 14A-2008-03-27.html
f:\CONSOL Energy Inc\1070412-CONSOL Energy Inc-DEF 14A-2009-03-23.html
f:\CONSOL Energy Inc\1070412-CONSOL Energy Inc-DEF 14A-2010-03-19.html
f:\CONSOL Energy Inc\1070412-CONSOL Energy Inc-DEF 14A-2011-03-28.html
f:\CONSOL Energy Inc\1070412-CONSOL Energy Inc-DEF 14A-2012-03-26.html
f:\CONSOL Energy Inc\1070412-CONSOL Energy Inc-DEF 14A-2013-03-29.html
f:\CONSOL Energy Inc\1070412-CONSOL Energy Inc-DEF 14A-2014-03-28.html
f:\CONSOL Energy Inc\1070412-CONSOL Energy Inc-DEF 14A-2015-03-25.html
f:\CONSOL Energy Inc\1070412-CONSOL Energy Inc-DEF 14A-2016-04-01.html
100%|██████████| 10/10 [00:00<00:00, 2501.08it/s]
Mercer Human Resource Consulting
Towers Perrin HR Services
Mercer Human Resource Consulting
Towers Perrin HR Services
Mercer Human Resource Consulting
Towers Perrin HR Services
Mercer (US) Inc.
Mercer
Deloitte Consulting LLP
Mercer (US) Inc

In [68]:
df=process2(df)

('TOWERS PERRIN HR SERVICES', 'MERCER HUMAN RESOURCE CONSULTING')
('DELOITTE CONSULTING LLP', 'MERCER (US) INC.')
('DELOITTE CONSULTING LLP', 'MERCER')
('DELOITTE CONSULTING LLP', 'DELOITTE')
('MERCER (US) INC.', 'MERCER')
('MERCER (US) INC.', 'DELOITTE')
('MERCER', 'DELOITTE')
('DELOITTE CONSULTING LLP', 'EOG RESOURCES, INC.')
('DELOITTE CONSULTING LLP', 'DELOITTE')
('EOG RESOURCES, INC.', 'DELOITTE')
('DELOITTE CONSULTING LLP', 'WATSON WYATT')
('DELOITTE CONSULTING LLP', 'MERCER')
('DELOITTE CONSULTING LLP', 'DELOITTE')
('WATSON WYATT', 'MERCER')
('WATSON WYATT', 'DELOITTE')
('MERCER', 'DELOITTE')
('DELOITTE TAX LLP', 'MERCER HUMAN RESOURCES CONSULTING')
('DELOITTE TAX LLP', 'DELOITTE')
('DELOITTE TAX LLP', 'DELOITTE CONSULTING LLP')
('DELOITTE TAX LLP', 'DELOITTE CONSULTING')
('MERCER HUMAN RESOURCES CONSULTING', 'DELOITTE')
('MERCER HUMAN RESOURCES CONSULTING', 'DELOITTE CONSULTING LLP')
('MERCER HUMAN RESOURCES CONSULTING', 'DELOITTE CONSULTING')
('DELOITTE', 'DELOITTE CONSULTING 

In [69]:
df

,file_name,firm,year,cik,tic,text,CC2,CC_sentence,CC,CC1
0,f:\CONSOL Energy Inc\1070412-CONSOL ENERGY INC...,CONSOL Energy Inc,2007,1070412,NaN,[oversight of the outside consultant engaged b...,"{TOWERS PERRIN HR SERVICES, MERCER HUMAN RESOU...","{In 2006, our Compensation Committee directly ...","{TOWERS PERRIN HR SERVICES, MERCER HUMAN RESOU...","{TOWERS PERRIN HR SERVICES, MERCER HUMAN RESOU..."
1,f:\CONSOL Energy Inc\1070412-CONSOL ENERGY INC...,CONSOL Energy Inc,2008,1070412,NaN,[oversight of the outside consultant engaged b...,"{MERCER, DELOITTE}",{The compensation committee of the board of di...,"{DELOITTE CONSULTING LLP, MERCER (US) INC., ME...","{DELOITTE CONSULTING LLP, MERCER (US) INC., ME..."
2,f:\CONSOL Energy Inc\1070412-CONSOL Energy Inc...,CONSOL Energy Inc,2009,1070412,NaN,[oversight of the outside consultant engaged b...,"{EOG RESOURCES, INC., DELOITTE}","{Applying this methodology, the consultant sug...","{DELOITTE CONSULTING LLP, EOG RESOURCES, INC.,...","{DELOITTE CONSULTING LLP, EOG RESOURCES, INC.,..."
3,f:\CONSOL Energy Inc\1070412-CONSOL Energy Inc...,CONSOL Energy Inc,2010,1070412,NaN,[oversight of the outside consultant engaged b...,{DELOITTE CONSULTING LLP},"{In 2009, our Compensation Committee directly ...",{DELOITTE CONSULTING LLP},{DELOITTE CONSULTING LLP}
4,f:\CONSOL Energy Inc\1070412-CONSOL Energy Inc...,CONSOL Energy Inc,2011,1070412,NaN,[overseeing the outside consultant engaged by ...,"{WATSON WYATT, MERCER, DELOITTE}",{Deloitte provided advice and/or recommendatio...,"{DELOITTE CONSULTING LLP, WATSON WYATT, MERCER...","{DELOITTE CONSULTING LLP, WATSON WYATT, MERCER..."
5,f:\CONSOL Energy Inc\1070412-CONSOL Energy Inc...,CONSOL Energy Inc,2012,1070412,NaN,"[Independent Management Consultant, overseeing...","{MERCER HUMAN RESOURCES CONSULTING, DELOITTE}",{Since we are unique in that we are both a maj...,"{DELOITTE TAX LLP, MERCER HUMAN RESOURCES CONS...","{DELOITTE TAX LLP, MERCER HUMAN RESOURCES CONS..."
6,f:\CONSOL Energy Inc\1070412-CONSOL Energy Inc...,CONSOL Energy Inc,2013,1070412,NaN,[Independent Management Consultant and Former ...,"{DELOITTE, MERCER US}",{the Deloitte consultants who provided service...,"{DELOITTE CONSULTING LLP, MERCER US, DELOITTE ...","{DELOITTE CONSULTING LLP, DELOITTE TAX LLP, DE..."
7,f:\CONSOL Energy Inc\1070412-CONSOL Energy Inc...,CONSOL Energy Inc,2014,1070412,NaN,[overseeing the outside compensation consultan...,"{TOWERS WATSON, DELOITTE}","{Before retaining Towers Watson, the Compensat...","{DELOITTE CONSULTING LLP, DELOITTE TAX LLP, DE...","{DELOITTE CONSULTING LLP, DELOITTE TAX LLP, TO..."
8,f:\CONSOL Energy Inc\1070412-CONSOL Energy Inc...,CONSOL Energy Inc,2015,1070412,NaN,[overseeing the outside compensation consultan...,{TOWERS WATSON},"{Before retaining Towers Watson, the Compensat...",{TOWERS WATSON},{TOWERS WATSON}
9,f:\CONSOL Energy Inc\1070412-CONSOL Energy Inc...,CONSOL Energy Inc,2016,1070412,NaN,[overseeing the outside compensation consultan...,{TOWERS WATSON},{Towers Watson did not provide any services to...,{TOWERS WATSON},{TOWERS WATSON}


In [73]:
df['CC_sentence'].iloc[7]


{'After such review, the Committee determined that the retention of the Towers Watson consulting team did not pose a conflict of interest.',
 'Before retaining Towers Watson, the Compensation Committee considered the factors set forth in the NYSE rules regarding the independence of advisors from management and other relevant factors.',
 'Deloitte provided advice and/or recommendations to the Compensation Committee regarding amounts and forms of executive compensation for 2013.',
 'In October 2013, the Compensation Committee retained Towers Watson to assist it with its evaluation of our compensation programs for executive officers and directors.',
 'Our Compensation Committee directly engaged Deloitte Consulting LLP ("Deloitte") through October 2013.',
 'The Compensation Committee engaged Deloitte and more recently Towers Watson to assist it with the development of our compensation program.',
 "The tax services were reviewed and approved by the Audit Committee and the Compensation Commi

In [81]:
dir=pathList[990]
os.chdir(dir)
    #os.getcwd()
proxies=find_proxies(dir)
data={}
pathList=[]
for i in proxies:
    filePath=os.path.join(dir, i)
    print(filePath)
    pathList.append(filePath) # create a list of file paths.
if pathList==[]:
    print('No DEF 14A in the folder')
else:    
    filename, text=open_proxy1(pathList)
    data.update({'file_name':filename, 'text':text})

f:\NEWS CORP\1308161-NEWS CORP-DEF 14A-2007-03-01.html
f:\NEWS CORP\1308161-NEWS CORP-DEF 14A-2007-09-06.html
f:\NEWS CORP\1308161-NEWS CORP-DEF 14A-2008-08-19.html
f:\NEWS CORP\1308161-NEWS CORP-DEF 14A-2009-08-20.html
f:\NEWS CORP\1308161-NEWS CORP-DEF 14A-2010-08-31.html
f:\NEWS CORP\1308161-NEWS CORP-DEF 14A-2011-09-02.html
f:\NEWS CORP\1308161-NEWS CORP-DEF 14A-2012-09-04.html
f:\NEWS CORP\1308161-NEWS CORP-DEF 14A-2013-04-30.html
f:\NEWS CORP\1564708-NEWS CORP-DEF 14A-2014-09-30.html
f:\NEWS CORP\1564708-NEWS CORP-DEF 14A-2015-09-01.html


In [83]:
f=pd.DataFrame()
df=pd.DataFrame.from_dict(data)
        # create columns with firm info
from tqdm.auto import tqdm
for i, row in tqdm(df.iterrows(), total=len(df)): # df.iterrows() will iterate each row
    filename = str(row["file_name"])  # row['column name'] to select certain column within a row
            #print('filename is', filename)
    firm, year, cik, tic= extract_firm_info(filename)
    print(firm, year, cik, tic)
    df.at[i, "firm"] = firm   # then, create a new column that contains the result of the operation.
    df.at[i, 'year'] = year
    df.at[i, 'cik'] = cik
    df.at[i, 'tic'] = tic

df['result']=df['text'].apply(NER2)
df['CC']=df['result'].apply(lambda x: x[0])
df['CC_sentence']=df['result'].apply(lambda x: x[1])


 80%|████████  | 8/10 [00:00<00:00, 1600.50it/s]NEWS CORP 2007 1308161 None
NEWS CORP 2007 1308161 None
NEWS CORP 2008 1308161 None
NEWS CORP 2009 1308161 None
NEWS CORP 2010 1308161 None
NEWS CORP 2011 1308161 None
NEWS CORP 2012 1308161 None
NEWS CORP 2013 1308161 None
NEWS CORP 2014 1564708 NWS



ValueError: could not convert string to float: 'NWS'

In [60]:
dir=pathList[1533]
dir

'f:\\ZIMMER HOLDINGS INC'

In [61]:
df=process(dir)

['1136869-ZIMMER HOLDINGS INC-DEF 14A-2002-03-26.html', '1136869-ZIMMER HOLDINGS INC-DEF 14A-2003-03-24.html', '1136869-ZIMMER HOLDINGS INC-DEF 14A-2004-03-22.html', '1136869-ZIMMER HOLDINGS INC-DEF 14A-2005-03-22.html', '1136869-ZIMMER HOLDINGS INC-DEF 14A-2006-03-22.html', '1136869-ZIMMER HOLDINGS INC-DEF 14A-2007-03-22.html', '1136869-ZIMMER HOLDINGS INC-DEF 14A-2008-03-20.html', '1136869-ZIMMER HOLDINGS INC-DEF 14A-2009-03-20.html', '1136869-ZIMMER HOLDINGS INC-DEF 14A-2010-03-19.html', '1136869-ZIMMER HOLDINGS INC-DEF 14A-2011-03-18.html', '1136869-ZIMMER HOLDINGS INC-DEF 14A-2012-03-23.html', '1136869-ZIMMER HOLDINGS INC-DEF 14A-2013-03-25.html', '1136869-ZIMMER HOLDINGS INC-DEF 14A-2014-03-24.html', '1136869-ZIMMER HOLDINGS INC-DEF 14A-2015-03-23.html']
f:\ZIMMER HOLDINGS INC\1136869-ZIMMER HOLDINGS INC-DEF 14A-2007-03-22.html
f:\ZIMMER HOLDINGS INC\1136869-ZIMMER HOLDINGS INC-DEF 14A-2008-03-20.html
f:\ZIMMER HOLDINGS INC\1136869-ZIMMER HOLDINGS INC-DEF 14A-2009-03-20.html
f:\Z

In [62]:
df=process2(df)

('WATSON WYATT & COMPANY', 'WATSON WYATT')
('WATSON', 'WYATT & COMPANY')
('WATSON', 'WATSON WYATT')
('WATSON', 'STEVEN HALL AND PARTNERS')
('WYATT & COMPANY', 'WATSON WYATT')
('WYATT & COMPANY', 'STEVEN HALL AND PARTNERS')
('WATSON WYATT', 'STEVEN HALL AND PARTNERS')
('STEVEN HALL AND PARTNERS', 'WATSON WYATT WORLDWIDE')
('STEVEN HALL AND PARTNERS', 'WATSON WYATT')
('STEVEN HALL AND PARTNERS', 'STEVEN HALL')
('STEVEN HALL AND PARTNERS', 'WYATT')
('STEVEN HALL AND PARTNERS', 'WATSON')
('WATSON WYATT WORLDWIDE', 'WATSON WYATT')
('WATSON WYATT WORLDWIDE', 'STEVEN HALL')
('WATSON WYATT WORLDWIDE', 'WYATT')
('WATSON WYATT WORLDWIDE', 'WATSON')
('WATSON WYATT', 'STEVEN HALL')
('WATSON WYATT', 'WYATT')
('WATSON WYATT', 'WATSON')
('STEVEN HALL', 'WYATT')
('STEVEN HALL', 'WATSON')
('WYATT', 'WATSON')
('WATSON WYATT WORLDWIDE', 'TOWERS WATSON & CO.')
('WATSON WYATT WORLDWIDE', 'TOWERS')
('WATSON WYATT WORLDWIDE', 'TOWERS WATSON')
('TOWERS WATSON & CO.', 'TOWERS')
('TOWERS WATSON & CO.', 'TOWERS 

In [63]:
df

,file_name,firm,year,cik,tic,text,CC2,CC_sentence,CC,CC1
0,f:\ZIMMER HOLDINGS INC\1136869-ZIMMER HOLDINGS...,ZIMMER HOLDINGS INC,2007,1136869,ZBH,"[Compensation Consultants, In the recent ...",{WATSON WYATT},"{Specifically, Watson Wyatt provided the commi...","{WATSON WYATT & COMPANY, WATSON WYATT}","{WATSON WYATT & COMPANY, WATSON WYATT}"
1,f:\ZIMMER HOLDINGS INC\1136869-ZIMMER HOLDINGS...,ZIMMER HOLDINGS INC,2008,1136869,ZBH,[ Management provides the committee with r...,"{WATSON, WYATT & COMPANY, STEVEN HALL AND PART...",{Steven Hall and Partners also assists the...,"{WATSON, WYATT & COMPANY, WATSON WYATT, STEVEN...","{WATSON, WYATT & COMPANY, WATSON WYATT, STEVEN..."
2,f:\ZIMMER HOLDINGS INC\1136869-ZIMMER HOLDINGS...,ZIMMER HOLDINGS INC,2009,1136869,ZBH,[ Management provides the committee with r...,"{STEVEN HALL, WYATT}",{Steven Hall and Partners also assists the hum...,"{STEVEN HALL AND PARTNERS, WATSON WYATT WORLDW...","{STEVEN HALL AND PARTNERS, WATSON WYATT WORLDW..."
3,f:\ZIMMER HOLDINGS INC\1136869-ZIMMER HOLDINGS...,ZIMMER HOLDINGS INC,2010,1136869,ZBH,[ When setting compensation for our execut...,"{WATSON WYATT WORLDWIDE, TOWERS}",{This amount was approved by the committee...,"{WATSON WYATT WORLDWIDE, TOWERS WATSON & CO., ...","{WATSON WYATT WORLDWIDE, TOWERS WATSON & CO., ..."
4,f:\ZIMMER HOLDINGS INC\1136869-ZIMMER HOLDINGS...,ZIMMER HOLDINGS INC,2011,1136869,ZBH,"[Consultant, Blackstone Healthcare Partners of...","{TOWERS, WATSON WYATT}",{At the request of the Compensation and Manage...,"{TOWERS WATSON & CO., TOWERS PERRIN, WATSON WY...","{TOWERS WATSON & CO., TOWERS PERRIN, WATSON WY..."
5,f:\ZIMMER HOLDINGS INC\1136869-ZIMMER HOLDINGS...,ZIMMER HOLDINGS INC,2012,1136869,ZBH,"[Consultant, Blackstone Healthcare Partners of...","{WATSON WYATT, TOWERS WATSON}","{Typically, Towers Watson attends committee me...","{TOWERS WATSON & CO., TOWERS PERRIN, WATSON WY...","{TOWERS WATSON & CO., TOWERS PERRIN, WATSON WY..."
6,f:\ZIMMER HOLDINGS INC\1136869-ZIMMER HOLDINGS...,ZIMMER HOLDINGS INC,2013,1136869,ZBH,"[Consultant, Blackstone Healthcare Partners of...","{CONSISTENT, WATSON WYATT, TOWERS WATSON}","{Consistent with past practice, the committee ...","{CONSISTENT, TOWERS WATSON & CO., TOWERS PERRI...","{CONSISTENT, TOWERS WATSON & CO., TOWERS PERRI..."
7,f:\ZIMMER HOLDINGS INC\1136869-ZIMMER HOLDINGS...,ZIMMER HOLDINGS INC,2014,1136869,ZBH,"[Consultant, Consultant, Blackstone Healthcare...","{CONSISTENT, WATSON WYATT, TOWERS WATSON}","{Consistent with past practice, the committee ...","{CONSISTENT, TOWERS WATSON & CO., TOWERS PERRI...","{CONSISTENT, TOWERS WATSON & CO., TOWERS PERRI..."
8,f:\ZIMMER HOLDINGS INC\1136869-ZIMMER HOLDINGS...,ZIMMER HOLDINGS INC,2015,1136869,ZBH,"[Consultant, Consultant, Blackstone Healthcare...","{WATSON WYATT, TOWERS WATSON}",{In addition to the market review of 2014 comp...,"{TOWERS WATSON & CO., TOWERS PERRIN, WATSON WY...","{TOWERS WATSON & CO., TOWERS PERRIN, WATSON WY..."


In [65]:
df['CC_sentence'].iloc[7]

{':  No Towers Watson team member has any business or personal relationship with any member of the committee.',
 'Annually, the committee is to receive a report of the total fees we paid to Towers Watson and its affiliates for executive or director compensation services and all other services.',
 "As a result of the merger of the Watson Wyatt and Towers Perrin consulting firms in 2010, our longstanding health and welfare consultants are now associated with the same firm as the committee's compensation consultant.",
 'At the joint request of the Corporate Governance Committee and the Compensation and Management Development Committee, in May 2013 Towers Watson & Co. ("Towers Watson") reviewed our non-employee director compensation program relative to market practices and proposed recommendations for adjustments.',
 'At the request of the Compensation and Management Development Committee, in 2010 Towers Watson completed a broad assessment of the risks associated with the compensation plan